# initialize

In [1]:
from __future__ import nested_scopes
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML('<style>.CodeMirror{font-family: "Courier New";font-size: 12pt;}</style>'))

In [2]:
import pyspark.sql.functions as F
import json
import builtins
from itertools import chain
import seaborn as sns


In [3]:
from pyspark.sql.types import (StructType, StructField, DateType,
    TimestampType, StringType, LongType, IntegerType, DoubleType,FloatType)

from pyspark.sql.functions import pandas_udf, PandasUDFType

from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import StringIndexer, VectorAssembler

from pyspark.sql.window import Window


In [4]:
import math
from functools import reduce
import re
import collections
from pyspark.ml import Pipeline
import pandas
import numpy
import time
from pandasql import sqldf

pandas.options.display.max_rows=50
pandas.options.display.max_columns=200
pandas.options.display.float_format = '{:,}'.format

In [5]:
from ipywidgets import IntProgress,Layout
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.lines as mlines


In [ ]:
import pyhdfs


fs = pyhdfs.HdfsClient(hosts='10.0.2.125:50070', user_name='yuzhou')

# monitor backup

In [13]:

import pyhdfs
fs = pyhdfs.HdfsClient(hosts='sr525:50070', user_name='yuzhou')


basedif="dgx-2"

emon_events='''
    -q -c -experimental -t0.5 -l100000 -u
    -C (

    INST_RETIRED.ANY
    CPU_CLK_UNHALTED.REF_TSC
    CPU_CLK_UNHALTED.THREAD
    CYCLE_ACTIVITY.STALLS_L3_MISS
    CYCLE_ACTIVITY.CYCLES_MEM_ANY
    CYCLE_ACTIVITY.STALLS_TOTAL
    DTLB_LOAD_MISSES.WALK_ACTIVE
    UNC_CHA_TOR_OCCUPANCY.IA_MISS:filter1=0x40432
    UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x40432
    UNC_CHA_CLOCKTICKS
    UNC_M_CAS_COUNT.RD
    UNC_M_CAS_COUNT.WR
    UNC_IIO_DATA_REQ_OF_CPU.MEM_WRITE.PART0
    UNC_IIO_DATA_REQ_OF_CPU.MEM_READ.PART0
    UNC_IIO_DATA_REQ_BY_CPU.MEM_WRITE.PART0
    UNC_IIO_DATA_REQ_BY_CPU.MEM_READ.PART0


    INST_RETIRED.ANY
    CPU_CLK_UNHALTED.REF_TSC
    CPU_CLK_UNHALTED.THREAD
    OFFCORE_RESPONSE:request=ALL_READS:response=L3_MISS.ANY_SNOOP:ocr_msr_val=0x3fB80007f7
    OFFCORE_RESPONSE:request=ALL_READS:response=L3_MISS.ANY_SNOOP:ocr_msr_val=0x3f840007f7
    MEM_LOAD_RETIRED.L3_MISS
    UOPS_ISSUED.STALL_CYCLES
    UNC_CHA_TOR_OCCUPANCY.IA_MISS:filter1=0x40431
    UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x40431
    UNC_CHA_CLOCKTICKS
    )
'''

def killsar(clients):
    for l in clients:
        out=!ssh $l "ps aux | grep -w sar | grep -v grep | tr -s ' ' | cut -d' ' -f2"
        for x in out:
            !ssh $l "kill $x > /dev/null 2>&1"
    for l in clients:
        out=!ssh $l "ps aux | grep -w pidstat | grep -v grep | tr -s ' ' | cut -d' ' -f2"
        for x in out:
            !ssh $l "kill $x > /dev/null 2>&1"
    for l in clients:
        out=!ssh $l "ps aux | grep -w perf | grep -v grep | tr -s ' ' | cut -d' ' -f2"
        for x in out:
            !ssh root@$l "kill $x > /dev/null 2>&1"
    for l in clients:
        out=!ssh $l "ps aux | grep -w nvidia-smi | grep -v grep | tr -s ' ' | cut -d' ' -f2"
        for x in out:
            !ssh root@$l "kill $x > /dev/null 2>&1"
    for l in clients:
        !ssh $l "source ~/sep_install/sep_vars.sh>/dev/null 2>&1; emon -stop > /dev/null 2>&1"
            
def startmonitor(clients,appid,**kwargs):
    local_profile_dir=home+"/profile/"
    prof=local_profile_dir+appid+"/"
    !mkdir -p $prof
    !echo "" > /tmp/training_time
    
    for l in clients:
        !ssh root@$l date
    
    killsar(clients)
    
    if kwargs.get("collect_emon",False):
        with open(home+"/emon.list",'w+') as f:
            f.write(emon_events)
        for l in clients:
            !scp {home}/emon.list {l}:{home}/emon.list  > /dev/null 2>&1
    
    perfsyscalls=kwargs.get("collect_perf_syscall",None)
    
    for l in clients:
        !mkdir -p $prof/$l/
        !ssh $l mkdir -p $prof/$l/
        !ssh $l "sar -o {prof}/{l}/sar.bin -r -u -d -n DEV 1 >/dev/null 2>&1 &"
        if kwargs.get("collect_pid",False):
            !ssh $l "{java_home}/bin/jps | grep CoarseGrainedExecutorBackend | head -n 1 | cut -d' ' -f 1 | xargs  -I % pidstat -h -t -p % 1  > {prof}/{l}/pidstat.out  2>/dev/null &"
        !ssh root@{l} 'cat /proc/uptime  | cut -d" " -f 1 | xargs -I ^ date -d "- ^ seconds"  +%s.%N' > $prof/$l/uptime.txt
        if kwargs.get("collect_sched",False):
            !ssh root@{l} 'perf trace -e "sched:sched_switch" -C 8-15 -o {prof}/{l}/sched.txt -T -- sleep 10000 >/dev/null 2>/dev/null &'
        if kwargs.get("collect_emon",False):
            !ssh {l} "source ~/sep_install/sep_vars.sh>/dev/null 2>&1; emon -i {home}/emon.list -f {prof}/{l}/emon.rst >/dev/null 2>&1 & "
        if perfsyscalls is not None:
            !ssh root@{l} "perf stat -e 'syscalls:sys_exit_poll,syscalls:sys_exit_epoll_wait' -a -I 1000 -o {prof}/{l}/perfstat.txt  >/dev/null 2>&1 & "
        if kwargs.get("collect_nv_smi",False):
            !ssh {l} "nvidia-smi --query-gpu=timestamp,index,utilization.gpu,utilization.memory,memory.used, -lms 500  --format=csv -f  {prof}/{l}/gpu.txt >/dev/null 2>&1 & "
    return prof

def stopmonitor(clients, sc, appid,**kwargs):
    %cd ~
    
    local_profile_dir=home+"/profile/"
    prof=local_profile_dir+appid+"/"
    !mkdir -p $prof
    
    killsar(clients)
    
    with open(f"{prof}/starttime","w"):
        f.write(time()*1000)
    
    for l in clients:
        !ssh $l "sar -f {prof}/{l}/sar.bin -r > {prof}/{l}/sar_mem.sar;sar -f {prof}/{l}/sar.bin -u > {prof}/{l}/sar_cpu.sar;sar -f {prof}/{l}/sar.bin -d > {prof}/{l}/sar_disk.sar;sar -f {prof}/{l}/sar.bin -n DEV > {prof}/{l}/sar_nic.sar;" 
        !ssh $l "grep -rI xgbtck --no-filename {spark_home}/work/{appid}/* | sed 's/^ //g'  > {prof}/{l}/xgbtck.txt"
        if l!= socket.gethostname():
            !scp -r $l:$prof/$l $prof/ > /dev/null 2>&1
        !ssh $l "{java_home}/bin/jps | grep CoarseGrainedExecutorBackend | head -n 2 | tail -n 1 | cut -d' ' -f 1  | xargs -I % ps -To tid p %" > $prof/$l/sched_threads.txt
        !ssh $l "source ~/sep_install/sep_vars.sh>/dev/null 2>&1; emon -v " > $prof/$l/emonv.txt
        !ssh $l "sar -V " > $prof/$l/sarv.txt
        !test -f $prof/$l/perfstat.txt && head -n 1 $prof/$l/perfstat.txt > $prof/$l/perfstarttime
    if sc is not None:
        sc.stop()
    
    xgbfiles=!find {home}/profile/{appid} | grep xgbtck
    !cat /tmp/training_time >> {xgbfiles[0]}
    
#    !hadoop fs -copyToLocal /tmp/sparkEventLog/$appid $prof/app.log
#    !cp {home}/spark_events/{appid}  $prof/app.log
    fs.mkdirs(f"/{basedif}/")
    v=[os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(local_profile_dir+appid)) for f in fn]
    for f in v:
        paths=os.path.split(f)
        fs.mkdirs("/"+basedif+"/"+paths[0][len(local_profile_dir):])
        fs.copy_from_local(f,"/"+basedif+"/"+paths[0][len(local_profile_dir):]+"/"+paths[1],overwrite=True)
#    !HADOOP_USER_NAME=yuzhou hadoop fs -put  file://{prof} hdfs://10.1.2.125:8020/{basedif}
    print('http://10.0.0.34:18080/history/{:s}/jobs'.format(appid))
    print('http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/{:s}.json'.format(appid))
    print(appid)

In [14]:
def pinexecutor(clients):
    for client in clients:
        pids=!ssh $client "jps | grep CoarseGrainedExecutorBackend | cut -d' ' -f1"
        print(client,":",len(pids)," ","\t".join(map(str,pids)))
        t1=[l for l in range(0,cores_per_executor)]
        t2=[executors_per_node*2+l for l in range(0,cores_per_executor)]
        node1=True
        for l in pids:
            t=t1 if node1==True else t2
            cpus=','.join(map(str, t))
            for i in range(0,cores_per_executor):
                t[i]=t[i]+cores_per_executor
            node1= not node1
            !ssh $client "taskset -a -p -c $cpus $l > /dev/null 2>&1 "


# base class

In [11]:
class SparkLog_Analysis:
    def __init__(self, appid,jobids,clients):
        pass

In [15]:
class Analysis:
    def __init__(self,file):
        self.file=file
        self.starttime=0
        self.df=None
    
    def load_data(self):
        pass
    
    def generate_trace_view_list(self,id=0, **kwargs):
        if self.df==None:
            self.load_data()
        trace_events=[]
        node=kwargs.get('node',"node")
        trace_events.append(json.dumps({"name": "process_name","ph": "M","pid":id,"tid":0,"args":{"name":" "+node}}))
        return trace_events

    
    def generate_trace_view(self, trace_output, **kwargs):
        traces=[]
        traces.extend(self.generate_trace_view_list(0,**kwargs))
        
        output='''
        {
            "traceEvents": [
        
        ''' + \
        ",\n".join(traces)\
       + '''
            ],
            "displayTimeUnit": "ns"
        }'''

        with open('/home/yuzhou/trace_result/'+trace_output+'.json', 'w') as outfile: 
            outfile.write(output)

        print("http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/"+trace_output+".json")
        

# EMON process

## emon metric

In [6]:
metrics='''<root>

    <metric name="metric_CPI">
        <throughput-metric-name>metric_cycles per txn</throughput-metric-name>
        <event alias="a">CPU_CLK_UNHALTED.THREAD</event>
        <event alias="b">INST_RETIRED.ANY</event>
        <formula>a/b</formula>
    </metric>

 	<metric name="metric_branch mispredict ratio">
		<event alias="a">BR_MISP_RETIRED.ALL_BRANCHES</event>
		<event alias="b">BR_INST_RETIRED.ALL_BRANCHES</event>
		<formula>a/b</formula>
	</metric>

	<metric name="metric_loads per instr">
        <throughput-metric-name>metric_loads per txn</throughput-metric-name>
		<event alias="a">MEM_INST_RETIRED.ALL_LOADS</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>

	<metric name="metric_stores per instr">
        <throughput-metric-name>metric_stores per txn</throughput-metric-name>
		<event alias="a">MEM_INST_RETIRED.ALL_STORES</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>	

	<metric name="metric_locks retired per instr">
        <throughput-metric-name>metric_locks retired per txn</throughput-metric-name>
		<event alias="a">MEM_INST_RETIRED.LOCK_LOADS</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>	

	<metric name="metric_uncacheable reads per instr">
        <throughput-metric-name>metric_uncacheable reads per txn</throughput-metric-name>
		<event alias="a">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x40e33</event>
		<event alias="c">INST_RETIRED.ANY</event>
		<formula>a/c</formula>
	</metric>	

	<metric name="metric_streaming stores (full line) per instr">
        <throughput-metric-name>metric_streaming stores (full line) per txn</throughput-metric-name>
		<event alias="a">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x41833</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>	

	<metric name="metric_streaming stores (partial line) per instr">
        <throughput-metric-name>metric_streaming stores (partial line) per txn</throughput-metric-name>
		<event alias="a">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x41a33</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>	

	<metric name="metric_L1D MPI (includes data+rfo w/ prefetches)">
        <throughput-metric-name>metric_L1D misses per txn (includes data+rfo w/ prefetches)</throughput-metric-name>
		<event alias="a">L1D.REPLACEMENT</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>

	<metric name="metric_L1D demand data read hits per instr">
        <throughput-metric-name>metric_L1D demand data read hits per txn</throughput-metric-name>
		<event alias="a">MEM_LOAD_RETIRED.L1_HIT</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>

	<metric name="metric_L1-I code read misses (w/ prefetches) per instr">
        <throughput-metric-name>metric_L1I code read misses (includes prefetches) per txn</throughput-metric-name>
		<event alias="a">L2_RQSTS.ALL_CODE_RD</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>

	<metric name="metric_L2 demand data read hits per instr">
        <throughput-metric-name>metric_L2 demand data read hits per txn</throughput-metric-name>
		<event alias="a">MEM_LOAD_RETIRED.L2_HIT</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>

	<metric name="metric_L2 MPI (includes code+data+rfo w/ prefetches)">
        <throughput-metric-name>metric_L2 misses per txn (includes code+data+rfo w/ prefetches)</throughput-metric-name>
		<event alias="a">L2_LINES_IN.ALL</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>
	
 	<metric name="metric_L2 demand data read MPI">
        <throughput-metric-name>metric_L2 demand data read misses per txn</throughput-metric-name>
		<event alias="a">MEM_LOAD_RETIRED.L2_MISS</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>
	
	<metric name="metric_L2 demand code MPI">
        <throughput-metric-name>metric_L2 demand code misses per txn</throughput-metric-name>
		<event alias="a">L2_RQSTS.CODE_RD_MISS</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>

	<metric name="metric_L2 Any local request that HITM in a sibling core (per instr)">
        <throughput-metric-name>metric_L2 Any local request that HITM in a sibling core per txn</throughput-metric-name>
		<event alias="a">OFFCORE_RESPONSE:request=ALL_READS:response=L3_HIT.HITM_OTHER_CORE</event>
		<event alias="c">INST_RETIRED.ANY</event>
		<formula>a/c</formula>
	</metric>	

	<metric name="metric_L2 Any local request that HIT in a sibling core and forwarded(per instr)">
        <throughput-metric-name>metric_L2 Any local request that HIT in a sibling core and forwarded per txn</throughput-metric-name>
		<event alias="a">OFFCORE_RESPONSE:request=ALL_READS:response=L3_HIT.HIT_OTHER_CORE_FWD</event>
		<event alias="c">INST_RETIRED.ANY</event>
		<formula>a/c</formula>
	</metric>

	<metric name="metric_L2 all L2 prefetches(per instr)">
        <throughput-metric-name>metric_L2 all L2 prefetches per txn</throughput-metric-name>
		<event alias="a">L2_RQSTS.ALL_PF</event>
		<event alias="c">INST_RETIRED.ANY</event>
		<formula>a/c</formula>
	</metric>	
	
	<metric name="metric_L2 % of L2 evictions that are allocated into L3">
		<event alias="a">L2_LINES_OUT.NON_SILENT</event>
		<event alias="b">IDI_MISC.WB_DOWNGRADE</event>
		<formula>100*(a-b)/a</formula>
	</metric>

	<metric name="metric_L2 % of L2 evictions that are NOT allocated into L3">
		<event alias="a">L2_LINES_OUT.NON_SILENT</event>
		<event alias="b">IDI_MISC.WB_DOWNGRADE</event>
		<formula>100*b/a</formula>
	</metric>

	<metric name="metric_LLC code references per instr (L3 prefetch excluded)">
        <throughput-metric-name>metric_LLC code references per txn (L3 prefetch excluded)</throughput-metric-name>
		<event alias="a">UNC_CHA_TOR_INSERTS.IA:filter1=0x40233</event>
		<event alias="d">INST_RETIRED.ANY</event>
		<formula>a/d</formula>
	</metric>

	<metric name="metric_LLC data read references per instr (L3 prefetch excluded)">
        <throughput-metric-name>metric_LLC data read references per txn (L3 prefetch excluded)</throughput-metric-name>
		<event alias="a">UNC_CHA_TOR_INSERTS.IA:filter1=0x40433</event>
		<event alias="d">INST_RETIRED.ANY</event>
		<formula>a/d</formula>
	</metric>

	<metric name="metric_LLC RFO references per instr (L3 prefetch excluded)">
        <throughput-metric-name>metric_LLC RFO references per txn (L3 prefetch excluded)</throughput-metric-name>
		<event alias="a">UNC_CHA_TOR_INSERTS.IA:filter1=0x40033</event>
		<event alias="d">INST_RETIRED.ANY</event>
		<formula>a/d</formula>
	</metric>
	
	<metric name="metric_LLC MPI (includes code+data+rfo w/ prefetches)">
        <throughput-metric-name>metric_LLC misses per txn (includes code+data+rfo w/ prefetches)</throughput-metric-name>
		<!-- L3 misses of L2 requests and L3 prefetches for data read -->
		<event alias="a">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12D40433</event>
		<!-- L3 misses of L2 requests + L3 prefetches for code read -->
		<event alias="b">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12CC0233</event>
		<!-- L3 misses of L2 requests + L3 prefetches for RFO -->
		<event alias="c">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12C40033</event>
		<event alias="d">INST_RETIRED.ANY</event>
		<formula>(a+b+c)/d</formula>
	</metric>

	<metric name="metric_LLC data read MPI (demand+prefetch)">
        <throughput-metric-name>metric_LLC data read (demand+prefetch) misses per txn</throughput-metric-name>
		<event alias="a">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12D40433</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>

	<metric name="metric_LLC RFO read MPI (demand+prefetch)">
        <throughput-metric-name>metric_LLC RFO read (demand+prefetch) misses per txn</throughput-metric-name>
		<event alias="a">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12C40033</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>

	<metric name="metric_LLC code read MPI (demand+prefetch)">
        <throughput-metric-name>metric_LLC code read (demand+prefetch) misses per txn</throughput-metric-name>
		<event alias="a">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12CC0233</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>

	<metric name="metric_LLC all LLC prefetches (per instr)">
        <throughput-metric-name>metric_LLC LLC prefetches per txn</throughput-metric-name>
		<event alias="a">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12C4B433</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>
	
	<metric name="metric_LLC total HITM (per instr) (excludes LLC prefetches)">
        <throughput-metric-name>metric_LLC total HITM per txn (excludes LLC prefetches)</throughput-metric-name>
		<event alias="a">OFFCORE_RESPONSE:request=ALL_READS:response=L3_MISS.REMOTE_HITM</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>

	<metric name="metric_LLC total HIT clean line forwards (per instr) (excludes LLC prefetches)">
        <throughput-metric-name>metric_LLC total HIT clean line forwards per txn (excludes LLC prefetches)</throughput-metric-name>
		<event alias="a">OFFCORE_RESPONSE:request=ALL_READS:response=L3_MISS.REMOTE_HIT_FORWARD</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>
	
	<metric name="metric_LLC % of LLC misses satisfied by remote caches">
		<!-- L3 misses of L2 requests and L3 prefetches for data read -->
		<event alias="a">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12D40433</event>
		<!-- L3 misses of L2 requests + L3 prefetches for code read -->
		<event alias="b">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12CC0233</event>
		<!-- L3 misses of L2 requests + L3 prefetches for RFO -->
		<event alias="c">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12C40033</event>
		<!-- L3 misses of L3 prefetches only -->
		<event alias="d">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12C4B433</event>
		<event alias="e">OFFCORE_RESPONSE:request=ALL_READS:response=L3_MISS.REMOTE_HIT_FORWARD</event>
		<event alias="f">OFFCORE_RESPONSE:request=ALL_READS:response=L3_MISS.REMOTE_HITM</event>
		<formula>100*(e+f)/(a+b+c-d)</formula>
	</metric>


	<metric name="metric_SF snoop filter capacity evictions (per instr)">
        <throughput-metric-name>metric_SF snoop filter capacity evictions per txn</throughput-metric-name>
		<event alias="a">UNC_CHA_SF_EVICTION.M_STATE</event>
		<event alias="b">UNC_CHA_SF_EVICTION.S_STATE</event>
		<event alias="c">UNC_CHA_SF_EVICTION.E_STATE</event>
		<event alias="d">INST_RETIRED.ANY</event>
		<formula>(a+b+c)/d</formula>
	</metric>

	<metric name="metric_SF % of L3 accesses that result in SF capacity evictions">
        <throughput-metric-name>metric_SF snoop filter capacity evictions per txn</throughput-metric-name>
		<event alias="a">UNC_CHA_SF_EVICTION.M_STATE</event>
		<event alias="b">UNC_CHA_SF_EVICTION.S_STATE</event>
		<event alias="c">UNC_CHA_SF_EVICTION.E_STATE</event>
		<event alias="d">L2_LINES_IN.ALL</event>
		<formula>100*(a+b+c)/d</formula>
	</metric>

	<metric name="metric_ITLB MPI">
        <throughput-metric-name>metric_ITLB misses per txn</throughput-metric-name>
		<event alias="a">ITLB_MISSES.WALK_COMPLETED</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>

	<metric name="metric_ITLB large page MPI">
        <throughput-metric-name>metric_ITLB large page misses per txn</throughput-metric-name>
		<event alias="a">ITLB_MISSES.WALK_COMPLETED_2M_4M</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>

	<metric name="metric_DTLB load MPI">
        <throughput-metric-name>metric_DTLB load misses per txn</throughput-metric-name>
		<event alias="a">DTLB_LOAD_MISSES.WALK_COMPLETED</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>

	<metric name="metric_DTLB 4KB page load MPI">
        <throughput-metric-name>metric_DTLB 4KB page load misses per txn</throughput-metric-name>
		<event alias="a">DTLB_LOAD_MISSES.WALK_COMPLETED</event>
		<event alias="b">DTLB_LOAD_MISSES.WALK_COMPLETED_2M_4M</event>
		<event alias="c">DTLB_LOAD_MISSES.WALK_COMPLETED_1G</event>
		<event alias="d">INST_RETIRED.ANY</event>
		<formula>(a-b-c)/d</formula>
	</metric>
	
	<metric name="metric_DTLB 2MB large page load MPI">
        <throughput-metric-name>metric_DTLB 2MB large page load misses per txn</throughput-metric-name>
		<event alias="a">DTLB_LOAD_MISSES.WALK_COMPLETED_2M_4M</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>

	<metric name="metric_DTLB 1GB large page load MPI">
        <throughput-metric-name>metric_DTLB 1GB large page load misses per txn</throughput-metric-name>
		<event alias="a">DTLB_LOAD_MISSES.WALK_COMPLETED_1G</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>
	
	<metric name="metric_DTLB store MPI">
        <throughput-metric-name>metric_DTLB store misses per txn</throughput-metric-name>
		<event alias="a">DTLB_STORE_MISSES.WALK_COMPLETED</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>

	<metric name="metric_DTLB load miss latency (in core clks)">
		<event alias="a">DTLB_LOAD_MISSES.WALK_ACTIVE</event>
		<event alias="b">DTLB_LOAD_MISSES.WALK_COMPLETED</event>
		<formula>a/b</formula>
	</metric>

	<metric name="metric_DTLB store miss latency (in core clks)">
		<event alias="a">DTLB_STORE_MISSES.WALK_ACTIVE</event>
		<event alias="b">DTLB_STORE_MISSES.WALK_COMPLETED</event>
		<formula>a/b</formula>
	</metric>

	<metric name="metric_ITLB miss latency (in core clks)">
		<event alias="a">ITLB_MISSES.WALK_ACTIVE</event>
		<event alias="b">ITLB_MISSES.WALK_COMPLETED</event>
		<formula>a/b</formula>
	</metric>

	<metric name="metric_NUMA %_Reads addressed to local DRAM">
		<event alias="a">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x40432</event>
		<event alias="b">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x40431</event>
		<formula>100*a/(a+b)</formula>
	</metric>

	<metric name="metric_NUMA %_Reads addressed to remote DRAM">
		<event alias="a">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x40432</event>
		<event alias="b">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x40431</event>
		<formula>100*b/(a+b)</formula>
	</metric>
	
	<metric name="metric_NUMA %_RFOs addressed to local DRAM">
		<event alias="a">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12C40031</event>
		<event alias="b">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12C40033</event>
		<formula>100*(b-a)/b</formula>
	</metric>

	<metric name="metric_NUMA %_RFOs addressed to remote DRAM">
		<event alias="a">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12C40031</event>
		<event alias="b">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12C40033</event>
		<formula>100*a/b</formula>
	</metric>
    
	<metric name="metric_uncore frequency GHz">
        <event alias="a">UNC_CHA_CLOCKTICKS</event>
        <constant alias="b">system.sockets[0].cores.count</constant>
        <constant alias="socket_count">2</constant>
        <formula>a/(b*socket_count)/1000000000</formula>
        <formula socket="0">a[0]/b/1000000000</formula>
        <formula socket="1">a[1]/b/1000000000</formula>
    </metric>

     <metric name="metric_UPI speed - GT/s">
        <constant alias="socket_count">2</constant>
        <constant alias="links_per_socket">2</constant>
        <event alias="a">UNC_UPI_CLOCKTICKS</event>
        <event alias="tsc">TSC</event>
        <event alias="f">UNC_UPI_L1_POWER_CYCLES</event>
        <event alias="c6">MSR_EVENT:msr=0x3F9:type=FREERUN:scope=PACKAGE</event>
       <constant alias="d">system.sockets[0].cores.count</constant>
  		<constant alias="threads">system.sockets[0][0].size</constant>      
        <formula>(((tsc/(d*threads)) / ((tsc/(d*threads)) - c6))*(a-f))  /(socket_count*links_per_socket)*8/1000000000</formula>
        <formula socket="0">((tsc/(d*threads)) / ((tsc/(d*threads)) - c6[0])*(a[0]-f[0]))/links_per_socket*8/1000000000</formula>
        <formula socket="1">((tsc/(d*threads)) / ((tsc/(d*threads)) - c6[1])*(a[1]-f[1]))/links_per_socket*8/1000000000</formula>
    </metric>

    <metric name="metric_UPI Data transmit BW (MB/sec) (only data)">
        <event alias="a">UNC_UPI_TxL_FLITS.ALL_DATA</event>
       <!-- 9 flits are needed to transmit a full cache line -->
        <formula>a*(64/9)/1000000</formula>
    </metric>

    <metric name="metric_UPI Transmit utilization_% (includes control)">
       <constant alias="socket_count">2</constant>
        <constant alias="links_per_socket">2</constant>
        <event alias="a">UNC_UPI_CLOCKTICKS</event>
        <event alias="tsc">TSC</event>
        <event alias="f">UNC_UPI_L1_POWER_CYCLES</event>
        <event alias="c6">MSR_EVENT:msr=0x3F9:type=FREERUN:scope=PACKAGE</event>
        <event alias="g">UNC_UPI_TxL_FLITS.ALL_DATA</event>
        <event alias="h">UNC_UPI_TxL_FLITS.NON_DATA</event>
       <constant alias="d">system.sockets[0].cores.count</constant>
  		<constant alias="threads">system.sockets[0][0].size</constant>      
        <!-- 5 flits are sent in every 6 clocks; NULL flits are sent across all 3 slots and hence b/3 -->
		<!-- So useful flits (non-idle) == (clocks*5/6) - (null_flits/3) -->	
        <formula>100*((g+h)/3)/(((((tsc/(d*threads)) / ((tsc/(d*threads)) - c6))*(a-f))*5/6))</formula>
    </metric>

    <metric name="metric_UPI % cycles transmit link is half-width (L0p)">
        <event alias="a">UNC_UPI_TxL0P_POWER_CYCLES</event>
        <event alias="b">UNC_UPI_CLOCKTICKS</event>
        <event alias="f">UNC_UPI_L1_POWER_CYCLES</event>
        <formula>100*(a/(b-f))</formula>
    </metric>

    <metric name="metric_UPI % cycles receive link is half-width (L0p)">
        <event alias="a">UNC_UPI_RxL0P_POWER_CYCLES</event>
        <event alias="b">UNC_UPI_CLOCKTICKS</event>
        <event alias="f">UNC_UPI_L1_POWER_CYCLES</event>
        <formula>100*(a/(b-f))</formula>
    </metric>

    <metric name="metric_HA - Reads vs. all requests">
      <event alias="a">UNC_CHA_REQUESTS.READS</event>
      <event alias="b">UNC_CHA_REQUESTS.WRITES</event>
      <formula>a/(a+b)</formula>
    </metric>

    <metric name="metric_HA - Writes vs. all requests">
      <event alias="a">UNC_CHA_REQUESTS.READS</event>
      <event alias="b">UNC_CHA_REQUESTS.WRITES</event>
      <formula>b/(a+b)</formula>
    </metric>

    <metric name="metric_HA % of all reads that are local">
      <event alias="a">UNC_CHA_REQUESTS.READS_LOCAL</event>
      <event alias="b">UNC_CHA_REQUESTS.READS</event>
      <formula>100*a/b</formula>
    </metric>

    <metric name="metric_HA % of all writes that are local">
      <event alias="a">UNC_CHA_REQUESTS.WRITES_LOCAL</event>
      <event alias="b">UNC_CHA_REQUESTS.WRITES</event>
      <formula>100*a/b</formula>
    </metric>

    <metric name="metric_HA conflict responses per instr">
        <throughput-metric-name>metric_HA conflict responses per txn</throughput-metric-name>
      <event alias="a">UNC_CHA_SNOOP_RESP.RSPCNFLCTS</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
    </metric>

    <metric name="metric_HA directory lookups that spawned a snoop (per instr)">
        <throughput-metric-name>metric_HA directory lookups that spawned a snoop (per txn)</throughput-metric-name>
      <event alias="a">UNC_CHA_DIR_LOOKUP.SNP</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
    </metric>

    <metric name="metric_HA directory lookups that did not spawn a snoop (per instr)">
        <throughput-metric-name>metric_HA directory lookups that did not spawn a snoop (per txn)</throughput-metric-name>
      <event alias="a">UNC_CHA_DIR_LOOKUP.NO_SNP</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
    </metric>

    <metric name="metric_M2M directory updates (per instr)">
        <throughput-metric-name>metric_M2M directory updates (per txn)</throughput-metric-name>
		<event alias="a">UNC_CHA_DIR_UPDATE.HA</event>
		<event alias="b">UNC_CHA_DIR_UPDATE.TOR</event>
		<event alias="c">UNC_M2M_DIRECTORY_UPDATE.ANY</event>
		<event alias="d">INST_RETIRED.ANY</event>
		<formula>(a+b+c)/d</formula>
    </metric>

    <metric name="metric_M2M XPT prefetches (per instr)">
        <throughput-metric-name>metric_M2M XPT prefetches (per txn)</throughput-metric-name>
		<event alias="a">UNC_M2M_PREFCAM_INSERTS</event>
		<event alias="b">UNC_M3UPI_UPI_PREFETCH_SPAWN</event>
		<event alias="c">INST_RETIRED.ANY</event>
		<formula>(a-b)/c</formula>
    </metric>
	
    <metric name="metric_M3UPI UPI prefetches (per instr)">
        <throughput-metric-name>metric_M3UPI UPI prefetches (per txn)</throughput-metric-name>
		<event alias="b">UNC_M3UPI_UPI_PREFETCH_SPAWN</event>
		<event alias="c">INST_RETIRED.ANY</event>
		<formula>b/c</formula>
    </metric>
	
    <metric name="metric_M2M extra reads from XPT-UPI prefetches (per instr)">
        <throughput-metric-name>metric_M2M useless XPT-UPI prefetches (per txn)</throughput-metric-name>
		<event alias="a">UNC_M2M_PREFCAM_INSERTS</event>
		<event alias="b">UNC_M2M_PREFCAM_DEMAND_PROMOTIONS</event>
		<event alias="c">INST_RETIRED.ANY</event>
		<formula>(a-b)/c</formula>
    </metric>
	
    <metric name="metric_DDR data rate (MT/sec)">
        <event alias="a">UNC_M_CLOCKTICKS</event>
        <constant alias="socket_count">2</constant>
        <!-- Assumed 6 memory channels populated in each socket; if not change the constant below -->
        <constant alias="channels_populated_per_socket">6</constant>
        <formula>2*a/(socket_count*channels_populated_per_socket)/1000000</formula>
        <formula socket="0">2*a[0]/channels_populated_per_socket/1000000</formula>
		<formula socket="1">2*a[1]/channels_populated_per_socket/1000000</formula>
     </metric>

    <metric name="metric_memory bandwidth read (MB/sec)">
        <event alias="a">UNC_M_CAS_COUNT.RD</event>
        <formula>a*64/1000000</formula>
    </metric>

    <metric name="metric_memory bandwidth write (MB/sec)">
        <event alias="a">UNC_M_CAS_COUNT.WR</event>
        <formula>a*64/1000000</formula>
    </metric>

    <metric name="metric_memory bandwidth total (MB/sec)">
        <event alias="a">UNC_M_CAS_COUNT.RD</event>
        <event alias="b">UNC_M_CAS_COUNT.WR</event>
        <formula>(a+b)*64/1000000</formula>
    </metric>

    <metric name="metric_memory extra read b/w due to XPT prefetches (MB/sec)">
        <event alias="a">UNC_M2M_PREFCAM_INSERTS</event>
        <event alias="b">UNC_M2M_PREFCAM_DEMAND_PROMOTIONS</event>
        <formula>(a-b)*64/1000000</formula>
    </metric>
	
    <metric name="metric_memory extra write b/w due to directory updates (MB/sec)">
		<event alias="a">UNC_CHA_DIR_UPDATE.HA</event>
		<event alias="b">UNC_CHA_DIR_UPDATE.TOR</event>
		<event alias="c">UNC_M2M_DIRECTORY_UPDATE.ANY</event>
        <formula>(a+b+c)*64/1000000</formula>
    </metric>

    <metric name="metric_2LM % of non-inclusive writes to near memory">
		<event alias="a">UNC_M2M_IMC_WRITES.NI</event>
		<event alias="b">UNC_M_DDRT_RDQ_INSERTS</event>
		<event alias="c">UNC_M_CAS_COUNT.WR</event>
        <formula>100*a/(c-b)</formula>
    </metric>

    <metric name="metric_2LM near memory cache read miss rate%">
		<event alias="a">UNC_M2M_TAG_HIT.NM_RD_HIT_CLEAN</event>
		<event alias="b">UNC_M2M_TAG_HIT.NM_RD_HIT_DIRTY</event>
		<event alias="c">UNC_M_DDRT_RDQ_INSERTS</event>
        <formula>100*c/(a+b+c)</formula>
    </metric>
	
    <metric name="metric_memory RPQ read latency (ns)">
        <event alias="a">UNC_M_CLOCKTICKS</event>
        <event alias="b">UNC_M_RPQ_INSERTS</event>
        <event alias="c">UNC_M_RPQ_OCCUPANCY</event>
        <constant alias="socket_count">2</constant>
         <!-- Assumed 6 memory channels populated in each socket; if not change the constant below -->
        <constant alias="channels_populated_per_socket">6</constant>
       <formula>((c/b)/(a/(socket_count*channels_populated_per_socket)))*1000000000</formula>
       <formula socket="0">((c[0]/b[0])/(a[0]/(channels_populated_per_socket)))*1000000000</formula>
       <formula socket="1">((c[1]/b[1])/(a[1]/(channels_populated_per_socket)))*1000000000</formula> 
    </metric>

    <metric name="metric_memory WPQ write latency (ns)">
        <event alias="a">UNC_M_CLOCKTICKS</event>
        <event alias="b">UNC_M_WPQ_INSERTS</event>
        <event alias="c">UNC_M_WPQ_OCCUPANCY</event>
         <constant alias="socket_count">2</constant>
        <!-- Assumed 6 memory channels populated in each socket; if not change the constant below -->
        <constant alias="channels_populated_per_socket">6</constant>
       <formula>((c/b)/(a/(socket_count*channels_populated_per_socket)))*1000000000</formula>
       <formula socket="0">((c[0]/b[0])/(a[0]/(channels_populated_per_socket)))*1000000000</formula>
       <formula socket="1">((c[1]/b[1])/(a[1]/(channels_populated_per_socket)))*1000000000</formula> 
    </metric>


    <metric name="metric_memory avg entries in RPQ">
        <event alias="a">UNC_M_CLOCKTICKS</event>
        <event alias="c">UNC_M_RPQ_OCCUPANCY</event>
       <formula>c/a</formula>
    </metric>

    <metric name="metric_memory avg entries in RPQ when not empty">
        <event alias="c">UNC_M_RPQ_OCCUPANCY</event>
        <event alias="a">UNC_M_RPQ_OCCUPANCY:t=1</event>
       <formula>c/a</formula>
    </metric>

	<metric name="metric_memory % cycles when RPQ is empty">
        <event alias="a">UNC_M_CLOCKTICKS</event>
        <event alias="c">UNC_M_RPQ_OCCUPANCY:t=1</event>
       <formula>100*(1-c/a)</formula>
    </metric>

	<metric name="metric_memory % cycles when RPQ has 1 or more entries">
        <event alias="a">UNC_M_CLOCKTICKS</event>
        <event alias="c">UNC_M_RPQ_OCCUPANCY:t=1</event>
       <formula>100*c/a</formula>
    </metric>

	<metric name="metric_memory % cycles when RPQ has 10 or more entries">
        <event alias="a">UNC_M_CLOCKTICKS</event>
        <event alias="c">UNC_M_RPQ_OCCUPANCY:t=10</event>
       <formula>100*c/a</formula>
    </metric>

	<metric name="metric_memory % cycles when RPQ has 20 or more entries">
        <event alias="a">UNC_M_CLOCKTICKS</event>
        <event alias="c">UNC_M_RPQ_OCCUPANCY:t=20</event>
       <formula>100*c/a</formula>
    </metric>

	<metric name="metric_memory % cycles when RPQ has 40 or more entries">
        <event alias="a">UNC_M_CLOCKTICKS</event>
        <event alias="c">UNC_M_RPQ_OCCUPANCY:t=40</event>
       <formula>100*c/a</formula>
    </metric>

	<metric name="metric_memory avg time (dclk) RPQ not empty">
        <event alias="a">UNC_M_RPQ_OCCUPANCY:t=1:e1</event>
        <event alias="c">UNC_M_RPQ_OCCUPANCY:t=1</event>
       <formula>c/a</formula>
    </metric>

	<metric name="metric_memory avg time (dclk) RPQ empty">
        <event alias="a">UNC_M_RPQ_OCCUPANCY:t=1:e1</event>
        <event alias="c">UNC_M_RPQ_OCCUPANCY:t=1</event>
        <event alias="d">UNC_M_CLOCKTICKS</event>
       <formula>(d-c)/a</formula>
    </metric>

	<metric name="metric_memory avg time with 40 or more entries">
        <event alias="a">UNC_M_RPQ_OCCUPANCY:t=40:e1</event>
        <event alias="c">UNC_M_RPQ_OCCUPANCY:t=40</event>
       <formula>c/a</formula>
    </metric>

	<metric name="metric_memory avg time with less than 40 entries)">
        <event alias="a">UNC_M_RPQ_OCCUPANCY:t=40:e1</event>
        <event alias="c">UNC_M_RPQ_OCCUPANCY:t=40</event>
        <event alias="d">UNC_M_CLOCKTICKS</event>
       <formula>(d-c)/a</formula>
    </metric>

    <metric name="metric_3DXP_memory bandwidth read (MB/sec)">
        <event alias="a">UNC_M_DDRT_RDQ_INSERTS</event>
        <formula>a*64/1000000</formula>
    </metric>

    <metric name="metric_3DXP_memory bandwidth write (MB/sec)">
        <event alias="a">UNC_M_DDRT_WPQ_INSERTS</event>
        <formula>a*64/1000000</formula>
    </metric>

    <metric name="metric_3DXP_memory bandwidth total (MB/sec)">
        <event alias="a">UNC_M_DDRT_RDQ_INSERTS</event>
        <event alias="b">UNC_M_DDRT_WPQ_INSERTS</event>
        <formula>(a+b)*64/1000000</formula>
    </metric>

    <metric name="metric_3DXP memory RPQ read latency (ns)">
        <event alias="a">UNC_M_CLOCKTICKS</event>
        <event alias="b">UNC_M_DDRT_RDQ_INSERTS</event>
        <event alias="c">UNC_M_DDRT_RDQ_OCCUPANCY.ALL</event>
        <constant alias="socket_count">2</constant>
         <!-- Assumed 6 memory channels populated in each socket; if not change the constant below -->
        <constant alias="channels_populated_per_socket">6</constant>
       <formula>((c/b)/(a/(socket_count*channels_populated_per_socket)))*1000000000</formula>
       <formula socket="0">((c[0]/b[0])/(a[0]/(channels_populated_per_socket)))*1000000000</formula>
       <formula socket="1">((c[1]/b[1])/(a[1]/(channels_populated_per_socket)))*1000000000</formula> 
    </metric>

    <metric name="metric_3DXP avg entries in RPQ">
        <event alias="b">UNC_M_CLOCKTICKS</event>
        <event alias="c">UNC_M_DDRT_RDQ_OCCUPANCY.ALL</event>
        <constant alias="channels_populated_per_socket">6</constant>
        <constant alias="aep_dimms_per_socket">6</constant>
       <formula>c/(b*aep_dimms_per_socket/channels_populated_per_socket)</formula>
    </metric>
	
    <metric name="metric_3DXP avg entries in RPQ when not empty">
        <event alias="b">UNC_M_DDRT_RDQ_OCCUPANCY.ALL:t=1</event>
        <event alias="c">UNC_M_DDRT_RDQ_OCCUPANCY.ALL</event>
       <formula>c/b</formula>
    </metric>

    <metric name="metric_3DXP % cycles when RPQ is empty">
        <event alias="a">UNC_M_CLOCKTICKS</event>
        <event alias="b">UNC_M_DDRT_RDQ_OCCUPANCY.ALL:t=1</event>
      <formula>100*(1-(b/a))</formula>
    </metric>

   <metric name="metric_3DXP % cycles when RPQ has 1 or more entries">
        <event alias="a">UNC_M_CLOCKTICKS</event>
        <event alias="b">UNC_M_DDRT_RDQ_OCCUPANCY.ALL:t=1</event>
        <constant alias="channels_populated_per_socket">6</constant>
        <constant alias="aep_dimms_per_socket">6</constant>
       <formula>100*(b/(a*aep_dimms_per_socket/channels_populated_per_socket))</formula>
    </metric>

   <metric name="metric_3DXP % cycles when RPQ has 10 or more entries">
        <event alias="a">UNC_M_CLOCKTICKS</event>
        <event alias="b">UNC_M_DDRT_RDQ_OCCUPANCY.ALL:t=10</event>
        <constant alias="channels_populated_per_socket">6</constant>
        <constant alias="aep_dimms_per_socket">6</constant>
       <formula>100*(b/(a*aep_dimms_per_socket/channels_populated_per_socket))</formula>
    </metric>

   <metric name="metric_3DXP % cycles when RPQ has 24 or more entries">
        <event alias="a">UNC_M_CLOCKTICKS</event>
        <event alias="b">UNC_M_DDRT_RDQ_OCCUPANCY.ALL:t=24</event>
        <constant alias="channels_populated_per_socket">6</constant>
        <constant alias="aep_dimms_per_socket">6</constant>
       <formula>100*(b/(a*aep_dimms_per_socket/channels_populated_per_socket))</formula>
    </metric>

   <metric name="metric_3DXP % cycles when RPQ has 36 or more entries">
        <event alias="a">UNC_M_CLOCKTICKS</event>
        <event alias="b">UNC_M_DDRT_RDQ_OCCUPANCY.ALL:t=36</event>
        <constant alias="channels_populated_per_socket">6</constant>
        <constant alias="aep_dimms_per_socket">6</constant>
       <formula>100*(b/(a*aep_dimms_per_socket/channels_populated_per_socket))</formula>
    </metric>

   <metric name="metric_3DXP avg time (dclk) RPQ not empty">
        <event alias="a">UNC_M_DDRT_RDQ_OCCUPANCY.ALL:t=1:e1</event>
        <event alias="b">UNC_M_DDRT_RDQ_OCCUPANCY.ALL:t=1</event>
       <formula>b/a</formula>
    </metric>

   <metric name="metric_3DXP avg time (dclk) RPQ empty">
        <event alias="a">UNC_M_DDRT_RDQ_OCCUPANCY.ALL:t=1:e1</event>
        <event alias="b">UNC_M_DDRT_RDQ_OCCUPANCY.ALL:t=1</event>
        <event alias="c">UNC_M_CLOCKTICKS</event>
         <!-- Assumed 6 memory channels populated in each socket; if not change the constants below -->
        <constant alias="channels_populated_per_socket">6</constant>
        <constant alias="aep_dimms_per_socket">6</constant>
       <formula>((c*aep_dimms_per_socket/channels_populated_per_socket)-b)/a</formula>
    </metric>

   <metric name="metric_3DXP avg time (dclk) with 36 or more entries in RPQ">
        <event alias="a">UNC_M_DDRT_RDQ_OCCUPANCY.ALL:t=36:e1</event>
        <event alias="b">UNC_M_DDRT_RDQ_OCCUPANCY.ALL:t=36</event>
       <formula>b/a</formula>
    </metric>

   <metric name="metric_3DXP avg time (dclk) with less than 36 entries in RPQ">
        <event alias="a">UNC_M_DDRT_RDQ_OCCUPANCY.ALL:t=36:e1</event>
        <event alias="b">UNC_M_DDRT_RDQ_OCCUPANCY.ALL:t=36</event>
        <event alias="c">UNC_M_CLOCKTICKS</event>
         <!-- Assumed 6 memory channels populated in each socket; if not change the constant below -->
        <constant alias="channels_populated_per_socket">6</constant>
        <constant alias="aep_dimms_per_socket">6</constant>
       <formula>((c*aep_dimms_per_socket/channels_populated_per_socket)-b)/a</formula>
    </metric>
	
	<metric name="metric_3DXP memory WPQ write latency (ns)">
        <event alias="a">UNC_M_CLOCKTICKS</event>
        <event alias="b">UNC_M_DDRT_WPQ_INSERTS</event>
        <event alias="c">UNC_M_DDRT_WPQ_OCCUPANCY.ALL</event>
        <constant alias="socket_count">2</constant>
         <!-- Assumed 6 memory channels populated in each socket; if not change the constant below -->
        <constant alias="channels_populated_per_socket">6</constant>
       <formula>((c/b)/(a/(socket_count*channels_populated_per_socket)))*1000000000</formula>
       <formula socket="0">((c[0]/b[0])/(a[0]/(channels_populated_per_socket)))*1000000000</formula>
       <formula socket="1">((c[1]/b[1])/(a[1]/(channels_populated_per_socket)))*1000000000</formula> 
    </metric>
	
    <metric name="metric_3DXP avg entries in WPQ">
        <event alias="b">UNC_M_CLOCKTICKS</event>
        <event alias="c">UNC_M_DDRT_WPQ_OCCUPANCY.ALL</event>
        <constant alias="channels_populated_per_socket">6</constant>
        <constant alias="aep_dimms_per_socket">6</constant>
       <formula>c/(b*aep_dimms_per_socket/channels_populated_per_socket)</formula>
    </metric>
	
    <metric name="metric_3DXP avg entries in WPQ when not empty">
        <event alias="b">UNC_M_DDRT_WPQ_OCCUPANCY.ALL:t=1</event>
        <event alias="c">UNC_M_DDRT_WPQ_OCCUPANCY.ALL</event>
       <formula>c/b</formula>
    </metric>
	
   <metric name="metric_3DXP % cycles when WPQ has 1 or more entries">
        <event alias="a">UNC_M_CLOCKTICKS</event>
        <event alias="b">UNC_M_DDRT_WPQ_OCCUPANCY.ALL:t=1</event>
         <!-- Assumed 6 memory channels populated in each socket; if not change the constant below -->
        <constant alias="channels_populated_per_socket">6</constant>
        <constant alias="aep_dimms_per_socket">6</constant>
       <formula>100*(b/(a*aep_dimms_per_socket/channels_populated_per_socket))</formula>
    </metric>

   <metric name="metric_3DXP % cycles when WPQ has 10 or more entries">
        <event alias="a">UNC_M_CLOCKTICKS</event>
        <event alias="b">UNC_M_DDRT_WPQ_OCCUPANCY.ALL:t=10</event>
         <!-- Assumed 6 memory channels populated in each socket; if not change the constant below -->
        <constant alias="channels_populated_per_socket">6</constant>
        <constant alias="aep_dimms_per_socket">6</constant>
      <formula>100*(b/(a*aep_dimms_per_socket/channels_populated_per_socket))</formula>
    </metric>
	
    <metric name="metric_3DXP % cycles when WPQ is empty">
        <event alias="a">UNC_M_CLOCKTICKS</event>
        <event alias="b">UNC_M_DDRT_WPQ_OCCUPANCY.ALL:t=1</event>
         <!-- Assumed 6 memory channels populated in each socket; if not change the constant below -->
        <constant alias="channels_populated_per_socket">6</constant>
        <constant alias="aep_dimms_per_socket">6</constant>
       <formula>100*(1-(b/(a*aep_dimms_per_socket/channels_populated_per_socket)))</formula>
    </metric>

   <metric name="metric_3DXP % cycles when WPQ has 20 or more entries">
        <event alias="a">UNC_M_CLOCKTICKS</event>
        <event alias="b">UNC_M_DDRT_WPQ_OCCUPANCY.ALL:t=20</event>
         <!-- Assumed 6 memory channels populated in each socket; if not change the constant below -->
        <constant alias="channels_populated_per_socket">6</constant>
        <constant alias="aep_dimms_per_socket">6</constant>
      <formula>100*(b/(a*aep_dimms_per_socket/channels_populated_per_socket))</formula>
    </metric>

   <metric name="metric_3DXP % cycles when WPQ has 30 or more entries">
        <event alias="a">UNC_M_CLOCKTICKS</event>
        <event alias="b">UNC_M_DDRT_WPQ_OCCUPANCY.ALL:t=30</event>
         <!-- Assumed 6 memory channels populated in each socket; if not change the constant below -->
        <constant alias="channels_populated_per_socket">6</constant>
        <constant alias="aep_dimms_per_socket">6</constant>
      <formula>100*(b/(a*aep_dimms_per_socket/channels_populated_per_socket))</formula>
    </metric>

   <metric name="metric_3DXP avg time (dclk) WPQ not empty">
        <event alias="a">UNC_M_DDRT_WPQ_OCCUPANCY.ALL:t=1:e1</event>
        <event alias="b">UNC_M_DDRT_WPQ_OCCUPANCY.ALL:t=1</event>
       <formula>b/a</formula>
    </metric>
	
   <metric name="metric_3DXP avg time (dclk) WPQ empty">
        <event alias="a">UNC_M_DDRT_WPQ_OCCUPANCY.ALL:t=1:e1</event>
        <event alias="b">UNC_M_DDRT_WPQ_OCCUPANCY.ALL:t=1</event>
        <event alias="c">UNC_M_CLOCKTICKS</event>
         <!-- Assumed 6 memory channels populated in each socket; if not change the constant below -->
        <constant alias="channels_populated_per_socket">6</constant>
        <constant alias="aep_dimms_per_socket">6</constant>
       <formula>((c*aep_dimms_per_socket/channels_populated_per_socket)-b)/a</formula>
    </metric>

   <metric name="metric_3DXP avg time (dclk) with 30 or more entries in WPQ">
        <event alias="a">UNC_M_DDRT_WPQ_OCCUPANCY.ALL:t=30:e1</event>
        <event alias="b">UNC_M_DDRT_WPQ_OCCUPANCY.ALL:t=30</event>
       <formula>b/a</formula>
    </metric>

	<metric name="metric_CHA % cyles Fast asserted">
		<event alias="a">UNC_CHA_FAST_ASSERTED.HORZ:u0x1</event>
        <event alias="c">UNC_CHA_CLOCKTICKS</event>
		<formula>100*a/c</formula>
	</metric>
	
   <metric name="metric_3DXP avg time (dclk) with less than 30 entries in WPQ">
        <event alias="a">UNC_M_DDRT_WPQ_OCCUPANCY.ALL:t=30:e1</event>
        <event alias="b">UNC_M_DDRT_WPQ_OCCUPANCY.ALL:t=30</event>
        <event alias="c">UNC_M_CLOCKTICKS</event>
         <!-- Assumed 6 memory channels populated in each socket; if not change the constant below -->
        <constant alias="channels_populated_per_socket">6</constant>
        <constant alias="aep_dimms_per_socket">6</constant>
       <formula>((c*aep_dimms_per_socket/channels_populated_per_socket)-b)/a</formula>
    </metric>

    <metric name="metric_CHA RxC IRQ latency (ns)">
        <event alias="a">UNC_CHA_RxC_OCCUPANCY.IRQ</event>
        <event alias="b">UNC_CHA_RxC_INSERTS.IRQ</event>
        <event alias="c">UNC_CHA_CLOCKTICKS</event>
       <constant alias="socket_count">2</constant>
		<!-- In case the # of cache slices is not the same as the # of cores (as in the case of offlining some cores,
			change the cores.count below to the actual number of slices present (which is the original core count -->
       <constant alias="d">system.sockets[0].cores.count</constant>
         <formula>1000000000*(a/b)/(c/(d*socket_count))</formula>
        <formula socket="0">1000000000*(a[0]/b[0])/(c[0]/d)</formula>
        <formula socket="1">1000000000*(a[1]/b[1])/(c[1]/d)</formula>
    </metric>
	
    <metric name="metric_CHA RxC IRQ avg entries">
        <event alias="a">UNC_CHA_RxC_OCCUPANCY.IRQ</event>
        <event alias="c">UNC_CHA_CLOCKTICKS</event>
        <formula>a/c</formula>
    </metric>
	
    <metric name="metric_CHA RxC IRQ % cycles when Q has 18 or more entries">
        <event alias="a">UNC_CHA_CLOCKTICKS</event>
        <event alias="c">UNC_CHA_RxC_OCCUPANCY.IRQ:t=18</event>
       <formula>100*c/a</formula>
    </metric>
	
    <metric name="metric_M2M avg entries in TxC AD Q">
        <event alias="a">UNC_CHA_CLOCKTICKS</event>
        <event alias="c">UNC_M2M_TxC_AD_OCCUPANCY</event>
       <formula>c/a</formula>
    </metric>

    <metric name="metric_M2M avg entries in TxC BL Q">
        <event alias="a">UNC_CHA_CLOCKTICKS</event>
        <event alias="c">UNC_M2M_TxC_BL_OCCUPANCY.ALL</event>
       <formula>c/a</formula>
    </metric>

    <metric name="metric_M2M RxC AD latency (ns)">
        <event alias="a">UNC_M2M_RxC_AD_OCCUPANCY</event>
        <event alias="b">UNC_M2M_RxC_AD_INSERTS</event>
        <event alias="c">UNC_CHA_CLOCKTICKS</event>
       <constant alias="socket_count">2</constant>
		<!-- In case the # of cache slices is not the same as the # of cores (as in the case of offlining some cores,
			change the cores.count below to the actual number of slices present (which is the original core count -->
       <constant alias="d">system.sockets[0].cores.count</constant>
         <formula>1000000000*(a/b)/(c/(d*socket_count))</formula>
        <formula socket="0">1000000000*(a[0]/b[0])/(c[0]/d)</formula>
        <formula socket="1">1000000000*(a[1]/b[1])/(c[1]/d)</formula>
    </metric>
	
    <metric name="metric_M2M RxC AD avg entries">
        <event alias="a">UNC_M2M_RxC_AD_OCCUPANCY</event>
        <event alias="c">UNC_CHA_CLOCKTICKS</event>
        <formula>a/c</formula>
    </metric>

    <metric name="metric_M2M RxC BL latency (ns)">
        <event alias="a">UNC_M2M_RxC_BL_OCCUPANCY</event>
        <event alias="b">UNC_M2M_RxC_BL_INSERTS</event>
        <event alias="c">UNC_CHA_CLOCKTICKS</event>
       <constant alias="socket_count">2</constant>
		<!-- In case the # of cache slices is not the same as the # of cores (as in the case of offlining some cores,
			change the cores.count below to the actual number of slices present (which is the original core count -->
       <constant alias="d">system.sockets[0].cores.count</constant>
         <formula>1000000000*(a/b)/(c/(d*socket_count))</formula>
        <formula socket="0">1000000000*(a[0]/b[0])/(c[0]/d)</formula>
        <formula socket="1">1000000000*(a[1]/b[1])/(c[1]/d)</formula>
    </metric>
	
    <metric name="metric_M2M RxC BL avg entries">
        <event alias="a">UNC_M2M_RxC_BL_OCCUPANCY</event>
        <event alias="c">UNC_CHA_CLOCKTICKS</event>
        <formula>a/c</formula>
    </metric>
	
    <metric name="metric_IO_bandwidth_disk_or_network_writes (MB/sec)">
        <event alias="a">UNC_IIO_PAYLOAD_BYTES_IN.MEM_READ.PART0</event>
        <event alias="b">UNC_IIO_PAYLOAD_BYTES_IN.MEM_READ.PART1</event>
        <event alias="c">UNC_IIO_PAYLOAD_BYTES_IN.MEM_READ.PART2</event>
        <event alias="d">UNC_IIO_PAYLOAD_BYTES_IN.MEM_READ.PART3</event>
        <formula>(a+b+c+d)*4/1000000</formula>
    </metric>

    <metric name="metric_IO_bandwidth_disk_or_network_reads (MB/sec)">
        <event alias="a">UNC_IIO_PAYLOAD_BYTES_IN.MEM_WRITE.PART0</event>
        <event alias="b">UNC_IIO_PAYLOAD_BYTES_IN.MEM_WRITE.PART1</event>
        <event alias="c">UNC_IIO_PAYLOAD_BYTES_IN.MEM_WRITE.PART2</event>
        <event alias="d">UNC_IIO_PAYLOAD_BYTES_IN.MEM_WRITE.PART3</event>
        <formula>(a+b+c+d)*4/1000000</formula>
    </metric>

    <metric name="metric_IO_number of partial PCI writes per sec">
        <event alias="a">UNC_CHA_TOR_INSERTS.IO_HIT:filter1=0x40033</event>
        <event alias="b">UNC_CHA_TOR_INSERTS.IO_MISS:filter1=0x40033</event>
        <formula>a+b</formula>
    </metric>

    <metric name="metric_IO_read cache miss(disk/network writes) bandwidth (MB/sec)">
        <event alias="a">UNC_CHA_TOR_INSERTS.IO_MISS:filter1=0x43c33</event>
        <formula>a*64/1000000</formula>
    </metric>

    <metric name="metric_IO_write cache miss(disk/network reads) bandwidth (MB/sec)">
        <event alias="a">UNC_CHA_TOR_INSERTS.IO_MISS:filter1=0x49033</event>
        <event alias="b">UNC_CHA_TOR_INSERTS.IO_MISS:filter1=0x40033</event>
        <formula>(a+b)*64/1000000</formula>
    </metric>

    <metric name="metric_IONUMA % disk/network reads addressed to local memory">
        <event alias="a">UNC_CHA_TOR_INSERTS.IO:filter1=0x49031</event>
        <event alias="b">UNC_CHA_TOR_INSERTS.IO:filter1=0x49032</event>
        <event alias="c">UNC_CHA_TOR_INSERTS.IO:filter1=0x40031</event>
        <event alias="d">UNC_CHA_TOR_INSERTS.IO:filter1=0x40032</event>
        <formula>100*(b+d)/(a+b+c+d)</formula>
    </metric>

    <metric name="metric_IONUMA % disk/network reads addressed to remote memory">
        <event alias="a">UNC_CHA_TOR_INSERTS.IO:filter1=0x49031</event>
        <event alias="b">UNC_CHA_TOR_INSERTS.IO:filter1=0x49032</event>
        <event alias="c">UNC_CHA_TOR_INSERTS.IO:filter1=0x40031</event>
        <event alias="d">UNC_CHA_TOR_INSERTS.IO:filter1=0x40032</event>
        <formula>100*(a+c)/(a+b+c+d)</formula>
    </metric>

    <metric name="metric_MMIO reads per instr">
       <throughput-metric-name>metric_MMIO reads per txn</throughput-metric-name>
		<event alias="a">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x40040e33</event>
		<event alias="b">INST_RETIRED.ANY</event>
        <formula>a/b</formula>
    </metric>

    <metric name="metric_MMIO writes per instr">
       <throughput-metric-name>metric_MMIO writes per txn</throughput-metric-name>
		<event alias="a">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x40041e33</event>
		<event alias="b">INST_RETIRED.ANY</event>
        <formula>a/b</formula>
    </metric>

    <metric name="metric_memory reads vs. all requests">
        <event alias="a">UNC_M_CAS_COUNT.RD</event>
        <event alias="b">UNC_M_CAS_COUNT.WR</event>
        <formula>a/(a+b)</formula>
    </metric>

    <metric name="metric_memory Page Empty vs. all requests">
        <event alias="a">UNC_M_PRE_COUNT.RD:u0xc</event>
        <event alias="c">UNC_M_PRE_COUNT.PAGE_MISS</event>
        <event alias="d">UNC_M_CAS_COUNT.RD</event>
        <event alias="e">UNC_M_CAS_COUNT.WR</event>
        <formula>(a-c)/(d+e)</formula>
    </metric>

    <metric name="metric_memory Page Misses vs. all requests">
        <event alias="b">UNC_M_PRE_COUNT.PAGE_MISS</event>
        <event alias="c">UNC_M_CAS_COUNT.RD</event>
        <event alias="d">UNC_M_CAS_COUNT.WR</event>
        <formula>b/(c+d)</formula>
    </metric>

    <metric name="metric_memory Page Hits vs. all requests">
        <event alias="a">UNC_M_PRE_COUNT.RD:u0xc</event>
        <event alias="c">UNC_M_CAS_COUNT.RD</event>
        <event alias="d">UNC_M_CAS_COUNT.WR</event>
        <formula>1-(a/(c+d))</formula>
    </metric>

    <metric name="metric_memory % Cycles where all DRAM ranks are in PPD mode">
        <event alias="a">UNC_M_POWER_CHANNEL_PPD</event>
        <event alias="b">UNC_M_CLOCKTICKS</event>
        <formula>100*a/b</formula>
    </metric>

    <metric name="metric_memory % Cycles Memory is in self refresh power mode">
        <event alias="a">UNC_M_POWER_SELF_REFRESH</event>
        <event alias="b">UNC_M_CLOCKTICKS</event>
        <formula>100*a/b</formula>
    </metric>

    <metric name="metric_ItoM operations (fast strings) that reference LLC per instr">
        <throughput-metric-name>metric_ItoM operations (fast strings) that reference LLC per txn</throughput-metric-name>
		<event alias="a">UNC_CHA_TOR_INSERTS.IA_HIT:filter1=0x49033</event>
		<event alias="b">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x49033</event>
		<event alias="c">INST_RETIRED.ANY</event>
		<formula>(a+b)/c</formula>
	</metric>
		
	<metric name="metric_ItoM operations (fast strings) that miss LLC per instr">
        <throughput-metric-name>metric_ItoM operations (fast strings) that miss LLC per txn</throughput-metric-name>
		<event alias="a">UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x49033</event>
		<event alias="c">INST_RETIRED.ANY</event>
		<formula>a/c</formula>
	</metric>
		
    <metric name="metric_% Uops delivered from decoded Icache (DSB)">
        <event alias="a">IDQ.DSB_UOPS</event>
        <event alias="b">UOPS_ISSUED.ANY</event>
        <formula>100*(a/b)</formula>
    </metric>

    <metric name="metric_% Uops delivered from legacy decode pipeline (MITE)">
        <event alias="a">IDQ.MITE_UOPS</event>
        <event alias="b">UOPS_ISSUED.ANY</event>
        <formula>100*(a/b)</formula>
    </metric>

    <metric name="metric_% Uops delivered from microcode sequencer (MS)">
        <event alias="a">IDQ.MS_UOPS</event>
        <event alias="b">UOPS_ISSUED.ANY</event>
        <formula>100*(a/b)</formula>
    </metric>

    <metric name="metric_% Uops delivered from loop stream detector (LSD)">
        <event alias="a">LSD.UOPS</event>
        <event alias="b">UOPS_ISSUED.ANY</event>
        <formula>100*(a/b)</formula>
    </metric>

	<metric name="metric_FP scalar single-precision FP instructions retired per instr">
        <throughput-metric-name>metric_FP scalar single-precision FP instructions retired per txn</throughput-metric-name>
		<event alias="a">FP_ARITH_INST_RETIRED.SCALAR_SINGLE</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>

	<metric name="metric_FP scalar double-precision FP instructions retired per instr">
        <throughput-metric-name>metric_FP scalar double-precision FP instructions retired per txn</throughput-metric-name>
		<event alias="a">FP_ARITH_INST_RETIRED.SCALAR_DOUBLE</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>

	<metric name="metric_FP 128-bit packed single-precision FP instructions retired per instr">
        <throughput-metric-name>metric_FP 128-bit packed single-precision FP instructions retired per txn</throughput-metric-name>
		<event alias="a">FP_ARITH_INST_RETIRED.128B_PACKED_SINGLE</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>
		
	<metric name="metric_FP 128-bit packed double-precision FP instructions retired per instr">
        <throughput-metric-name>metric_FP 128-bit packed double-precision FP instructions retired per txn</throughput-metric-name>
		<event alias="a">FP_ARITH_INST_RETIRED.128B_PACKED_DOUBLE</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>
		
	<metric name="metric_FP 256-bit packed single-precision FP instructions retired per instr">
        <throughput-metric-name>metric_FP 256-bit packed single-precision FP instructions retired per txn</throughput-metric-name>
		<event alias="a">FP_ARITH_INST_RETIRED.256B_PACKED_SINGLE</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>
		
	<metric name="metric_FP 256-bit packed double-precision FP instructions retired per instr">
        <throughput-metric-name>metric_FP 256-bit packed double-precision FP instructions retired per txn</throughput-metric-name>
		<event alias="a">FP_ARITH_INST_RETIRED.256B_PACKED_DOUBLE</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>

	<metric name="metric_FP 512-bit packed single-precision FP instructions retired per instr">
        <throughput-metric-name>metric_FP 512-bit packed single-precision FP instructions retired per txn</throughput-metric-name>
		<event alias="a">FP_ARITH_INST_RETIRED.512B_PACKED_SINGLE</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>
		
	<metric name="metric_FP 512-bit packed double-precision FP instructions retired per instr">
        <throughput-metric-name>metric_FP 512-bit packed double-precision FP instructions retired per txn</throughput-metric-name>
		<event alias="a">FP_ARITH_INST_RETIRED.512B_PACKED_DOUBLE</event>
		<event alias="b">INST_RETIRED.ANY</event>
		<formula>a/b</formula>
	</metric>

	<metric name="metric_DRAM power (watts)">
		<event alias="a">MSR_EVENT:msr=0x619:type=FREERUN:scope=PACKAGE</event>
		<formula>a*15.3/1000000</formula>
	</metric>
	
	<metric name="metric_package power (watts)">
		<event alias="a">MSR_EVENT:msr=0x611:type=FREERUN:scope=PACKAGE</event>
		<formula>a*61/1000000</formula>
	</metric>

	<metric name="metric_core c6 residency %">
		<event alias="a">MSR_EVENT:msr=0x3FD:type=FREERUN:scope=THREAD</event>
		<event alias="b">TSC</event>
		<formula>100*a/b</formula>
	</metric>

	<metric name="metric_package c2 residency %">
		<event alias="a">MSR_EVENT:msr=0x60D:type=FREERUN:scope=PACKAGE</event>
		<event alias="b">TSC</event>
		<constant alias="cpu_count">system.sockets[0].cpus.count</constant>      
		<formula>100*a*cpu_count/b</formula>
	</metric>
	
	<metric name="metric_package c6 residency %">
		<event alias="a">MSR_EVENT:msr=0x3F9:type=FREERUN:scope=PACKAGE</event>
		<event alias="b">TSC</event>
		<constant alias="cpu_count">system.sockets[0].cpus.count</constant>      
		<formula>100*a*cpu_count/b</formula>
	</metric>
	
	<metric name="metric_core SW prefetch NTA per instr">
        <throughput-metric-name>metric_core SW prefetch NTA per txn</throughput-metric-name>
		<event alias="a">SW_PREFETCH_ACCESS.NTA</event>
		<event alias="c">INST_RETIRED.ANY</event>
		<formula>a/c</formula>
	</metric>

	<metric name="metric_core uncacheable access and clflushes per instr">
        <throughput-metric-name>metric_core uncacheable access and clflushes per txn</throughput-metric-name>
		<event alias="a">OFFCORE_REQUESTS.MEM_UC</event>
		<event alias="c">INST_RETIRED.ANY</event>
		<formula>a/c</formula>
	</metric>

	<metric name="metric_core % cycles core power throttled">
		<event alias="a">CORE_POWER.THROTTLE</event>
		<event alias="c">CPU_CLK_UNHALTED.THREAD</event>
		<formula>100*a/c</formula>
	</metric>

	<metric name="metric_core % cycles in non AVX license">
		<event alias="a">CORE_POWER.LVL0_TURBO_LICENSE</event>
		<event alias="b">CORE_POWER.LVL1_TURBO_LICENSE</event>
		<event alias="c">CORE_POWER.LVL2_TURBO_LICENSE</event>
		<formula>100*a/(a+b+c)</formula>
	</metric>

	<metric name="metric_core % cycles in AVX2 license">
		<event alias="a">CORE_POWER.LVL0_TURBO_LICENSE</event>
		<event alias="b">CORE_POWER.LVL1_TURBO_LICENSE</event>
		<event alias="c">CORE_POWER.LVL2_TURBO_LICENSE</event>
		<formula>100*b/(a+b+c)</formula>
	</metric>

	<metric name="metric_core % cycles in AVX-512 license">
		<event alias="a">CORE_POWER.LVL0_TURBO_LICENSE</event>
		<event alias="b">CORE_POWER.LVL1_TURBO_LICENSE</event>
		<event alias="c">CORE_POWER.LVL2_TURBO_LICENSE</event>
		<formula>100*c/(a+b+c)</formula>
	</metric>

	<metric name="metric_SMI number of SMIs per sec">
		<event alias="a">MSR_EVENT:msr=0x34:type=FREERUN:scope=PACKAGE</event>
		<formula>a+0</formula>
	</metric>

    <metric name="metric_core initiated local dram read bandwidth (MB/sec)">
         <!-- Includes demand+L2pref+L3 pref for all data read, code read and rfo that hit local dram-->
        <event alias="a">OFFCORE_RESPONSE:request=ALL_READS:response=L3_MISS.ANY_SNOOP:ocr_msr_val=0x3f840007f7</event>
        <formula>a*64/1000000</formula>
    </metric>

    <metric name="metric_core initiated remote dram read bandwidth (MB/sec)">
         <!-- Includes demand+L2pref+L3 pref for all data read, code read and rfo that hit remote dram -->
        <event alias="a">OFFCORE_RESPONSE:request=ALL_READS:response=L3_MISS.ANY_SNOOP:ocr_msr_val=0x3fB80007f7</event>
        <formula>a*64/1000000</formula>
    </metric>

    <metric name="metric_core initiated local DCPMEM read bandwidth (MB/sec)">
         <!-- Includes demand+L2pref+L3 pref for all data read, code read and rfo that hit local DCPMEM-->
        <event alias="a">OFFCORE_RESPONSE:request=ALL_READS:response=L3_MISS.ANY_SNOOP:ocr_msr_val=0x3f804007f7</event>
        <formula>a*64/1000000</formula>
    </metric>

    <metric name="metric_core initiated remote DCPMEM read bandwidth (MB/sec)">
         <!-- Includes demand+L2pref+L3 pref for all data read, code read and rfo that hit remote DCPMEM -->
        <event alias="a">OFFCORE_RESPONSE:request=ALL_READS:response=L3_MISS.ANY_SNOOP:ocr_msr_val=0x3f838007f7</event>
        <formula>a*64/1000000</formula>
    </metric>

    <metric name="metric_M3UPI avg all VN0 entries">
        <event alias="a">UNC_M3UPI_CLOCKTICKS</event>
        <event alias="c">UNC_M3UPI_RxC_OCCUPANCY_VN0.BL_RSP:u0x7f</event>
       <formula>c/a</formula>
    </metric>

	<metric name="metric_M3UPI % cycles when all VN0 has 1 or more entries">
        <event alias="a">UNC_M3UPI_CLOCKTICKS</event>
        <event alias="c">UNC_M3UPI_RxC_OCCUPANCY_VN0.BL_RSP:u0x7f:t=1</event>
       <formula>100*c/a</formula>
    </metric>

	<metric name="metric_M3UPI % cycles when all VN0 has 10 or more entries">
        <event alias="a">UNC_M3UPI_CLOCKTICKS</event>
        <event alias="c">UNC_M3UPI_RxC_OCCUPANCY_VN0.BL_RSP:u0x7f:t=10</event>
       <formula>100*c/a</formula>
    </metric>

	<metric name="metric_M3UPI % cycles when all VN0 has 30 or more entries">
        <event alias="a">UNC_M3UPI_CLOCKTICKS</event>
        <event alias="c">UNC_M3UPI_RxC_OCCUPANCY_VN0.BL_RSP:u0x7f:t=30</event>
       <formula>100*c/a</formula>
    </metric>

	<metric name="metric_M3UPI % cycles when all VN0 has 50 or more entries">
        <event alias="a">UNC_M3UPI_CLOCKTICKS</event>
        <event alias="c">UNC_M3UPI_RxC_OCCUPANCY_VN0.BL_RSP:u0x7f:t=50</event>
       <formula>100*c/a</formula>
    </metric>

	<metric name="metric_TMAM_Info_CoreIPC">
        <event alias="a">INST_RETIRED.ANY</event>
		<event alias="b">CPU_CLK_UNHALTED.THREAD_ANY</event>
		<constant alias="threads">system.sockets[0][0].size</constant>      
		<formula>(a/(b/threads))</formula>
	</metric>	

	<metric name="metric_TMAM_Info_Memory Level Parallelism">
		<event alias="a">L1D_PEND_MISS.PENDING</event>
		<event alias="b">L1D_PEND_MISS.PENDING_CYCLES_ANY</event>		
		<constant alias="threads">system.sockets[0][0].size</constant>      
		<formula>(a/(b/threads))</formula>
	</metric>	

	<metric name="metric_TMAM_Info_L1D_Load_Miss_Latency(ns)">
		<event alias="a">L1D_PEND_MISS.PENDING</event>
		<event alias="f">MEM_LOAD_RETIRED.FB_HIT</event>		
		<event alias="g">MEM_LOAD_RETIRED.L1_MISS</event>		
        <event alias="c">CPU_CLK_UNHALTED.THREAD</event>
        <event alias="d">CPU_CLK_UNHALTED.REF_TSC</event>
        <constant alias="e">system.tsc_freq</constant>
        <formula>1000000000*(a/(f+g))/(c/d*e)</formula>
	</metric>	

	<metric name="metric_TMAM_Info_cycles_both_threads_active(%)">
		<event alias="a">CPU_CLK_THREAD_UNHALTED.ONE_THREAD_ACTIVE</event>
		<event alias="b">CPU_CLK_THREAD_UNHALTED.REF_XCLK_ANY</event>
		<constant alias="threads">system.sockets[0][0].size</constant>      
		<formula>100*(((threads)&lt;2)?0:(1-a/(b/2)))</formula>
	</metric>
	
	<metric name="metric_TMAM_Frontend_Bound(%)">
		<event alias="a">CPU_CLK_UNHALTED.THREAD_ANY</event>
		<event alias="c">IDQ_UOPS_NOT_DELIVERED.CORE</event>
		<constant alias="threads">system.sockets[0][0].size</constant>      
		<formula>100*c/(4*(a/threads))</formula>
	</metric>

	<metric name="metric_TMAM_..Frontend_Latency(%)">
		<event alias="a">CPU_CLK_UNHALTED.THREAD_ANY</event>
		<event alias="c">IDQ_UOPS_NOT_DELIVERED.CYCLES_0_UOPS_DELIV.CORE</event>
		<constant alias="threads">system.sockets[0][0].size</constant>      
		<formula>100*c/(a/threads)</formula>
	</metric>
	
	<metric name="metric_TMAM_....ICache_Misses(%)">
		<event alias="a">ICACHE_16B.IFDATA_STALL</event>
		<event alias="b">ICACHE_16B.IFDATA_STALL:c1:e1</event>
		<event alias="d">CPU_CLK_UNHALTED.THREAD</event>
		<formula>100*(a+2*b)/d</formula>
	</metric>

	<metric name="metric_TMAM_....ITLB_Misses(%)">
		<event alias="a">ICACHE_64B.IFTAG_STALL</event>
		<event alias="d">CPU_CLK_UNHALTED.THREAD</event>
		<formula>100*a/d</formula>
	</metric>

	<metric name="metric_TMAM_....Branch_Resteers(%)">
        <event alias="a">INT_MISC.CLEAR_RESTEER_CYCLES</event>
        <event alias="b">BACLEARS.ANY</event>
		<event alias="j">CPU_CLK_UNHALTED.THREAD</event>
		<formula>100*(a+9*b)/j</formula>
	</metric>

	<metric name="metric_TMAM_......Mispredicts_Resteers(%)">
        <event alias="a">INT_MISC.CLEAR_RESTEER_CYCLES</event>
        <event alias="b">MACHINE_CLEARS.COUNT</event>
        <event alias="c">BR_MISP_RETIRED.ALL_BRANCHES</event>
		<event alias="j">CPU_CLK_UNHALTED.THREAD</event>
		<formula>100*a*(c/(b+c))/j</formula>
	</metric>
	
	<metric name="metric_TMAM_......Clears_Resteers(%)">
        <event alias="a">INT_MISC.CLEAR_RESTEER_CYCLES</event>
        <event alias="b">MACHINE_CLEARS.COUNT</event>
        <event alias="c">BR_MISP_RETIRED.ALL_BRANCHES</event>
		<event alias="j">CPU_CLK_UNHALTED.THREAD</event>
		<formula>100*a*(1-(c/(b+c)))/j</formula>
	</metric>

	<metric name="metric_TMAM_......Unknown_Branches_Resteers(%)">
        <event alias="b">BACLEARS.ANY</event>
		<event alias="j">CPU_CLK_UNHALTED.THREAD</event>
		<formula>100*(9*b)/j</formula>
	</metric>

	<metric name="metric_TMAM_....DSB_Switches(%)">
		<event alias="a">DSB2MITE_SWITCHES.PENALTY_CYCLES</event>
		<event alias="d">CPU_CLK_UNHALTED.THREAD</event>
		<formula>100*a/d</formula>
	</metric>
	
	<metric name="metric_TMAM_....MS_Switches(%)">
		<event alias="a">IDQ.MS_SWITCHES</event>
		<event alias="d">CPU_CLK_UNHALTED.THREAD</event>
		<!-- 2 cycle is the MS switch cost -->
		<formula>100*2*a/d</formula>
	</metric>
	
	<metric name="metric_TMAM_..Frontend_Bandwidth(%)">
		<event alias="a">CPU_CLK_UNHALTED.THREAD_ANY</event>
		<event alias="c">IDQ_UOPS_NOT_DELIVERED.CORE</event>
		<event alias="d">IDQ_UOPS_NOT_DELIVERED.CYCLES_0_UOPS_DELIV.CORE</event>		
		<constant alias="threads">system.sockets[0][0].size</constant>      
		<formula>100*(c-4*d)/(4*(a/threads))</formula>
	</metric>

    <metric name="metric_TMAM_Bad_Speculation(%)">
        <event alias="a">UOPS_ISSUED.ANY</event>
        <event alias="b">UOPS_RETIRED.RETIRE_SLOTS</event>
        <event alias="c">INT_MISC.RECOVERY_CYCLES_ANY</event>
        <event alias="d">CPU_CLK_UNHALTED.THREAD_ANY</event>
		<constant alias="threads">system.sockets[0][0].size</constant>      
        <formula>100*(a-b+(4*c/threads))/(4*d/threads)</formula>
    </metric>

	<metric name="metric_TMAM_..Branch_Mispredicts(%)">
        <event alias="a">UOPS_ISSUED.ANY</event>
        <event alias="b">UOPS_RETIRED.RETIRE_SLOTS</event>
        <event alias="c">INT_MISC.RECOVERY_CYCLES_ANY</event>
        <event alias="d">CPU_CLK_UNHALTED.THREAD_ANY</event>
        <event alias="e">BR_MISP_RETIRED.ALL_BRANCHES</event>
        <event alias="f">MACHINE_CLEARS.COUNT</event>
		<constant alias="threads">system.sockets[0][0].size</constant>      
        <formula>(e/(e+f))*100*(a-b+(4*c/threads))/(4*d/threads)</formula>
    </metric>

    <metric name="metric_TMAM_..Machine_Clears(%)">
        <event alias="a">UOPS_ISSUED.ANY</event>
        <event alias="b">UOPS_RETIRED.RETIRE_SLOTS</event>
        <event alias="c">INT_MISC.RECOVERY_CYCLES_ANY</event>
        <event alias="d">CPU_CLK_UNHALTED.THREAD_ANY</event>
        <event alias="e">BR_MISP_RETIRED.ALL_BRANCHES</event>
        <event alias="f">MACHINE_CLEARS.COUNT</event>
		<constant alias="threads">system.sockets[0][0].size</constant>      
        <formula>(f/(e+f))*100*(a-b+(4*c/threads))/(4*d/threads)</formula>
    </metric>

	<metric name="metric_TMAM_Backend_bound(%)">
		<event alias="a">IDQ_UOPS_NOT_DELIVERED.CORE</event>
		<event alias="b">UOPS_ISSUED.ANY</event>
		<event alias="c">INT_MISC.RECOVERY_CYCLES_ANY</event>
        <event alias="d">CPU_CLK_UNHALTED.THREAD_ANY</event>
        <event alias="e">UOPS_RETIRED.RETIRE_SLOTS</event>
		<constant alias="threads">system.sockets[0][0].size</constant>      
		<formula>100-(100*(b-e+4*(c/threads)+a+e)/(4*d/threads))</formula>
	</metric>

	<metric name="metric_TMAM_..Memory_Bound(%)">
		<event alias="a">IDQ_UOPS_NOT_DELIVERED.CORE</event>
		<event alias="b">UOPS_ISSUED.ANY</event>
		<event alias="c">INT_MISC.RECOVERY_CYCLES_ANY</event>
        <event alias="d">CPU_CLK_UNHALTED.THREAD_ANY</event>
        <event alias="e">UOPS_RETIRED.RETIRE_SLOTS</event>
        <event alias="f">CYCLE_ACTIVITY.STALLS_MEM_ANY</event>
        <event alias="g">EXE_ACTIVITY.BOUND_ON_STORES</event>
        <event alias="j">EXE_ACTIVITY.1_PORTS_UTIL</event>
        <event alias="k">EXE_ACTIVITY.2_PORTS_UTIL</event>
        <event alias="m">EXE_ACTIVITY.EXE_BOUND_0_PORTS</event>
        <event alias="p">CPU_CLK_UNHALTED.THREAD</event>
        <event alias="q">INST_RETIRED.ANY</event>
		<constant alias="threads">system.sockets[0][0].size</constant>      
		<formula>100*(1-((b-e+4*(c/threads)+a+e)/(4*d/threads)))*(f+g)/(m+j+(((q/p)&gt;1.8)?k:0)+f+g)</formula>
	</metric>
 
	<metric name="metric_TMAM_....L1_Bound(%)">
 		<event alias="a">CYCLE_ACTIVITY.STALLS_MEM_ANY</event>
        <event alias="b">CYCLE_ACTIVITY.STALLS_L1D_MISS</event>
        <event alias="c">CPU_CLK_UNHALTED.THREAD</event>
		<formula>100*(a-b)/c</formula>
    </metric>

	<metric name="metric_TMAM_......DTLB_Load(%)">
        <event alias="a">DTLB_LOAD_MISSES.STLB_HIT</event>
        <event alias="b">DTLB_LOAD_MISSES.WALK_ACTIVE</event>
        <event alias="c">CPU_CLK_UNHALTED.THREAD</event>
		<formula>100*(7*a+b)/c</formula>
    </metric>

	<metric name="metric_TMAM_......Store_Fwd_Blk(%)">
		<event alias="a">LD_BLOCKS.STORE_FORWARD</event>
		<event alias="b">CPU_CLK_UNHALTED.THREAD</event>
		<formula>100*((13 * a) / b) </formula>				
	</metric>

	<metric name="metric_TMAM_......Lock_Latency(%)">
		<event alias="a">MEM_INST_RETIRED.LOCK_LOADS</event>
		<event alias="b">CPU_CLK_UNHALTED.THREAD</event>
		<event alias="c">MEM_INST_RETIRED.ALL_STORES</event>
		<event alias="d">OFFCORE_REQUESTS_OUTSTANDING.CYCLES_WITH_DEMAND_RFO</event>
		<formula>100*(((a / c)* [1*b ,1*d].min)/ b) </formula>			
	</metric>

	<metric name="metric_TMAM_....L2_Bound(%)">
        <event alias="a">CYCLE_ACTIVITY.STALLS_L1D_MISS</event>
        <event alias="b">CYCLE_ACTIVITY.STALLS_L2_MISS</event>
        <event alias="c">CPU_CLK_UNHALTED.THREAD</event>
		<formula>100*(a-b)/c</formula>
    </metric>

	<metric name="metric_TMAM_....L3_Bound(%)">
        <event alias="a">CYCLE_ACTIVITY.STALLS_L2_MISS</event>
        <event alias="c">CYCLE_ACTIVITY.STALLS_L3_MISS</event>
        <event alias="d">CPU_CLK_UNHALTED.THREAD</event>
		<formula>100*(a-c)/d</formula>
    </metric>
	
	<metric name="metric_TMAM_......Contested_Accesses(%)">
		<event alias="a">MEM_LOAD_L3_HIT_RETIRED.XSNP_HITM</event>
		<event alias="b">CPU_CLK_UNHALTED.THREAD</event>
		<event alias="c">MEM_LOAD_L3_HIT_RETIRED.XSNP_MISS</event>
		<formula>100*(60 * ( a + c ) / b) </formula>			
	</metric>

	<metric name="metric_TMAM_......Data_Sharing(%)">
		<event alias="a">MEM_LOAD_L3_HIT_RETIRED.XSNP_HIT</event>
		<event alias="b">CPU_CLK_UNHALTED.THREAD</event>
		<formula>100*(43 * a /b) </formula>			
	</metric>

	<metric name="metric_TMAM_......L3_Latency(%)">
        <event alias="b">OFFCORE_REQUESTS_OUTSTANDING.DEMAND_DATA_RD_GE_6</event>
        <event alias="c">OFFCORE_REQUESTS_OUTSTANDING.L3_MISS_DEMAND_DATA_RD_GE_6</event>
        <event alias="d">CPU_CLK_UNHALTED.THREAD</event>
        <event alias="e">OFFCORE_REQUESTS_OUTSTANDING.CYCLES_WITH_DEMAND_DATA_RD</event>
        <event alias="f">OFFCORE_REQUESTS_OUTSTANDING.CYCLES_WITH_L3_MISS_DEMAND_DATA_RD</event>
		<formula>100*((([1*e ,1*d].min-[1*f ,1*d].min)/d)-(([1*b ,1*d].min-[1*c ,1*d].min)/d))</formula>
    </metric>

	<metric name="metric_TMAM_......L3_Bandwidth(%)">
        <event alias="b">OFFCORE_REQUESTS_OUTSTANDING.DEMAND_DATA_RD_GE_6</event>
        <event alias="c">OFFCORE_REQUESTS_OUTSTANDING.L3_MISS_DEMAND_DATA_RD_GE_6</event>
        <event alias="d">CPU_CLK_UNHALTED.THREAD</event>
		<formula>100*([1*b ,1*d].min-[1*c ,1*d].min)/d</formula>
    </metric>

	<metric name="metric_TMAM_......SQ_Full(%)">
		<event alias="a">CPU_CLK_UNHALTED.THREAD_ANY</event>
		<event alias="b">OFFCORE_REQUESTS_BUFFER.SQ_FULL</event>
		<constant alias="threads">system.sockets[0][0].size</constant>      
		<formula>100*(b/threads)/(a/threads)</formula>
	</metric>
	
	<metric name="metric_TMAM_....MEM_Bound(%)">
        <event alias="a">CYCLE_ACTIVITY.STALLS_L3_MISS</event>
         <event alias="d">CPU_CLK_UNHALTED.THREAD</event>
		<formula>100*(a/d)</formula>
    </metric>

	<metric name="metric_TMAM_......MEM_Bandwidth(%)">
        <event alias="a">OFFCORE_REQUESTS_OUTSTANDING.L3_MISS_DEMAND_DATA_RD_GE_6</event>
        <event alias="b">CPU_CLK_UNHALTED.THREAD</event>
		<formula>100*([1*a, 1*b].min)/b</formula>
    </metric>

	<metric name="metric_TMAM_......MEM_Latency(%)">
        <event alias="a">OFFCORE_REQUESTS_OUTSTANDING.L3_MISS_DEMAND_DATA_RD_GE_6</event>
        <event alias="b">OFFCORE_REQUESTS_OUTSTANDING.CYCLES_WITH_L3_MISS_DEMAND_DATA_RD</event>
        <event alias="c">CPU_CLK_UNHALTED.THREAD</event>
		<formula>100*(([1*b, 1*c].min)-([1*a, 1*c].min))/c</formula>
    </metric>

	<metric name="metric_TMAM_....Stores_Bound(%)">
        <event alias="a">EXE_ACTIVITY.BOUND_ON_STORES</event>
        <event alias="b">CPU_CLK_UNHALTED.THREAD</event>
		<formula>100*(a/b)</formula>
    </metric>

	<metric name="metric_TMAM_......DTLB_Store(%)">
        <event alias="a">DTLB_STORE_MISSES.STLB_HIT</event>
        <event alias="b">DTLB_STORE_MISSES.WALK_ACTIVE</event>
        <event alias="c">CPU_CLK_UNHALTED.THREAD_ANY</event>
		<constant alias="threads">system.sockets[0][0].size</constant>      
		<formula>100*(7*a+b)/(c/threads)</formula>
    </metric>

	<metric name="metric_TMAM_..Core_Bound(%)">
		<event alias="a">IDQ_UOPS_NOT_DELIVERED.CORE</event>
		<event alias="b">UOPS_ISSUED.ANY</event>
		<event alias="c">INT_MISC.RECOVERY_CYCLES_ANY</event>
        <event alias="d">CPU_CLK_UNHALTED.THREAD_ANY</event>
        <event alias="e">UOPS_RETIRED.RETIRE_SLOTS</event>
        <event alias="f">CYCLE_ACTIVITY.STALLS_MEM_ANY</event>
        <event alias="g">EXE_ACTIVITY.BOUND_ON_STORES</event>
         <event alias="j">EXE_ACTIVITY.1_PORTS_UTIL</event>
        <event alias="k">EXE_ACTIVITY.2_PORTS_UTIL</event>
        <event alias="m">EXE_ACTIVITY.EXE_BOUND_0_PORTS</event>
        <event alias="p">CPU_CLK_UNHALTED.THREAD</event>
        <event alias="q">INST_RETIRED.ANY</event>
		<constant alias="threads">system.sockets[0][0].size</constant>      
		<formula>100*(1-((b-e+4*(c/threads)+a+e)/(4*d/threads)))*(1-((f+g)/(m+j+(((q/p)&gt;1.8)?k:0)+f+g)))</formula>
	</metric>

	<metric name="metric_TMAM_....Divider(%)">
        <event alias="a">ARITH.DIVIDER_ACTIVE</event>
        <event alias="b">CPU_CLK_UNHALTED.THREAD</event>
		<formula>100*a/b</formula>
    </metric>

	<metric name="metric_TMAM_....Ports_Utilization(%)">
        <event alias="d">EXE_ACTIVITY.EXE_BOUND_0_PORTS</event>
        <event alias="f">EXE_ACTIVITY.1_PORTS_UTIL</event>
        <event alias="g">CYCLE_ACTIVITY.STALLS_MEM_ANY</event>
        <event alias="h">EXE_ACTIVITY.BOUND_ON_STORES</event>
        <event alias="j">EXE_ACTIVITY.2_PORTS_UTIL</event>
        <event alias="p">CPU_CLK_UNHALTED.THREAD</event>
        <event alias="q">INST_RETIRED.ANY</event>
		<formula>100*((d+f+(((q/p)&gt;1.8)?j:0)+g+h)-g-h)/p</formula>
	</metric>

	<metric name="metric_TMAM_......0_Ports_Utilized(%)">
		<event alias="a">UOPS_EXECUTED.CORE_CYCLES_NONE</event>
		<event alias="b">CPU_CLK_UNHALTED.THREAD_ANY</event>
		<event alias="c">EXE_ACTIVITY.EXE_BOUND_0_PORTS</event>
		<constant alias="threads">system.sockets[0][0].size</constant>      
		<formula>100*((((threads)&gt;1)?(a/2):c)/(b/threads))</formula>		
	</metric>

	<metric name="metric_TMAM_......1_Port_Utilized(%)">
		<event alias="a">UOPS_EXECUTED.CORE_CYCLES_GE_1</event>
		<event alias="b">CPU_CLK_UNHALTED.THREAD_ANY</event>
		<event alias="c">UOPS_EXECUTED.CORE_CYCLES_GE_2</event>
		<event alias="d">EXE_ACTIVITY.1_PORTS_UTIL</event>
		<constant alias="threads">system.sockets[0][0].size</constant>      
		<formula>100*((((threads)&gt;1)?((a-c)/2):d)/(b/threads))</formula>			
	</metric>

	<metric name="metric_TMAM_......2_Ports_Utilized(%)">
		<event alias="a">UOPS_EXECUTED.CORE_CYCLES_GE_2</event>
		<event alias="b">CPU_CLK_UNHALTED.THREAD_ANY</event>
		<event alias="c">UOPS_EXECUTED.CORE_CYCLES_GE_3</event>
		<event alias="d">EXE_ACTIVITY.2_PORTS_UTIL</event>
		<constant alias="threads">system.sockets[0][0].size</constant>      
		<formula>100*((((threads)&gt;1)?((a-c)/2):d)/(b/threads))</formula>			
	</metric>	

	<metric name="metric_TMAM_......3m_Ports_Utilized(%)">
		<event alias="a">UOPS_EXECUTED.CORE_CYCLES_GE_3</event>
		<event alias="b">CPU_CLK_UNHALTED.THREAD_ANY</event>
		<constant alias="threads">system.sockets[0][0].size</constant>      
		<formula>100*(a/threads)/(b/threads)</formula>			
	</metric>	

	<metric name="metric_TMAM_Retiring(%)">
 		<event alias="a">CPU_CLK_UNHALTED.THREAD_ANY</event>
        <event alias="b">UOPS_RETIRED.RETIRE_SLOTS</event>
		<constant alias="threads">system.sockets[0][0].size</constant>      
		<formula>100*(b/(4*(a/threads)))</formula>
    </metric>

	<metric name="metric_TMAM_..Base(%)">
 		<event alias="a">CPU_CLK_UNHALTED.THREAD_ANY</event>
        <event alias="b">UOPS_RETIRED.RETIRE_SLOTS</event>
        <event alias="c">IDQ.MS_UOPS</event>
        <event alias="d">UOPS_ISSUED.ANY</event>
		<constant alias="threads">system.sockets[0][0].size</constant>      
		<formula>100*((b/(4*(a/threads)))-((b/d)*c/(4*(a/threads))))</formula>
    </metric>
	
	<metric name="metric_TMAM_....FP_Arith(%)">
 		<event alias="a">INST_RETIRED.ANY</event>
        <event alias="b">UOPS_EXECUTED.X87</event>
        <event alias="c">UOPS_RETIRED.RETIRE_SLOTS</event>
        <event alias="d">FP_ARITH_INST_RETIRED.SCALAR_SINGLE</event>
        <event alias="e">FP_ARITH_INST_RETIRED.SCALAR_DOUBLE</event>
        <event alias="f">FP_ARITH_INST_RETIRED.128B_PACKED_DOUBLE</event>
        <event alias="g">FP_ARITH_INST_RETIRED.128B_PACKED_SINGLE</event>
        <event alias="h">FP_ARITH_INST_RETIRED.256B_PACKED_SINGLE</event>
        <event alias="j">FP_ARITH_INST_RETIRED.256B_PACKED_DOUBLE</event>
        <event alias="k">UOPS_EXECUTED.THREAD</event>
        <event alias="m">FP_ARITH_INST_RETIRED.512B_PACKED_SINGLE</event>
        <event alias="n">FP_ARITH_INST_RETIRED.512B_PACKED_DOUBLE</event>
		<formula>100*((b/k)+((d+e+f+g+h+j+m+n)/c))</formula>
    </metric>

	<metric name="metric_TMAM_....Other(%)">
 		<event alias="a">INST_RETIRED.ANY</event>
        <event alias="b">UOPS_EXECUTED.X87</event>
        <event alias="c">UOPS_RETIRED.RETIRE_SLOTS</event>
        <event alias="d">FP_ARITH_INST_RETIRED.SCALAR_SINGLE</event>
        <event alias="e">FP_ARITH_INST_RETIRED.SCALAR_DOUBLE</event>
        <event alias="f">FP_ARITH_INST_RETIRED.128B_PACKED_DOUBLE</event>
        <event alias="g">FP_ARITH_INST_RETIRED.128B_PACKED_SINGLE</event>
        <event alias="h">FP_ARITH_INST_RETIRED.256B_PACKED_SINGLE</event>
        <event alias="j">FP_ARITH_INST_RETIRED.256B_PACKED_DOUBLE</event>
        <event alias="k">UOPS_EXECUTED.THREAD</event>
        <event alias="m">FP_ARITH_INST_RETIRED.512B_PACKED_SINGLE</event>
        <event alias="n">FP_ARITH_INST_RETIRED.512B_PACKED_DOUBLE</event>
		<formula>100*(1-((b/k)+((d+e+f+g+h+j+m+n)/c)))</formula>
    </metric>
	
	<metric name="metric_TMAM_..Microcode_Sequencer(%)">
 		<event alias="a">CPU_CLK_UNHALTED.THREAD_ANY</event>
        <event alias="b">UOPS_RETIRED.RETIRE_SLOTS</event>
        <event alias="c">IDQ.MS_UOPS</event>
        <event alias="d">UOPS_ISSUED.ANY</event>
		<constant alias="threads">system.sockets[0][0].size</constant>      
		<formula>100*((b/d)*c/(4*(a/threads)))</formula>
    </metric>

	<metric name="metric_EDP SKX XML version">
 		<constant alias="version">3.9</constant>      
		<formula>version</formula>
   </metric>
	
</root>'''
import xml.etree.ElementTree as ET
tree = ET.fromstring(metrics)
lst = tree.findall('metric')
metrics_list=[]
for item in lst:
    if len(item.findall('constant'))==0:
        out='''             '{:s}':{{
                                'sum_func':.self.cores_sum,   
                                'events':{{
                                      {:s}
                                }},
                                'formula':{{
                                       '{:s}':'{:s}'
                                }},
                                'fmt':lambda l: F.round(l, 3)
                            }}'''.format(item.get('name'),','.join(["'{:s}':'{:s}'".format(l.get('alias'),l.text) for l in item.findall('event')]),item.get('name'),item.findtext('formula'))
        metrics_list.append(out)
                

In [10]:
def get_alias_name(metric,func):
    return metric+"_"+func.__name__

In [3]:
def splits_fill0(x):
    fi=[]
    for l in x:
        li=re.split(r'\s+',l.strip())
        li=[l.replace(",","") for l in li]
        for j in range(len(li),256):
            li.append('0')
        fi.append(li)
    return iter(fi)

In [1]:
"ss,dd".replace(",","")

'ssdd'

## emon analysis

In [1]:
class Emon_Analysis(Analysis):
    def __init__(self,emon_file):
        Analysis.__init__(self,emon_file)
        
        paths=os.path.split(self.file)
        if fs.exists(paths[0]+"/emonv.txt"):
            with fs.open(paths[0]+"/emonv.txt") as f:
                allcnt = f.read().decode('ascii')
            for l in allcnt.split("\n"):
                if l.startswith("total_number_of_processors"):
                    self.totalcores=int(re.split(" +",l)[2])
                elif re.search("Number of Packages: +(\d+)",l):
                    self.numberofpackages=int(re.search("Number of Packages: +(\d+)",l).group(1))
                elif re.search("Cores Per Package: +(\d+)",l):
                    self.coresperpackage=int(re.search("Cores Per Package: +(\d+)",l).group(1))
                elif re.search("Threads Per Package: +(\d+)",l):
                    self.threadsperpackage=int(re.search("Threads Per Package: +(\d+)",l).group(1))
                elif re.search("TSC Freq +[.]+ +([0-9.]+)",l):
                    self.tsc=int(float(re.search("TSC Freq +[.]+ +([0-9.]+)",l).group(1))*1000000)
        else:
            print("Wrong, no emonv specified")


        if fs.exists(paths[0]+"/emon.rst"):
            with fs.open(paths[0]+"/emon.rst") as f:
                self.unc_cha_cnt = len(next((l.decode('ascii').split("\t") for l in f if "UNC_CHA_CLOCKTICKS" in l.decode('ascii')),""))-3
            
        self.begin_clk=0
        self.end_clk=0

#        paths=os.path.split(self.file)
#        if fs.exists(paths[0]+"/xgbtck.txt"):
#            with fs.open(paths[0]+"/xgbtck.txt") as f:
#                allcnt = f.read().decode('ascii')
#            if len(allcnt)>0:
#                begins=[]
#                ends=[]
#                for l in allcnt.split("\n"):
#                    if ("begin" in l):
#                        begins.append(l.split(" "))
#                    if ('end' in l):
#                        ends.append(l.split(" "))
#                if len(begins)>0:
#                    self.begin_clk=min([int(l[3]) for l in begins])
#                if len(ends)>0:
#                    self.end_clk=max([int(l[3]) for l in ends])
            
        self.corecnt=self.totalcores
        
        self.emon_metrics=collections.OrderedDict({
            'emon_cpuutil':{
                'sum_func':self.cores_sum,   
                'events':{
                    'a':'CPU_CLK_UNHALTED.REF_TSC'
                },
                'formula':{
                    'cpu%':'a/({:f}*{:d})'.format(self.tsc,self.corecnt)
                },
                'fmt':lambda l: F.round(l, 3)
            },
            'emon_cpufreq':{
                'sum_func':self.cores_sum,   
                'events':{
                    'a':'CPU_CLK_UNHALTED.THREAD',
                    'b':'CPU_CLK_UNHALTED.REF_TSC'
                },
                'formula':{
                    'cpu freq':'a/b*{:f}'.format(self.tsc/1000000)
                },
                'fmt':lambda l: F.round(l, 3)
            },
            'emon_instr_retired':{
                'sum_func':self.cores_sum,   
                'events':{
                    'a':'INST_RETIRED.ANY'
                },
                'formula':{
                    'pathlength':'a/1000000000'
                },
                'fmt':lambda l: F.round(l, 0)
            },
            'emon_ipc':{
                'sum_func':self.cores_sum,   
                'events':{
                    'a':'CPU_CLK_UNHALTED.THREAD',
                    'b':'INST_RETIRED.ANY'
                },
                'formula':{
                    'ipc':'b/a'
                },
                'fmt':lambda l: F.round(l, 3)
            },
            'emon_L3 stall':{'sum_func':self.cores_sum,   'events':{'a':'CYCLE_ACTIVITY.STALLS_L3_MISS','b':'CPU_CLK_UNHALTED.THREAD'},'formula':{'l3 stall':'a/b'},                         'fmt':lambda l: F.round(l, 3)},
            'emon_mem stall':{'sum_func':self.cores_sum,   'events':{'a':'CYCLE_ACTIVITY.CYCLES_MEM_ANY','b':'CPU_CLK_UNHALTED.THREAD'},'formula':{'mem stall':'a/b'},                         'fmt':lambda l: F.round(l, 3)},
            'emon_total stall':{'sum_func':self.cores_sum,   'events':{'a':'CYCLE_ACTIVITY.STALLS_TOTAL','b':'CPU_CLK_UNHALTED.THREAD'},'formula':{'total stall':'a/b'},                         'fmt':lambda l: F.round(l, 3)},
            'emon_dtlb walk':{'sum_func':self.cores_sum,   'events':{'a':'DTLB_LOAD_MISSES.WALK_ACTIVE','b':'CPU_CLK_UNHALTED.THREAD'},'formula':{'dtlb walk':'a/b'},                         'fmt':lambda l: F.round(l, 3)},
            'emon_uop issue stall':{'sum_func':self.cores_sum,   'events':{'a':'UOPS_ISSUED.STALL_CYCLES','b':'CPU_CLK_UNHALTED.THREAD'},'formula':{'issue stall':'a/b'},                         'fmt':lambda l: F.round(l, 3)},
           
            'emon_mem_bw':  {'sum_func':self.mem_sum,     'events':{'a':'UNC_M_CAS_COUNT.RD','b':'UNC_M_CAS_COUNT.WR'},                'formula':{'mem_bw_rd':'a*64/1000000','mem_bw_wr':'b*64/1000000'},   'fmt':lambda l: F.round(l, 0)},
            'emon_remote ratio':  {'sum_func':self.cores_sum,   'events':{'a':'UNC_CHA_TOR_INSERTS.IA_MISS_DRD_REMOTE','b':'UNC_CHA_TOR_INSERTS.IA_MISS_DRD_LOCAL'},  'formula':{'remote ratio':'a/(a+b)'},   'fmt':lambda l: F.round(l, 1)},
            
            
            'emon_local lat':  {'sum_func':self.cores_sum,   'events':{'a':'UNC_CHA_TOR_OCCUPANCY.IA_MISS_DRD_LOCAL','b':'UNC_CHA_TOR_INSERTS.IA_MISS_DRD_LOCAL','c':'UNC_CHA_CLOCKTICKS'},  'formula':{'local_mem_lat(ns)':'1000000000*a/b/c*{:d}'.format(self.unc_cha_cnt)},   'fmt':lambda l: F.round(l, 1)},
            'emon_remote lat':  {'sum_func':self.cores_sum,   'events':{'a':'UNC_CHA_TOR_OCCUPANCY.IA_MISS_DRD_REMOTE','b':'UNC_CHA_TOR_INSERTS.IA_MISS_DRD_REMOTE','c':'UNC_CHA_CLOCKTICKS'},  'formula':{'remote_mem_lat(ns)':'1000000000*a/b/c*{:d}'.format(self.unc_cha_cnt)},   'fmt':lambda l: F.round(l, 1)},
            
            'emon_fp_vec':  {'sum_func':self.cores_sum,   'events':{'a':'FP_ARITH_INST_RETIRED.VECTOR','b':'INST_RETIRED.ANY'},  'formula':{'fp_vec/all inst':'a/b'},   'fmt':lambda l: F.round(l, 3)},
            'emon_l3 miss':  {'sum_func':self.cores_sum,   'events':{'a':'MEM_LOAD_RETIRED.L3_MISS','b':'INST_RETIRED.ANY'},  'formula':{'l3 miss per 1K inst':'1000*a/b'},   'fmt':lambda l: F.round(l, 3)},
            'emon_l3 read miss bw':{'sum_func':self.cores_sum,   'events':{'a':'OCR.READS_TO_CORE.REMOTE_DRAM:ocr_msr_val=0x3fB80007f7','b':'OCR.READS_TO_CORE.LOCAL_DRAM:ocr_msr_val=0x3f840007f7'},  'formula':{'remote miss':'a*64/1000000','local miss':"b*64/1000000"},   'fmt':lambda l: F.round(l, 3)},
            
            
            'emon_faststring bw':  {'sum_func':self.cores_sum,   'events':{'a':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x49033'},  'formula':{'fast store bw':'a*64/1000000'},   'fmt':lambda l: F.round(l, 0)},
            'emon_pcie_bw': {
                'sum_func':self.pcie_sum,   
                'events':{
                    'a':'UNC_IIO_DATA_REQ_OF_CPU.MEM_WRITE.PART0',
                    'b':'UNC_IIO_DATA_REQ_OF_CPU.MEM_READ.PART0',
                    'c':'UNC_IIO_DATA_REQ_BY_CPU.MEM_WRITE.PART0',
                    'd':'UNC_IIO_DATA_REQ_BY_CPU.MEM_READ.PART0'
                },  
                'formula':{
                    'inbound write':'a*4/1000000',
                    'inbound read':'b*4/1000000',
                    'outbound write':'c*4/1000000',
                    'outbound read':'d*4/1000000'},   
                'fmt':lambda l: F.round(l, 0)
            },
            'emon_pcie_util':{
                'sum_func':self.pcie_sum,   
                'events':{
                    'a':'UNC_IIO_UTIL_IN.PORT0',
                    'b':'UNC_IIO_UTIL_OUT.PORT0',
                    'c':'UNC_IIO_CLOCKTICKS'
                },  
                'formula':{
                    'pcie %':'(a+b)/c'},   
                'fmt':lambda l: F.round(l, 2)
            },
            'emon_streaming bw':  {'sum_func':self.cores_sum,   'events':{'a':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x41833'},  'formula':{'streaming store bw':'a*64/1000000'},   'fmt':lambda l: F.round(l, 0)},
                         'metric_CPI':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'CPU_CLK_UNHALTED.THREAD','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_CPI':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_branch mispredict ratio':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'BR_MISP_RETIRED.ALL_BRANCHES','b':'BR_INST_RETIRED.ALL_BRANCHES'
                                },
                                'formula':{
                                       'metric_branch mispredict ratio':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_loads per instr':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'MEM_INST_RETIRED.ALL_LOADS','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_loads per instr':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_stores per instr':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'MEM_INST_RETIRED.ALL_STORES','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_stores per instr':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_locks retired per instr':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'MEM_INST_RETIRED.LOCK_LOADS','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_locks retired per instr':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_uncacheable reads per instr':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x40e33','c':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_uncacheable reads per instr':'a/c'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_streaming stores (full line) per instr':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x41833','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_streaming stores (full line) per instr':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_streaming stores (partial line) per instr':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x41a33','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_streaming stores (partial line) per instr':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_L1D MPI (includes data+rfo w/ prefetches)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'L1D.REPLACEMENT','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_L1D MPI (includes data+rfo w/ prefetches)':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_L1D demand data read hits per instr':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'MEM_LOAD_RETIRED.L1_HIT','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_L1D demand data read hits per instr':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_L1-I code read misses (w/ prefetches) per instr':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'L2_RQSTS.ALL_CODE_RD','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_L1-I code read misses (w/ prefetches) per instr':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_L2 demand data read hits per instr':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'MEM_LOAD_RETIRED.L2_HIT','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_L2 demand data read hits per instr':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_L2 MPI (includes code+data+rfo w/ prefetches)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'L2_LINES_IN.ALL','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_L2 MPI (includes code+data+rfo w/ prefetches)':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_L2 demand data read MPI':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'MEM_LOAD_RETIRED.L2_MISS','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_L2 demand data read MPI':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_L2 demand code MPI':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'L2_RQSTS.CODE_RD_MISS','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_L2 demand code MPI':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_L2 Any local request that HITM in a sibling core (per instr)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'OFFCORE_RESPONSE:request=ALL_READS:response=L3_HIT.HITM_OTHER_CORE','c':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_L2 Any local request that HITM in a sibling core (per instr)':'a/c'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_L2 Any local request that HIT in a sibling core and forwarded(per instr)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'OFFCORE_RESPONSE:request=ALL_READS:response=L3_HIT.HIT_OTHER_CORE_FWD','c':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_L2 Any local request that HIT in a sibling core and forwarded(per instr)':'a/c'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_L2 all L2 prefetches(per instr)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'L2_RQSTS.ALL_PF','c':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_L2 all L2 prefetches(per instr)':'a/c'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_L2 % of L2 evictions that are allocated into L3':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'L2_LINES_OUT.NON_SILENT','b':'IDI_MISC.WB_DOWNGRADE'
                                },
                                'formula':{
                                       'metric_L2 % of L2 evictions that are allocated into L3':'100*(a-b)/a'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_L2 % of L2 evictions that are NOT allocated into L3':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'L2_LINES_OUT.NON_SILENT','b':'IDI_MISC.WB_DOWNGRADE'
                                },
                                'formula':{
                                       'metric_L2 % of L2 evictions that are NOT allocated into L3':'100*b/a'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_LLC code references per instr (L3 prefetch excluded)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_TOR_INSERTS.IA:filter1=0x40233','d':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_LLC code references per instr (L3 prefetch excluded)':'a/d'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_LLC data read references per instr (L3 prefetch excluded)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_TOR_INSERTS.IA:filter1=0x40433','d':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_LLC data read references per instr (L3 prefetch excluded)':'a/d'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_LLC RFO references per instr (L3 prefetch excluded)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_TOR_INSERTS.IA:filter1=0x40033','d':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_LLC RFO references per instr (L3 prefetch excluded)':'a/d'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_LLC MPI (includes code+data+rfo w/ prefetches)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12D40433','b':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12CC0233','c':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12C40033','d':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_LLC MPI (includes code+data+rfo w/ prefetches)':'(a+b+c)/d'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_LLC data read MPI (demand+prefetch)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12D40433','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_LLC data read MPI (demand+prefetch)':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_LLC RFO read MPI (demand+prefetch)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12C40033','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_LLC RFO read MPI (demand+prefetch)':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_LLC code read MPI (demand+prefetch)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12CC0233','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_LLC code read MPI (demand+prefetch)':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_LLC all LLC prefetches (per instr)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12C4B433','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_LLC all LLC prefetches (per instr)':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_LLC total HITM (per instr) (excludes LLC prefetches)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'OFFCORE_RESPONSE:request=ALL_READS:response=L3_MISS.REMOTE_HITM','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_LLC total HITM (per instr) (excludes LLC prefetches)':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_LLC total HIT clean line forwards (per instr) (excludes LLC prefetches)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'OFFCORE_RESPONSE:request=ALL_READS:response=L3_MISS.REMOTE_HIT_FORWARD','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_LLC total HIT clean line forwards (per instr) (excludes LLC prefetches)':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_LLC % of LLC misses satisfied by remote caches':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12D40433','b':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12CC0233','c':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12C40033','d':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12C4B433','e':'OFFCORE_RESPONSE:request=ALL_READS:response=L3_MISS.REMOTE_HIT_FORWARD','f':'OFFCORE_RESPONSE:request=ALL_READS:response=L3_MISS.REMOTE_HITM'
                                },
                                'formula':{
                                       'metric_LLC % of LLC misses satisfied by remote caches':'100*(e+f)/(a+b+c-d)'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_SF snoop filter capacity evictions (per instr)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_SF_EVICTION.M_STATE','b':'UNC_CHA_SF_EVICTION.S_STATE','c':'UNC_CHA_SF_EVICTION.E_STATE','d':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_SF snoop filter capacity evictions (per instr)':'(a+b+c)/d'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_SF % of L3 accesses that result in SF capacity evictions':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_SF_EVICTION.M_STATE','b':'UNC_CHA_SF_EVICTION.S_STATE','c':'UNC_CHA_SF_EVICTION.E_STATE','d':'L2_LINES_IN.ALL'
                                },
                                'formula':{
                                       'metric_SF % of L3 accesses that result in SF capacity evictions':'100*(a+b+c)/d'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_ITLB MPI':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'ITLB_MISSES.WALK_COMPLETED','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_ITLB MPI':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_ITLB large page MPI':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'ITLB_MISSES.WALK_COMPLETED_2M_4M','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_ITLB large page MPI':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_DTLB load MPI':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'DTLB_LOAD_MISSES.WALK_COMPLETED','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_DTLB load MPI':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_DTLB 4KB page load MPI':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'DTLB_LOAD_MISSES.WALK_COMPLETED','b':'DTLB_LOAD_MISSES.WALK_COMPLETED_2M_4M','c':'DTLB_LOAD_MISSES.WALK_COMPLETED_1G','d':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_DTLB 4KB page load MPI':'(a-b-c)/d'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_DTLB 2MB large page load MPI':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'DTLB_LOAD_MISSES.WALK_COMPLETED_2M_4M','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_DTLB 2MB large page load MPI':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_DTLB 1GB large page load MPI':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'DTLB_LOAD_MISSES.WALK_COMPLETED_1G','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_DTLB 1GB large page load MPI':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_DTLB store MPI':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'DTLB_STORE_MISSES.WALK_COMPLETED','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_DTLB store MPI':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_DTLB load miss latency (in core clks)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'DTLB_LOAD_MISSES.WALK_ACTIVE','b':'DTLB_LOAD_MISSES.WALK_COMPLETED'
                                },
                                'formula':{
                                       'metric_DTLB load miss latency (in core clks)':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_DTLB store miss latency (in core clks)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'DTLB_STORE_MISSES.WALK_ACTIVE','b':'DTLB_STORE_MISSES.WALK_COMPLETED'
                                },
                                'formula':{
                                       'metric_DTLB store miss latency (in core clks)':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_ITLB miss latency (in core clks)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'ITLB_MISSES.WALK_ACTIVE','b':'ITLB_MISSES.WALK_COMPLETED'
                                },
                                'formula':{
                                       'metric_ITLB miss latency (in core clks)':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_NUMA %_Reads addressed to local DRAM':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x40432','b':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x40431'
                                },
                                'formula':{
                                       'metric_NUMA %_Reads addressed to local DRAM':'100*a/(a+b)'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_NUMA %_Reads addressed to remote DRAM':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x40432','b':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x40431'
                                },
                                'formula':{
                                       'metric_NUMA %_Reads addressed to remote DRAM':'100*b/(a+b)'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_NUMA %_RFOs addressed to local DRAM':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12C40031','b':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12C40033'
                                },
                                'formula':{
                                       'metric_NUMA %_RFOs addressed to local DRAM':'100*(b-a)/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_NUMA %_RFOs addressed to remote DRAM':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12C40031','b':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x12C40033'
                                },
                                'formula':{
                                       'metric_NUMA %_RFOs addressed to remote DRAM':'100*a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_UPI Data transmit BW (MB/sec) (only data)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_UPI_TxL_FLITS.ALL_DATA'
                                },
                                'formula':{
                                       'metric_UPI Data transmit BW (MB/sec) (only data)':'a*(64/9)/1000000'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_UPI % cycles transmit link is half-width (L0p)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_UPI_TxL0P_POWER_CYCLES','b':'UNC_UPI_CLOCKTICKS','f':'UNC_UPI_L1_POWER_CYCLES'
                                },
                                'formula':{
                                       'metric_UPI % cycles transmit link is half-width (L0p)':'100*(a/(b-f))'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_UPI % cycles receive link is half-width (L0p)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_UPI_RxL0P_POWER_CYCLES','b':'UNC_UPI_CLOCKTICKS','f':'UNC_UPI_L1_POWER_CYCLES'
                                },
                                'formula':{
                                       'metric_UPI % cycles receive link is half-width (L0p)':'100*(a/(b-f))'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_HA - Reads vs. all requests':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_REQUESTS.READS','b':'UNC_CHA_REQUESTS.WRITES'
                                },
                                'formula':{
                                       'metric_HA - Reads vs. all requests':'a/(a+b)'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_HA - Writes vs. all requests':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_REQUESTS.READS','b':'UNC_CHA_REQUESTS.WRITES'
                                },
                                'formula':{
                                       'metric_HA - Writes vs. all requests':'b/(a+b)'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_HA % of all reads that are local':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_REQUESTS.READS_LOCAL','b':'UNC_CHA_REQUESTS.READS'
                                },
                                'formula':{
                                       'metric_HA % of all reads that are local':'100*a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_HA % of all writes that are local':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_REQUESTS.WRITES_LOCAL','b':'UNC_CHA_REQUESTS.WRITES'
                                },
                                'formula':{
                                       'metric_HA % of all writes that are local':'100*a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_HA conflict responses per instr':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_SNOOP_RESP.RSPCNFLCTS','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_HA conflict responses per instr':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_HA directory lookups that spawned a snoop (per instr)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_DIR_LOOKUP.SNP','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_HA directory lookups that spawned a snoop (per instr)':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_HA directory lookups that did not spawn a snoop (per instr)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_DIR_LOOKUP.NO_SNP','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_HA directory lookups that did not spawn a snoop (per instr)':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_M2M directory updates (per instr)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_DIR_UPDATE.HA','b':'UNC_CHA_DIR_UPDATE.TOR','c':'UNC_M2M_DIRECTORY_UPDATE.ANY','d':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_M2M directory updates (per instr)':'(a+b+c)/d'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_M2M XPT prefetches (per instr)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M2M_PREFCAM_INSERTS','b':'UNC_M3UPI_UPI_PREFETCH_SPAWN','c':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_M2M XPT prefetches (per instr)':'(a-b)/c'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_M3UPI UPI prefetches (per instr)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'b':'UNC_M3UPI_UPI_PREFETCH_SPAWN','c':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_M3UPI UPI prefetches (per instr)':'b/c'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_M2M extra reads from XPT-UPI prefetches (per instr)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M2M_PREFCAM_INSERTS','b':'UNC_M2M_PREFCAM_DEMAND_PROMOTIONS','c':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_M2M extra reads from XPT-UPI prefetches (per instr)':'(a-b)/c'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_memory bandwidth read (MB/sec)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_CAS_COUNT.RD'
                                },
                                'formula':{
                                       'metric_memory bandwidth read (MB/sec)':'a*64/1000000'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_memory bandwidth write (MB/sec)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_CAS_COUNT.WR'
                                },
                                'formula':{
                                       'metric_memory bandwidth write (MB/sec)':'a*64/1000000'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_memory bandwidth total (MB/sec)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_CAS_COUNT.RD','b':'UNC_M_CAS_COUNT.WR'
                                },
                                'formula':{
                                       'metric_memory bandwidth total (MB/sec)':'(a+b)*64/1000000'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_memory extra read b/w due to XPT prefetches (MB/sec)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M2M_PREFCAM_INSERTS','b':'UNC_M2M_PREFCAM_DEMAND_PROMOTIONS'
                                },
                                'formula':{
                                       'metric_memory extra read b/w due to XPT prefetches (MB/sec)':'(a-b)*64/1000000'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_memory extra write b/w due to directory updates (MB/sec)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_DIR_UPDATE.HA','b':'UNC_CHA_DIR_UPDATE.TOR','c':'UNC_M2M_DIRECTORY_UPDATE.ANY'
                                },
                                'formula':{
                                       'metric_memory extra write b/w due to directory updates (MB/sec)':'(a+b+c)*64/1000000'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_2LM % of non-inclusive writes to near memory':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M2M_IMC_WRITES.NI','b':'UNC_M_DDRT_RDQ_INSERTS','c':'UNC_M_CAS_COUNT.WR'
                                },
                                'formula':{
                                       'metric_2LM % of non-inclusive writes to near memory':'100*a/(c-b)'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_2LM near memory cache read miss rate%':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M2M_TAG_HIT.NM_RD_HIT_CLEAN','b':'UNC_M2M_TAG_HIT.NM_RD_HIT_DIRTY','c':'UNC_M_DDRT_RDQ_INSERTS'
                                },
                                'formula':{
                                       'metric_2LM near memory cache read miss rate%':'100*c/(a+b+c)'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_memory avg entries in RPQ':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_CLOCKTICKS','c':'UNC_M_RPQ_OCCUPANCY'
                                },
                                'formula':{
                                       'metric_memory avg entries in RPQ':'c/a'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_memory avg entries in RPQ when not empty':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'c':'UNC_M_RPQ_OCCUPANCY','a':'UNC_M_RPQ_OCCUPANCY:t=1'
                                },
                                'formula':{
                                       'metric_memory avg entries in RPQ when not empty':'c/a'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_memory % cycles when RPQ is empty':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_CLOCKTICKS','c':'UNC_M_RPQ_OCCUPANCY:t=1'
                                },
                                'formula':{
                                       'metric_memory % cycles when RPQ is empty':'100*(1-c/a)'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_memory % cycles when RPQ has 1 or more entries':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_CLOCKTICKS','c':'UNC_M_RPQ_OCCUPANCY:t=1'
                                },
                                'formula':{
                                       'metric_memory % cycles when RPQ has 1 or more entries':'100*c/a'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_memory % cycles when RPQ has 10 or more entries':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_CLOCKTICKS','c':'UNC_M_RPQ_OCCUPANCY:t=10'
                                },
                                'formula':{
                                       'metric_memory % cycles when RPQ has 10 or more entries':'100*c/a'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_memory % cycles when RPQ has 20 or more entries':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_CLOCKTICKS','c':'UNC_M_RPQ_OCCUPANCY:t=20'
                                },
                                'formula':{
                                       'metric_memory % cycles when RPQ has 20 or more entries':'100*c/a'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_memory % cycles when RPQ has 40 or more entries':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_CLOCKTICKS','c':'UNC_M_RPQ_OCCUPANCY:t=40'
                                },
                                'formula':{
                                       'metric_memory % cycles when RPQ has 40 or more entries':'100*c/a'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_memory avg time (dclk) RPQ not empty':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_RPQ_OCCUPANCY:t=1:e1','c':'UNC_M_RPQ_OCCUPANCY:t=1'
                                },
                                'formula':{
                                       'metric_memory avg time (dclk) RPQ not empty':'c/a'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_memory avg time (dclk) RPQ empty':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_RPQ_OCCUPANCY:t=1:e1','c':'UNC_M_RPQ_OCCUPANCY:t=1','d':'UNC_M_CLOCKTICKS'
                                },
                                'formula':{
                                       'metric_memory avg time (dclk) RPQ empty':'(d-c)/a'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_memory avg time with 40 or more entries':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_RPQ_OCCUPANCY:t=40:e1','c':'UNC_M_RPQ_OCCUPANCY:t=40'
                                },
                                'formula':{
                                       'metric_memory avg time with 40 or more entries':'c/a'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_memory avg time with less than 40 entries)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_RPQ_OCCUPANCY:t=40:e1','c':'UNC_M_RPQ_OCCUPANCY:t=40','d':'UNC_M_CLOCKTICKS'
                                },
                                'formula':{
                                       'metric_memory avg time with less than 40 entries)':'(d-c)/a'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_3DXP_memory bandwidth read (MB/sec)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_DDRT_RDQ_INSERTS'
                                },
                                'formula':{
                                       'metric_3DXP_memory bandwidth read (MB/sec)':'a*64/1000000'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_3DXP_memory bandwidth write (MB/sec)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_DDRT_WPQ_INSERTS'
                                },
                                'formula':{
                                       'metric_3DXP_memory bandwidth write (MB/sec)':'a*64/1000000'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_3DXP_memory bandwidth total (MB/sec)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_DDRT_RDQ_INSERTS','b':'UNC_M_DDRT_WPQ_INSERTS'
                                },
                                'formula':{
                                       'metric_3DXP_memory bandwidth total (MB/sec)':'(a+b)*64/1000000'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_3DXP avg entries in RPQ when not empty':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'b':'UNC_M_DDRT_RDQ_OCCUPANCY.ALL:t=1','c':'UNC_M_DDRT_RDQ_OCCUPANCY.ALL'
                                },
                                'formula':{
                                       'metric_3DXP avg entries in RPQ when not empty':'c/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_3DXP % cycles when RPQ is empty':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_CLOCKTICKS','b':'UNC_M_DDRT_RDQ_OCCUPANCY.ALL:t=1'
                                },
                                'formula':{
                                       'metric_3DXP % cycles when RPQ is empty':'100*(1-(b/a))'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_3DXP avg time (dclk) RPQ not empty':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_DDRT_RDQ_OCCUPANCY.ALL:t=1:e1','b':'UNC_M_DDRT_RDQ_OCCUPANCY.ALL:t=1'
                                },
                                'formula':{
                                       'metric_3DXP avg time (dclk) RPQ not empty':'b/a'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_3DXP avg time (dclk) with 36 or more entries in RPQ':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_DDRT_RDQ_OCCUPANCY.ALL:t=36:e1','b':'UNC_M_DDRT_RDQ_OCCUPANCY.ALL:t=36'
                                },
                                'formula':{
                                       'metric_3DXP avg time (dclk) with 36 or more entries in RPQ':'b/a'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_3DXP avg entries in WPQ when not empty':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'b':'UNC_M_DDRT_WPQ_OCCUPANCY.ALL:t=1','c':'UNC_M_DDRT_WPQ_OCCUPANCY.ALL'
                                },
                                'formula':{
                                       'metric_3DXP avg entries in WPQ when not empty':'c/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_3DXP avg time (dclk) WPQ not empty':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_DDRT_WPQ_OCCUPANCY.ALL:t=1:e1','b':'UNC_M_DDRT_WPQ_OCCUPANCY.ALL:t=1'
                                },
                                'formula':{
                                       'metric_3DXP avg time (dclk) WPQ not empty':'b/a'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_3DXP avg time (dclk) with 30 or more entries in WPQ':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_DDRT_WPQ_OCCUPANCY.ALL:t=30:e1','b':'UNC_M_DDRT_WPQ_OCCUPANCY.ALL:t=30'
                                },
                                'formula':{
                                       'metric_3DXP avg time (dclk) with 30 or more entries in WPQ':'b/a'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_CHA % cyles Fast asserted':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_FAST_ASSERTED.HORZ:u0x1','c':'UNC_CHA_CLOCKTICKS'
                                },
                                'formula':{
                                       'metric_CHA % cyles Fast asserted':'100*a/c'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_CHA RxC IRQ avg entries':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_RxC_OCCUPANCY.IRQ','c':'UNC_CHA_CLOCKTICKS'
                                },
                                'formula':{
                                       'metric_CHA RxC IRQ avg entries':'a/c'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_CHA RxC IRQ % cycles when Q has 18 or more entries':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_CLOCKTICKS','c':'UNC_CHA_RxC_OCCUPANCY.IRQ:t=18'
                                },
                                'formula':{
                                       'metric_CHA RxC IRQ % cycles when Q has 18 or more entries':'100*c/a'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_M2M avg entries in TxC AD Q':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_CLOCKTICKS','c':'UNC_M2M_TxC_AD_OCCUPANCY'
                                },
                                'formula':{
                                       'metric_M2M avg entries in TxC AD Q':'c/a'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_M2M avg entries in TxC BL Q':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_CLOCKTICKS','c':'UNC_M2M_TxC_BL_OCCUPANCY.ALL'
                                },
                                'formula':{
                                       'metric_M2M avg entries in TxC BL Q':'c/a'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_M2M RxC AD avg entries':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M2M_RxC_AD_OCCUPANCY','c':'UNC_CHA_CLOCKTICKS'
                                },
                                'formula':{
                                       'metric_M2M RxC AD avg entries':'a/c'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_M2M RxC BL avg entries':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M2M_RxC_BL_OCCUPANCY','c':'UNC_CHA_CLOCKTICKS'
                                },
                                'formula':{
                                       'metric_M2M RxC BL avg entries':'a/c'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_IO_bandwidth_disk_or_network_writes (MB/sec)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_IIO_PAYLOAD_BYTES_IN.MEM_READ.PART0','b':'UNC_IIO_PAYLOAD_BYTES_IN.MEM_READ.PART1','c':'UNC_IIO_PAYLOAD_BYTES_IN.MEM_READ.PART2','d':'UNC_IIO_PAYLOAD_BYTES_IN.MEM_READ.PART3'
                                },
                                'formula':{
                                       'metric_IO_bandwidth_disk_or_network_writes (MB/sec)':'(a+b+c+d)*4/1000000'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_IO_bandwidth_disk_or_network_reads (MB/sec)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_IIO_PAYLOAD_BYTES_IN.MEM_WRITE.PART0','b':'UNC_IIO_PAYLOAD_BYTES_IN.MEM_WRITE.PART1','c':'UNC_IIO_PAYLOAD_BYTES_IN.MEM_WRITE.PART2','d':'UNC_IIO_PAYLOAD_BYTES_IN.MEM_WRITE.PART3'
                                },
                                'formula':{
                                       'metric_IO_bandwidth_disk_or_network_reads (MB/sec)':'(a+b+c+d)*4/1000000'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_IO_number of partial PCI writes per sec':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_TOR_INSERTS.IO_HIT:filter1=0x40033','b':'UNC_CHA_TOR_INSERTS.IO_MISS:filter1=0x40033'
                                },
                                'formula':{
                                       'metric_IO_number of partial PCI writes per sec':'a+b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_IO_read cache miss(disk/network writes) bandwidth (MB/sec)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_TOR_INSERTS.IO_MISS:filter1=0x43c33'
                                },
                                'formula':{
                                       'metric_IO_read cache miss(disk/network writes) bandwidth (MB/sec)':'a*64/1000000'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_IO_write cache miss(disk/network reads) bandwidth (MB/sec)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_TOR_INSERTS.IO_MISS:filter1=0x49033','b':'UNC_CHA_TOR_INSERTS.IO_MISS:filter1=0x40033'
                                },
                                'formula':{
                                       'metric_IO_write cache miss(disk/network reads) bandwidth (MB/sec)':'(a+b)*64/1000000'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_IONUMA % disk/network reads addressed to local memory':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_TOR_INSERTS.IO:filter1=0x49031','b':'UNC_CHA_TOR_INSERTS.IO:filter1=0x49032','c':'UNC_CHA_TOR_INSERTS.IO:filter1=0x40031','d':'UNC_CHA_TOR_INSERTS.IO:filter1=0x40032'
                                },
                                'formula':{
                                       'metric_IONUMA % disk/network reads addressed to local memory':'100*(b+d)/(a+b+c+d)'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_IONUMA % disk/network reads addressed to remote memory':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_TOR_INSERTS.IO:filter1=0x49031','b':'UNC_CHA_TOR_INSERTS.IO:filter1=0x49032','c':'UNC_CHA_TOR_INSERTS.IO:filter1=0x40031','d':'UNC_CHA_TOR_INSERTS.IO:filter1=0x40032'
                                },
                                'formula':{
                                       'metric_IONUMA % disk/network reads addressed to remote memory':'100*(a+c)/(a+b+c+d)'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_MMIO reads per instr':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x40040e33','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_MMIO reads per instr':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_MMIO writes per instr':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x40041e33','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_MMIO writes per instr':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_memory reads vs. all requests':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_CAS_COUNT.RD','b':'UNC_M_CAS_COUNT.WR'
                                },
                                'formula':{
                                       'metric_memory reads vs. all requests':'a/(a+b)'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_memory Page Empty vs. all requests':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_PRE_COUNT.RD:u0xc','c':'UNC_M_PRE_COUNT.PAGE_MISS','d':'UNC_M_CAS_COUNT.RD','e':'UNC_M_CAS_COUNT.WR'
                                },
                                'formula':{
                                       'metric_memory Page Empty vs. all requests':'(a-c)/(d+e)'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_memory Page Misses vs. all requests':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'b':'UNC_M_PRE_COUNT.PAGE_MISS','c':'UNC_M_CAS_COUNT.RD','d':'UNC_M_CAS_COUNT.WR'
                                },
                                'formula':{
                                       'metric_memory Page Misses vs. all requests':'b/(c+d)'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_memory Page Hits vs. all requests':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_PRE_COUNT.RD:u0xc','c':'UNC_M_CAS_COUNT.RD','d':'UNC_M_CAS_COUNT.WR'
                                },
                                'formula':{
                                       'metric_memory Page Hits vs. all requests':'1-(a/(c+d))'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_memory % Cycles where all DRAM ranks are in PPD mode':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_POWER_CHANNEL_PPD','b':'UNC_M_CLOCKTICKS'
                                },
                                'formula':{
                                       'metric_memory % Cycles where all DRAM ranks are in PPD mode':'100*a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_memory % Cycles Memory is in self refresh power mode':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M_POWER_SELF_REFRESH','b':'UNC_M_CLOCKTICKS'
                                },
                                'formula':{
                                       'metric_memory % Cycles Memory is in self refresh power mode':'100*a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_ItoM operations (fast strings) that reference LLC per instr':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_TOR_INSERTS.IA_HIT:filter1=0x49033','b':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x49033','c':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_ItoM operations (fast strings) that reference LLC per instr':'(a+b)/c'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_ItoM operations (fast strings) that miss LLC per instr':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_CHA_TOR_INSERTS.IA_MISS:filter1=0x49033','c':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_ItoM operations (fast strings) that miss LLC per instr':'a/c'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_% Uops delivered from decoded Icache (DSB)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'IDQ.DSB_UOPS','b':'UOPS_ISSUED.ANY'
                                },
                                'formula':{
                                       'metric_% Uops delivered from decoded Icache (DSB)':'100*(a/b)'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_% Uops delivered from legacy decode pipeline (MITE)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'IDQ.MITE_UOPS','b':'UOPS_ISSUED.ANY'
                                },
                                'formula':{
                                       'metric_% Uops delivered from legacy decode pipeline (MITE)':'100*(a/b)'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_% Uops delivered from microcode sequencer (MS)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'IDQ.MS_UOPS','b':'UOPS_ISSUED.ANY'
                                },
                                'formula':{
                                       'metric_% Uops delivered from microcode sequencer (MS)':'100*(a/b)'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_% Uops delivered from loop stream detector (LSD)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'LSD.UOPS','b':'UOPS_ISSUED.ANY'
                                },
                                'formula':{
                                       'metric_% Uops delivered from loop stream detector (LSD)':'100*(a/b)'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_FP scalar single-precision FP instructions retired per instr':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'FP_ARITH_INST_RETIRED.SCALAR_SINGLE','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_FP scalar single-precision FP instructions retired per instr':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_FP scalar double-precision FP instructions retired per instr':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'FP_ARITH_INST_RETIRED.SCALAR_DOUBLE','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_FP scalar double-precision FP instructions retired per instr':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_FP 128-bit packed single-precision FP instructions retired per instr':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'FP_ARITH_INST_RETIRED.128B_PACKED_SINGLE','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_FP 128-bit packed single-precision FP instructions retired per instr':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_FP 128-bit packed double-precision FP instructions retired per instr':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'FP_ARITH_INST_RETIRED.128B_PACKED_DOUBLE','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_FP 128-bit packed double-precision FP instructions retired per instr':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_FP 256-bit packed single-precision FP instructions retired per instr':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'FP_ARITH_INST_RETIRED.256B_PACKED_SINGLE','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_FP 256-bit packed single-precision FP instructions retired per instr':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_FP 256-bit packed double-precision FP instructions retired per instr':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'FP_ARITH_INST_RETIRED.256B_PACKED_DOUBLE','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_FP 256-bit packed double-precision FP instructions retired per instr':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_FP 512-bit packed single-precision FP instructions retired per instr':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'FP_ARITH_INST_RETIRED.512B_PACKED_SINGLE','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_FP 512-bit packed single-precision FP instructions retired per instr':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_FP 512-bit packed double-precision FP instructions retired per instr':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'FP_ARITH_INST_RETIRED.512B_PACKED_DOUBLE','b':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_FP 512-bit packed double-precision FP instructions retired per instr':'a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_DRAM power (watts)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'MSR_EVENT:msr=0x619:type=FREERUN:scope=PACKAGE'
                                },
                                'formula':{
                                       'metric_DRAM power (watts)':'a*15.3/1000000'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_package power (watts)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'MSR_EVENT:msr=0x611:type=FREERUN:scope=PACKAGE'
                                },
                                'formula':{
                                       'metric_package power (watts)':'a*61/1000000'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_core c6 residency %':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'MSR_EVENT:msr=0x3FD:type=FREERUN:scope=THREAD','b':'TSC'
                                },
                                'formula':{
                                       'metric_core c6 residency %':'100*a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_core SW prefetch NTA per instr':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'SW_PREFETCH_ACCESS.NTA','c':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_core SW prefetch NTA per instr':'a/c'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_core uncacheable access and clflushes per instr':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'OFFCORE_REQUESTS.MEM_UC','c':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_core uncacheable access and clflushes per instr':'a/c'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_core % cycles core power throttled':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'CORE_POWER.THROTTLE','c':'CPU_CLK_UNHALTED.THREAD'
                                },
                                'formula':{
                                       'metric_core % cycles core power throttled':'100*a/c'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_core % cycles in non AVX license':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'CORE_POWER.LVL0_TURBO_LICENSE','b':'CORE_POWER.LVL1_TURBO_LICENSE','c':'CORE_POWER.LVL2_TURBO_LICENSE'
                                },
                                'formula':{
                                       'metric_core % cycles in non AVX license':'100*a/(a+b+c)'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_core % cycles in AVX2 license':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'CORE_POWER.LVL0_TURBO_LICENSE','b':'CORE_POWER.LVL1_TURBO_LICENSE','c':'CORE_POWER.LVL2_TURBO_LICENSE'
                                },
                                'formula':{
                                       'metric_core % cycles in AVX2 license':'100*b/(a+b+c)'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_core % cycles in AVX-512 license':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'CORE_POWER.LVL0_TURBO_LICENSE','b':'CORE_POWER.LVL1_TURBO_LICENSE','c':'CORE_POWER.LVL2_TURBO_LICENSE'
                                },
                                'formula':{
                                       'metric_core % cycles in AVX-512 license':'100*c/(a+b+c)'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_SMI number of SMIs per sec':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'MSR_EVENT:msr=0x34:type=FREERUN:scope=PACKAGE'
                                },
                                'formula':{
                                       'metric_SMI number of SMIs per sec':'a+0'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_core initiated local dram read bandwidth (MB/sec)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'OFFCORE_RESPONSE:request=ALL_READS:response=L3_MISS.ANY_SNOOP:ocr_msr_val=0x3f840007f7'
                                },
                                'formula':{
                                       'metric_core initiated local dram read bandwidth (MB/sec)':'a*64/1000000'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_core initiated remote dram read bandwidth (MB/sec)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'OFFCORE_RESPONSE:request=ALL_READS:response=L3_MISS.ANY_SNOOP:ocr_msr_val=0x3fB80007f7'
                                },
                                'formula':{
                                       'metric_core initiated remote dram read bandwidth (MB/sec)':'a*64/1000000'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_core initiated local DCPMEM read bandwidth (MB/sec)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'OFFCORE_RESPONSE:request=ALL_READS:response=L3_MISS.ANY_SNOOP:ocr_msr_val=0x3f804007f7'
                                },
                                'formula':{
                                       'metric_core initiated local DCPMEM read bandwidth (MB/sec)':'a*64/1000000'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_core initiated remote DCPMEM read bandwidth (MB/sec)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'OFFCORE_RESPONSE:request=ALL_READS:response=L3_MISS.ANY_SNOOP:ocr_msr_val=0x3f838007f7'
                                },
                                'formula':{
                                       'metric_core initiated remote DCPMEM read bandwidth (MB/sec)':'a*64/1000000'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_M3UPI avg all VN0 entries':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M3UPI_CLOCKTICKS','c':'UNC_M3UPI_RxC_OCCUPANCY_VN0.BL_RSP:u0x7f'
                                },
                                'formula':{
                                       'metric_M3UPI avg all VN0 entries':'c/a'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_M3UPI % cycles when all VN0 has 1 or more entries':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M3UPI_CLOCKTICKS','c':'UNC_M3UPI_RxC_OCCUPANCY_VN0.BL_RSP:u0x7f:t=1'
                                },
                                'formula':{
                                       'metric_M3UPI % cycles when all VN0 has 1 or more entries':'100*c/a'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_M3UPI % cycles when all VN0 has 10 or more entries':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M3UPI_CLOCKTICKS','c':'UNC_M3UPI_RxC_OCCUPANCY_VN0.BL_RSP:u0x7f:t=10'
                                },
                                'formula':{
                                       'metric_M3UPI % cycles when all VN0 has 10 or more entries':'100*c/a'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_M3UPI % cycles when all VN0 has 30 or more entries':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M3UPI_CLOCKTICKS','c':'UNC_M3UPI_RxC_OCCUPANCY_VN0.BL_RSP:u0x7f:t=30'
                                },
                                'formula':{
                                       'metric_M3UPI % cycles when all VN0 has 30 or more entries':'100*c/a'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_M3UPI % cycles when all VN0 has 50 or more entries':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'UNC_M3UPI_CLOCKTICKS','c':'UNC_M3UPI_RxC_OCCUPANCY_VN0.BL_RSP:u0x7f:t=50'
                                },
                                'formula':{
                                       'metric_M3UPI % cycles when all VN0 has 50 or more entries':'100*c/a'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_....ICache_Misses(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'ICACHE_16B.IFDATA_STALL','b':'ICACHE_16B.IFDATA_STALL:c1:e1','d':'CPU_CLK_UNHALTED.THREAD'
                                },
                                'formula':{
                                       'metric_TMAM_....ICache_Misses(%)':'100*(a+2*b)/d'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_....ITLB_Misses(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'ICACHE_64B.IFTAG_STALL','d':'CPU_CLK_UNHALTED.THREAD'
                                },
                                'formula':{
                                       'metric_TMAM_....ITLB_Misses(%)':'100*a/d'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_....Branch_Resteers(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'INT_MISC.CLEAR_RESTEER_CYCLES','b':'BACLEARS.ANY','j':'CPU_CLK_UNHALTED.THREAD'
                                },
                                'formula':{
                                       'metric_TMAM_....Branch_Resteers(%)':'100*(a+9*b)/j'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_......Mispredicts_Resteers(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'INT_MISC.CLEAR_RESTEER_CYCLES','b':'MACHINE_CLEARS.COUNT','c':'BR_MISP_RETIRED.ALL_BRANCHES','j':'CPU_CLK_UNHALTED.THREAD'
                                },
                                'formula':{
                                       'metric_TMAM_......Mispredicts_Resteers(%)':'100*a*(c/(b+c))/j'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_......Clears_Resteers(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'INT_MISC.CLEAR_RESTEER_CYCLES','b':'MACHINE_CLEARS.COUNT','c':'BR_MISP_RETIRED.ALL_BRANCHES','j':'CPU_CLK_UNHALTED.THREAD'
                                },
                                'formula':{
                                       'metric_TMAM_......Clears_Resteers(%)':'100*a*(1-(c/(b+c)))/j'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_......Unknown_Branches_Resteers(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'b':'BACLEARS.ANY','j':'CPU_CLK_UNHALTED.THREAD'
                                },
                                'formula':{
                                       'metric_TMAM_......Unknown_Branches_Resteers(%)':'100*(9*b)/j'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_....DSB_Switches(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'DSB2MITE_SWITCHES.PENALTY_CYCLES','d':'CPU_CLK_UNHALTED.THREAD'
                                },
                                'formula':{
                                       'metric_TMAM_....DSB_Switches(%)':'100*a/d'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_....MS_Switches(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'IDQ.MS_SWITCHES','d':'CPU_CLK_UNHALTED.THREAD'
                                },
                                'formula':{
                                       'metric_TMAM_....MS_Switches(%)':'100*2*a/d'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_....L1_Bound(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'CYCLE_ACTIVITY.STALLS_MEM_ANY','b':'CYCLE_ACTIVITY.STALLS_L1D_MISS','c':'CPU_CLK_UNHALTED.THREAD'
                                },
                                'formula':{
                                       'metric_TMAM_....L1_Bound(%)':'100*(a-b)/c'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_......DTLB_Load(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'DTLB_LOAD_MISSES.STLB_HIT','b':'DTLB_LOAD_MISSES.WALK_ACTIVE','c':'CPU_CLK_UNHALTED.THREAD'
                                },
                                'formula':{
                                       'metric_TMAM_......DTLB_Load(%)':'100*(7*a+b)/c'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_......Store_Fwd_Blk(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'LD_BLOCKS.STORE_FORWARD','b':'CPU_CLK_UNHALTED.THREAD'
                                },
                                'formula':{
                                       'metric_TMAM_......Store_Fwd_Blk(%)':'100*((13 * a) / b) '
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_......Lock_Latency(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'MEM_INST_RETIRED.LOCK_LOADS','b':'CPU_CLK_UNHALTED.THREAD','c':'MEM_INST_RETIRED.ALL_STORES','d':'OFFCORE_REQUESTS_OUTSTANDING.CYCLES_WITH_DEMAND_RFO'
                                },
                                'formula':{
                                       'metric_TMAM_......Lock_Latency(%)':'100*(((a / c)* (case when b > d then d else b end))/ b) '
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_....L2_Bound(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'CYCLE_ACTIVITY.STALLS_L1D_MISS','b':'CYCLE_ACTIVITY.STALLS_L2_MISS','c':'CPU_CLK_UNHALTED.THREAD'
                                },
                                'formula':{
                                       'metric_TMAM_....L2_Bound(%)':'100*(a-b)/c'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_....L3_Bound(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'CYCLE_ACTIVITY.STALLS_L2_MISS','c':'CYCLE_ACTIVITY.STALLS_L3_MISS','d':'CPU_CLK_UNHALTED.THREAD'
                                },
                                'formula':{
                                       'metric_TMAM_....L3_Bound(%)':'100*(a-c)/d'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_......Contested_Accesses(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'MEM_LOAD_L3_HIT_RETIRED.XSNP_HITM','b':'CPU_CLK_UNHALTED.THREAD','c':'MEM_LOAD_L3_HIT_RETIRED.XSNP_MISS'
                                },
                                'formula':{
                                       'metric_TMAM_......Contested_Accesses(%)':'100*(60 * ( a + c ) / b) '
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_......Data_Sharing(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'MEM_LOAD_L3_HIT_RETIRED.XSNP_HIT','b':'CPU_CLK_UNHALTED.THREAD'
                                },
                                'formula':{
                                       'metric_TMAM_......Data_Sharing(%)':'100*(43 * a /b) '
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_......L3_Latency(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'b':'OFFCORE_REQUESTS_OUTSTANDING.DEMAND_DATA_RD_GE_6','c':'OFFCORE_REQUESTS_OUTSTANDING.L3_MISS_DEMAND_DATA_RD_GE_6','d':'CPU_CLK_UNHALTED.THREAD','e':'OFFCORE_REQUESTS_OUTSTANDING.CYCLES_WITH_DEMAND_DATA_RD','f':'OFFCORE_REQUESTS_OUTSTANDING.CYCLES_WITH_L3_MISS_DEMAND_DATA_RD'
                                },
                                'formula':{
                                       'metric_TMAM_......L3_Latency(%)':'100*((((case when e > d then d else e end)-(case when f > d then d else f end))/d)-(((case when b > d then d else b end)-(case when d > d then d else c end))/d))'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_......L3_Bandwidth(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'b':'OFFCORE_REQUESTS_OUTSTANDING.DEMAND_DATA_RD_GE_6','c':'OFFCORE_REQUESTS_OUTSTANDING.L3_MISS_DEMAND_DATA_RD_GE_6','d':'CPU_CLK_UNHALTED.THREAD'
                                },
                                'formula':{
                                       'metric_TMAM_......L3_Bandwidth(%)':'100*((case when b > d then d else b end)-(case when c > d then d else c end))/d'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_....MEM_Bound(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'CYCLE_ACTIVITY.STALLS_L3_MISS','d':'CPU_CLK_UNHALTED.THREAD'
                                },
                                'formula':{
                                       'metric_TMAM_....MEM_Bound(%)':'100*(a/d)'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_......MEM_Bandwidth(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'OFFCORE_REQUESTS_OUTSTANDING.L3_MISS_DEMAND_DATA_RD_GE_6','b':'CPU_CLK_UNHALTED.THREAD'
                                },
                                'formula':{
                                       'metric_TMAM_......MEM_Bandwidth(%)':'100*((case when a > b then b else a end))/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_......MEM_Latency(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'OFFCORE_REQUESTS_OUTSTANDING.L3_MISS_DEMAND_DATA_RD_GE_6','b':'OFFCORE_REQUESTS_OUTSTANDING.CYCLES_WITH_L3_MISS_DEMAND_DATA_RD','c':'CPU_CLK_UNHALTED.THREAD'
                                },
                                'formula':{
                                       'metric_TMAM_......MEM_Latency(%)':'100*(((case when b > c then c else b end))-((case when a > c then c else a end)))/c'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_....Stores_Bound(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'EXE_ACTIVITY.BOUND_ON_STORES','b':'CPU_CLK_UNHALTED.THREAD'
                                },
                                'formula':{
                                       'metric_TMAM_....Stores_Bound(%)':'100*(a/b)'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_....Divider(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'ARITH.DIVIDER_ACTIVE','b':'CPU_CLK_UNHALTED.THREAD'
                                },
                                'formula':{
                                       'metric_TMAM_....Divider(%)':'100*a/b'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_....Ports_Utilization(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'd':'EXE_ACTIVITY.EXE_BOUND_0_PORTS','f':'EXE_ACTIVITY.1_PORTS_UTIL','g':'CYCLE_ACTIVITY.STALLS_MEM_ANY','h':'EXE_ACTIVITY.BOUND_ON_STORES','j':'EXE_ACTIVITY.2_PORTS_UTIL','p':'CPU_CLK_UNHALTED.THREAD','q':'INST_RETIRED.ANY'
                                },
                                'formula':{
                                       'metric_TMAM_....Ports_Utilization(%)':'100*((d+f+(case when ((q/p)>1.8) then j else 0 end)+g+h)-g-h)/p'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_....FP_Arith(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'INST_RETIRED.ANY','b':'UOPS_EXECUTED.X87','c':'UOPS_RETIRED.RETIRE_SLOTS','d':'FP_ARITH_INST_RETIRED.SCALAR_SINGLE','e':'FP_ARITH_INST_RETIRED.SCALAR_DOUBLE','f':'FP_ARITH_INST_RETIRED.128B_PACKED_DOUBLE','g':'FP_ARITH_INST_RETIRED.128B_PACKED_SINGLE','h':'FP_ARITH_INST_RETIRED.256B_PACKED_SINGLE','j':'FP_ARITH_INST_RETIRED.256B_PACKED_DOUBLE','k':'UOPS_EXECUTED.THREAD','m':'FP_ARITH_INST_RETIRED.512B_PACKED_SINGLE','n':'FP_ARITH_INST_RETIRED.512B_PACKED_DOUBLE'
                                },
                                'formula':{
                                       'metric_TMAM_....FP_Arith(%)':'100*((b/k)+((d+e+f+g+h+j+m+n)/c))'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            },
             'metric_TMAM_....Other(%)':{
                                'sum_func':self.cores_sum,   
                                'events':{
                                      'a':'INST_RETIRED.ANY','b':'UOPS_EXECUTED.X87','c':'UOPS_RETIRED.RETIRE_SLOTS','d':'FP_ARITH_INST_RETIRED.SCALAR_SINGLE','e':'FP_ARITH_INST_RETIRED.SCALAR_DOUBLE','f':'FP_ARITH_INST_RETIRED.128B_PACKED_DOUBLE','g':'FP_ARITH_INST_RETIRED.128B_PACKED_SINGLE','h':'FP_ARITH_INST_RETIRED.256B_PACKED_SINGLE','j':'FP_ARITH_INST_RETIRED.256B_PACKED_DOUBLE','k':'UOPS_EXECUTED.THREAD','m':'FP_ARITH_INST_RETIRED.512B_PACKED_SINGLE','n':'FP_ARITH_INST_RETIRED.512B_PACKED_DOUBLE'
                                },
                                'formula':{
                                       'metric_TMAM_....Other(%)':'100*(1-((b/k)+((d+e+f+g+h+j+m+n)/c)))'
                                },
                                'fmt':lambda l: F.round(l, 3)
                            }
        })
        self.effective_metric=None
        self.appclients=[] # there is no appid and client column

    def count_sum(self,collected_cores):
        return F.expr('+'.join(['_{:d}/_2*{:d}'.format(c+3,self.tsc) for c in collected_cores]))

    def cores_sum(self,collected_cores):
        return self.count_sum(collected_cores)

    def mem_sum(self,collected_cores):
        return self.count_sum(list(range(40)))

    def pcie_sum(self,collected_cores):
        return self.count_sum([2,3,7,8])
        
    def list_metric(self):
        if self.effective_metric is None:
            self.get_effective_metric()
        for k in self.effective_metric:
            m=self.emon_metrics[k]
            print(k)
            for fk,fm in m['formula'].items():
                print("    ",fk)
            
    def load_data(self):
        paths=os.path.split(self.file)
        if fs.exists(paths[0]+"/emon.parquet/_SUCCESS"):
            self.df=spark.read.parquet(paths[0]+"/emon.parquet")
            self.df.cache()
            return
        
        emondata=sc.textFile(self.file)
        emondf=emondata.mapPartitions(splits_fill0).toDF()
        emondf=emondf.withColumn("id", F.monotonically_increasing_id())
        giddf=emondf.where(emondf._1.rlike("======")).selectExpr("id as g_id")
        
        iddf=emondf.where(emondf._1.rlike("\d\d/")).selectExpr("_1 as r_1","_2 as r_2","id as r_id")
        jfid=emondf.where(emondf._1.rlike("[A-Z]{3}")).join(iddf,on=[emondf.id>iddf.r_id]).groupBy('id').agg(F.max('r_id').alias('r_id'))
        iddf=iddf.join(jfid,on='r_id',how='left')
        emondf=emondf.where(emondf._1.rlike("[A-Z]{3}")).join(iddf,on='id',how='left')
        
        jfid=emondf.join(giddf,on=[emondf.id>giddf.g_id]).groupBy('id').agg(F.max('g_id').alias('g_id'))
        giddf=giddf.join(jfid,on='g_id',how='left')
        emondf=emondf.join(giddf,on='id',how='inner')
        
        df=emondf
        for i in (range(2,256)):
            df=df.withColumn(f"_{i}",F.col(f"_{i}").astype(LongType()))
        df=df.withColumn("timestamp",F.unix_timestamp(F.concat_ws(' ','r_1','r_2'),'MM/dd/yyyy HH:mm:ss')*F.lit(1000)+(F.split(F.col('r_2'),'\.')[1]).astype(IntegerType()))
        df=df.drop("r_1")
        df=df.drop("r_2")
        cores=list(range(0,self.totalcores))
        df=df.withColumn('sum',
                         F.when(F.col("_1").startswith("UNC_IIO"),self.pcie_sum(cores))
                         .otherwise(self.cores_sum(cores)))
        if self.begin_clk>0 and self.end_clk>0:
            df=df.withColumn('valid',((F.col("timestamp")>F.lit(self.begin_clk)) & (F.col("timestamp")<F.lit(self.end_clk))))
        else:
            df=df.withColumn('valid',F.lit(True))
        
        df.repartition(3).write.mode("overwrite").parquet(paths[0]+"/emon.parquet")
        self.df=df
        df.cache()
        
    def get_effective_metric(self):
        if self.df==None:
            self.load_emon()

        emondf=self.df
        gid=emondf.agg(F.min('g_id')).collect()[0]['min(g_id)']
        emondf=emondf.where(F.col("g_id")==gid)
        emondf=emondf.cache()

        effective_metric=[]

        progress = IntProgress(layout=Layout(width='80%', height='40px'))
        progress.max = len(self.emon_metrics)
        progress.description = 'Calculate Effective Metrics'
        display(progress)
        progress.value=0

        for k,m in self.emon_metrics.items():
            join_df=None
            progress.value=progress.value+1
            for alias,event in m['events'].items():
                if join_df is None:
                    join_df=emondf.where("_1='{:s}'".format(event)).select('r_id','g_id')
                else:
                    tdf=emondf.where("_1='{:s}'".format(event)).select('r_id','g_id')
                    join_dft=join_df.join(tdf.drop('g_id'),on='r_id',how='inner')
                    if join_dft.count()==0:
                        join_df=join_df.join(tdf.drop('r_id'),on='g_id',how='inner')
                    else:
                        join_df=join_dft
            if join_df.count()>0:
                effective_metric.append(k)
        progress.value=progress.value+1
        self.effective_metric=effective_metric
        emondf.unpersist()
    
    def gen_metric(self,emondf, m):
        join_df=None
        for alias,event in m['events'].items():
            if join_df is None:
                join_df=emondf.where("_1='{:s}'".format(event)).select('timestamp','_1','_2','r_id','g_id',*self.appclients,F.col('sum').alias(alias))
            else:
                tdf=emondf.where("_1='{:s}'".format(event)).select('_1','_2','r_id','g_id',*self.appclients,F.col('sum').alias(alias))
                join_dft=join_df.join(tdf.drop('g_id'),on=['r_id',*self.appclients],how='inner')
                if join_dft.count()==0:
                    join_df=join_df.join(tdf.drop('r_id'),on=['g_id',*self.appclients],how='inner')
                else:
                    join_df=join_dft
        return join_df

    
    
    def generate_trace_view_list(self,id=0, **kwargs):
        trace_events=Analysis.generate_trace_view_list(self,id, **kwargs)
        
        cores=list(range(0,self.totalcores))
        
        emondf=self.df
        if 'collected_cores' in kwargs:
            cores=kwargs.get("collected_cores",None)
            emondf=emondf.withColumn('sum',
                     F.when(F.col("_1").startswith("UNC_IIO"),self.pcie_sum(cores))
                     .otherwise(self.cores_sum(cores)))
        show_metric= kwargs.get('show_metric', None)
            
        if show_metric is None and self.effective_metric is None:
            self.get_effective_metric()

        self.effective_metric=show_metric if show_metric is not None else self.effective_metric
        
        emondf=self.df
        
        tid=0
        for k in self.effective_metric:
            m=self.emon_metrics[k]
            join_df=self.gen_metric(emondf,m)
            rstdf=join_df.select(
                            F.lit(tid).alias('tid'),
                            F.lit(id).alias('pid'),
                            F.lit('C').alias('ph'),
                            F.lit(k).alias('name'),
                            (F.col('timestamp')-F.lit(self.starttime)).alias("ts"),
                            F.struct(*[m['fmt'](F.expr(formula)).alias(col_name) for col_name,formula in m['formula'].items() ]).alias('args')
            ).where(F.col("ts").isNotNull()).orderBy('ts')
            trace_events.extend(rstdf.toJSON().collect())
            print(trace_events[-1],"pid is ", id)
            trace_events.append(json.dumps({"name": "thread_sort_index","ph": "M","pid":id,"tid":tid,"args":{"sort_index ":tid}}))
            tid=tid+1
            
#        if self.begin_clk>0:
#            trace_events.append(json.dumps({"name":"begin",
#                         "ph":"i",
#                         "ts":self.begin_clk-self.starttime,
#                         "pid":id,
#                        "tid":tid,
#                         "s":"p"
#                        }))
 #       if self.end_clk>0:
 #           trace_events.append(json.dumps({"name":"end",
 #                        "ph":"i",
 #                        "ts":self.end_clk-self.starttime,
 #                        "pid":id,
 #                        "tid":tid,
 #                        "s":"p"
 #                       }))            

        return trace_events
    
    def show_emon_metric(self,metric,sub_metric,core,draw=True,metric_define=None, **kwargs):
        if self.df==None:
            self.load_data()
        emondf=self.df
        
        showalltime=kwargs.get("showalltime",True)
        
        if not showalltime:
            emondf=emondf.filter(F.col("valid")==F.lit(True))
        
        if metric is None or metric=='':
            for k in self.effective_metric:
                m=self.emon_metrics[k]
                if sub_metric in m['formula']:
                    break
            else:
                print("can't find metric",sub_metric)
                return        
        else:
            k=metric
        if metric_define is None:
            m= self.emon_metrics[k]
        else:
            m= metric_define[k]

        if type(core)==int:
            core=[core,]
        emondf=emondf.withColumn('sum',
                 F.when(F.col("_1").startswith("UNC_IIO"),self.pcie_sum(core))
                 .otherwise(self.count_sum(core)))
            
        join_df=self.gen_metric(emondf,m)
        
        rstdf=join_df.select(
                    F.col('timestamp').alias('ts'),
                    m['fmt'](F.expr(m['formula'][sub_metric])).alias(sub_metric),
                    'r_id'
        ).where(F.col("timestamp").isNotNull()).orderBy('timestamp')
        
        metric_sum=rstdf.select(sub_metric).summary().toPandas()
        display(metric_sum)
        
        if draw:
            pddf=rstdf.toPandas()
            pddf['ts']=(pddf['ts']-pddf.loc[0,'ts'])/1000
            fig, axs = plt.subplots(nrows=1, ncols=2, sharey=True,figsize=(30,8),gridspec_kw = {'width_ratios':[1, 5]})
            plt.subplots_adjust(wspace=0.01)
            sns.violinplot(y=sub_metric, data=pddf, ax=axs[0],palette=['g'])
            axs[0].yaxis.grid(True, which='major')
            ax=axs[1]
            ax.stackplot(pddf['ts'], pddf[sub_metric],colors=['bisque'])
            #ymin, ymax = ax.get_ylim()
            ax2 = ax.twinx()
            ax2.set_ylim(ax.get_ylim())
            ax2.axhline(y=float(metric_sum.loc[4,sub_metric]), linewidth=2, color='r')
            ax2.axhline(y=float(metric_sum.loc[5,sub_metric]), linewidth=2, color='r')
            ax2.axhline(y=float(metric_sum.loc[6,sub_metric]), linewidth=2, color='r')
            ax2.axhline(y=float(metric_sum.loc[7,sub_metric]), linewidth=2, color='r')
            ax.set_xlabel('time (s)')
            ax.yaxis.grid(True, which='major')
            plt.show()
            
            hist_elapsedtime=rstdf.select('`{:s}`'.format(sub_metric)).rdd.flatMap(lambda x: x).histogram(15)
            fig, axs = plt.subplots(figsize=(30, 5))
            ax=axs
            binSides, binCounts = hist_elapsedtime
            binSides=[builtins.round(l,2) for l in binSides]

            N = len(binCounts)
            ind = numpy.arange(N)
            width = 0.5

            rects1 = ax.bar(ind+0.5, binCounts, width, color='b')

            ax.set_ylabel('Frequencies')
            ax.set_title(sub_metric)
            ax.set_xticks(numpy.arange(N+1))
            ax.set_xticklabels(binSides)
        return rstdf
        

    def gen_reduce_metric(self,metric,core,sub_metric,agg_func):
        if self.df==None:
            self.load_data()
        emondf=self.df
        
        emondf=emondf.where(F.col("valid")==F.lit(True))
        
        k=metric
        m= self.emon_metrics[k]

        if type(core)==int:
            core=[core,]
        
        if len(core)<self.totalcores:
            emondf=emondf.withColumn('sum',
                     F.when(F.col("_1").startswith("UNC_IIO"),self.pcie_sum(core))
                     .otherwise(self.count_sum(core)))

        join_df=self.gen_metric(emondf,m)
        
        rstdf=join_df.select(
                *self.appclients,
                m['fmt'](F.expr(m['formula'][sub_metric])).alias(sub_metric)
        ).where(F.col("timestamp").isNotNull())
        return rstdf
    
    def get_reduce_metric(self,metric,core,sub_metric,agg_func):
        rstdf=self.gen_reduce_metric(metric,core,sub_metric,agg_func)
        return rstdf.agg(*[l("`{:s}`".format(sub_metric)).alias(get_alias_name(sub_metric,l)) for l in agg_func]).toPandas()
   
    def get_reduce_metrics(self,core=None,agg_func=[F.max,F.mean,F.min,F.sum]):
        coldf=None
        if self.effective_metric is None:
            self.get_effective_metric()

        if core is None:
            core=list(range(0,self.totalcores))
        progress = IntProgress(layout=Layout(width='80%', height='40px'))
        progress.max = len(self.effective_metric)
        progress.description = 'Calculate Effective Metrics'
        display(progress)
        progress.value=0
        
        columns=[f.__name__ for f in agg_func]
            
        for k in self.effective_metric:
            progress.value=progress.value+1
            m=self.emon_metrics[k]
            for fk,fm in m['formula'].items():
                df=self.get_reduce_metric(k,core,fk,agg_func)
                df.columns=columns
                df.index=[fk]
                if coldf is None:
                    coldf=df
                else:
                    coldf=coldf.append(df)
        progress.value=progress.value+1
        return coldf
    

class Emon_Analysis_All(Emon_Analysis):
    def __init__(self,emon_files):
        Emon_Analysis.__init__(self,emon_files[0])
        self.emon_files=emon_files
        self.appclients=['appid','client']
        
    def load_data(self):
        spark.clearCache()
        emondf=spark.read.format("parquet").load(self.emon_files)
        emondf=emondf.withColumn("file",F.input_file_name())
        filepath=emondf.select(F.col("file")).limit(1).collect()[0]['file']
        length=len(filepath.split("/"))
        emondf=emondf.withColumn("appid",F.split("file","/")[length-4])
        emondf=emondf.withColumn("client",F.split("file","/")[length-3]).drop("file")
        emondf=emondf.cache()
        self.df=emondf
        
    def get_reduce_metric(self,metric,core=None,sub_metric=None,agg_func=[F.max,F.mean,F.min,F.sum]):
        
        if core is None:
            core=list(range(0,self.totalcores))
        if sub_metric is None:
            m=self.emon_metrics[metric]
            sub_metric = list(m['formula'].keys())[0]
        
        rstdf=self.gen_reduce_metric(metric,core,sub_metric,agg_func)
        return rstdf.groupBy("appid").agg(*[l("`{:s}`".format(sub_metric)).alias(get_alias_name(sub_metric,l)) for l in agg_func]).toPandas()
    
    def get_reduce_metrics(self,core=None,agg_func=[F.max,F.mean,F.min,F.sum]):
        return None
    
    def generate_trace_view_list(self, id , **kwargs):
        Analysis.generate_trace_view_list(self,0)
        
        cores=list(range(0,self.totalcores))
        
        pidmap=kwargs.get("pidmap",None)
        if pidmap is None:
            print("multiple emon process needs pidmap in {'client':pid,} format")
            return []
        else:
            display(pidmap)
            
        emondf=self.df
        if 'collected_cores' in kwargs:
            cores=kwargs.get("collected_cores",None)
            emondf=emondf.withColumn('sum',
                     F.when(F.col("_1").startswith("UNC_IIO"),self.pcie_sum(cores))
                     .otherwise(self.cores_sum(cores)))
        show_metric= kwargs.get('show_metric', None)
            
        if show_metric is None and self.effective_metric is None:
            self.get_effective_metric()

        self.effective_metric=show_metric if show_metric is not None else self.effective_metric
        
        mapexpr=F.create_map([F.lit(x) for x in chain(*pidmap.items())])
        
        trace_events=[]
        for c,id in pidmap.items():
            trace_events.append(json.dumps({"name": "process_name","ph": "M","pid":id,"tid":0,"args":{"name":" "+c}}))
        tid=0
        for k in self.effective_metric:
            m=self.emon_metrics[k]
            join_df=self.gen_metric(emondf,m)
            join_df=join_df.withColumn('pid',mapexpr.getItem(F.col("client")))
            rstdf=join_df.select(
                            F.lit(tid).alias('tid'),
                            F.col('pid').alias('pid'),
                            F.lit('C').alias('ph'),
                            F.lit(k).alias('name'),
                            (F.col('timestamp')-F.lit(self.starttime)).alias("ts"),
                            F.struct(*[m['fmt'](F.expr(formula)).alias(col_name) for col_name,formula in m['formula'].items() ]).alias('args')
            ).where(F.col("ts").isNotNull()).orderBy('ts')
            trace_events.extend(rstdf.toJSON().collect())
            for id in pidmap.values():
                trace_events.append(json.dumps({"name": "thread_sort_index","ph": "M","pid":id,"tid":tid,"args":{"sort_index ":tid}}))
            tid=tid+1
            
#        infodf=emondf.where(F.col("valid")==F.lit(True)).groupBy("client").agg(F.min('timestamp'),F.max('timestamp')).cache()
#        infodf=infodf.withColumn('pid',mapexpr.getItem(F.col("client")))
#        trace_events.extend(infodf.select(
#            F.lit('begin').alias("name"),
#            F.lit("i").alias("ph"),
#            (F.col("min(timestamp)")-F.lit(self.starttime)).alias('ts'),
#             F.col("pid"),
#             F.lit(0).alias("tid"),
#             F.lit("p").alias("s")
#        ).toJSON().collect())
#        trace_events.extend(infodf.select(
#            F.lit('begin').alias("name"),
#            F.lit("i").alias("ph"),
#            (F.col("max(timestamp)")-F.lit(self.starttime)).alias('ts'),
#             F.col("pid"),
#             F.lit(0).alias("tid"),
#             F.lit("p").alias("s")
#        ).toJSON().collect())
#        infodf.unpersist()
        return trace_events    
    
    
def get_emon_parquets(apps,basedir):

    emondfunion=None
    emondfs=[]
    for appid in apps:
        slaves=fs.list_status("/"+basedir+"/"+appid)
        slaves=[f['pathSuffix'] for f in slaves if f['type']=='DIRECTORY']
        for client in slaves:
            if not fs.exists(f"/{basedir}/{appid}/{client}/emon.parquet"):
                print(f"/{basedir}/{appid}/{client}/emon.parquet is not found, trying to load data ...")
                emonals=Emon_Analysis(f"/{basedir}/{appid}/{client}/emon.rst")
                emonals.load_data()
            emondfs.append(f"/{basedir}/{appid}/{client}/emon.parquet")
    return emondfs

NameError: name 'Analysis' is not defined

In [ ]:
def background_gradient(s, m, M, cmap='PuBu', low=0, high=0):
    from matplotlib import colors
    rng = M - m
    norm = colors.Normalize(m - (rng * low),
                            M + (rng * high))
    normed = norm(s.values)
    c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    return ['background-color: {:s}'.format(color) for color in c]

# app log analysis

In [ ]:
from pyspark.sql.functions import udf
@udf("long")
def isfinish_udf(s):
    def isfinish(root):
        if "isFinalPlan=false" in root['simpleString']:
            return 0
        if root['children'] is not None:
            for c in root["children"]:
                if isfinish(c)==0:
                    return 0
        return 1
    if len(s)>0:
        return isfinish(s[0])
    else:
        return 0
    
@pandas_udf("taskid long, start long, dur long, name string", PandasUDFType.GROUPED_MAP)
def time_breakdown(pdf):
    ltime=pdf['Launch Time'][0]+2
    pdf['start']=0
    pdf['dur']=0
    outpdf=[]
    ratio=(pdf["Finish Time"][0]-pdf["Launch Time"][0])/pdf["Update"].sum()
    ratio=1 if ratio>1 else ratio
    for idx,l in pdf.iterrows():
        if(l["Update"]*ratio>1):
            outpdf.append([l["Task ID"],ltime,int(l["Update"]*ratio),l["mname"]])
            ltime=ltime+int(l["Update"]*ratio)
    if len(outpdf)>0:
        return pandas.DataFrame(outpdf)
    else:
        return pandas.DataFrame({'taskid': pandas.Series([], dtype='long'),
                   'start': pandas.Series([], dtype='long'),
                   'dur': pandas.Series([], dtype='long'),
                   'name': pandas.Series([], dtype='str'),
                                })
    
class App_Log_Analysis(Analysis):
    def __init__(self, file, jobids):
        Analysis.__init__(self,file)
        self.jobids=[] if jobids is None else [str(l) for l in jobids]
        self.df=None
        self.pids=[]
        
    def load_data(self):
        print("load data ", self.file)
        jobids=self.jobids
        df=spark.read.json(self.file)
        if 'App ID' in df.columns:
            self.appid=df.where("`App ID` is not null").collect()[0]["App ID"]
        else:
            self.appid="Application-00000000"
        
        if df.where("Event='org.apache.spark.sql.execution.ui.SparkListenerDriverAccumUpdates'").count()>0:
            self.dfacc=df.where("Event='org.apache.spark.sql.execution.ui.SparkListenerDriverAccumUpdates'").select(F.col("executionId").alias("queryid"),F.explode("accumUpdates"))
        else:
            self.dfacc = None
            
        if "sparkPlanInfo" in df.columns:
            self.queryplans=df.where("(Event='org.apache.spark.sql.execution.ui.SparkListenerSQLExecutionStart' or Event='org.apache.spark.sql.execution.ui.SparkListenerSQLAdaptiveExecutionUpdate') \
                                  and (sparkPlanInfo.nodeName!='AdaptiveSparkPlan' or sparkPlanInfo.simpleString='AdaptiveSparkPlan isFinalPlan=true') ").select(F.col("executionId").alias("queryid"),"sparkPlanInfo.*")
        else:
            self.queryplans=None
        
        seen = set()
        
        if self.queryplans is not None:
            self.queryplans=self.queryplans.where(isfinish_udf("children")==1)
        
            self.allmetrics=[]
            if self.queryplans.count() > 0:
                metrics=self.queryplans.collect()
                def get_metric(root):
                    for l in root["metrics"]:
                        if l['accumulatorId'] not in seen:
                            seen.add(l['accumulatorId'])
                            self.allmetrics.append([l['accumulatorId'],l["metricType"],l['name'],root["nodeName"]])
                    if root['children'] is not None:
                        for c in root["children"]:
                            get_metric(c)
                for c in metrics:
                    get_metric(c)
        
            amsdf=spark.createDataFrame(self.allmetrics)
            amsdf=amsdf.withColumnRenamed("_1","ID").withColumnRenamed("_2","type").withColumnRenamed("_3","Name").withColumnRenamed("_4","nodeName")
        
        
        if self.dfacc is not None:
            self.dfacc=self.dfacc.select("queryid",(F.col("col")[0]).alias("ID"),(F.col("col")[1]).alias("Update")).join(amsdf,on=["ID"])
        
        if self.queryplans is not None:
            self.metricscollect=[l for l in self.allmetrics if l[1] in ['nsTiming','timing'] and (l[2].startswith("totaltime_") or l[2].startswith("scan time") or l[2].startswith("shuffle write time")) and l[2] not in("totaltime_collectbatch") ]
        
        #config=df.where("event='SparkListenerJobStart' and Properties.`spark.executor.cores` is not null").select("Properties.*").limit(1).collect()
        config=df.select("`Spark Properties`.*").where("`spark.app.id` is not null").limit(1).collect()
    
        configdic=config[0].asDict()
        self.parallelism=int(configdic['spark.sql.shuffle.partitions']) if 'spark.sql.shuffle.partitions' in configdic else 1
        self.executor_cores=int(configdic['spark.executor.cores']) if 'spark.executor.cores' in configdic else 1
        self.executor_instances=int(configdic['spark.executor.instances']) if 'spark.executor.instances' in configdic else 1
        self.taskcpus= int(configdic['spark.task.cpus'])if 'spark.task.cpus' in configdic else 1
        self.batchsize= int(configdic['spark.sql.execution.arrow.maxRecordsPerBatch'])if 'spark.sql.execution.arrow.maxRecordsPerBatch' in configdic else 1
        
        self.realexecutors = df.select("Executor ID").distinct().count()

        if "spark.sql.execution.id" in df.where("Event='SparkListenerJobStart'").select("Properties.*").columns:
            df_jobstart=df.where("Event='SparkListenerJobStart'").select("Job ID","Submission Time",F.col("Properties.`spark.sql.execution.id`").alias("queryid"),"Stage IDs")
        else:
            df_jobstart=df.where("Event='SparkListenerJobStart'").select("Job ID","Submission Time",F.lit(0).alias("queryid"),"Stage IDs")
        df_jobend=df.where("Event='SparkListenerJobEnd'").select("`Job ID`","Completion Time")
        df_job=df_jobstart.join(df_jobend,"Job ID")
        df_job=df_job.withColumnRenamed("Submission Time","job_start_time")
        df_job=df_job.withColumnRenamed("Completion Time","job_stop_time")
        self.df_job=df_job
        
        jobstage=df_job.select("*",F.explode("Stage IDs").alias("Stage ID"))
        task=df.where("(Event='SparkListenerTaskEnd' or Event='SparkListenerTaskStart') ").select("Event","Stage ID","task info.*","task metrics.*")
        
        self.failed_stages = [str(l['Stage ID']) for l in task.where("Failed='true'").select("Stage ID").distinct().collect()]
        
        
        taskjob=task.\
            select("Host","`Event`","`Launch Time`","`Executor ID`","`Task ID`","`Finish Time`",
                    "`Stage ID`","`Input Metrics`.`Bytes Read`","`Disk Bytes Spilled`","`Memory Bytes Spilled`","`Shuffle Read Metrics`.`Local Bytes Read`","`Shuffle Read Metrics`.`Remote Bytes Read`",
                   "`Shuffle Write Metrics`.`Shuffle Bytes Written`","`Executor Deserialize Time`","`Shuffle Read Metrics`.`Fetch Wait Time`","`Executor Run Time`","`Shuffle Write Metrics`.`Shuffle Write Time`",
                   "`Result Serialization Time`","`Getting Result Time`","`JVM GC Time`","`Executor CPU Time`","Accumulables",
                    F.when(task['Finish Time']==0,task['Launch Time']).otherwise(task['Finish Time']).alias('eventtime')
        ).join(jobstage,"Stage ID")
        
        self.df=taskjob
        
        if len(jobids)>0:
            self.df=self.df.where('`Job ID` in ({:s})'.format(','.join(jobids)))
        
        queryids=self.df.select(F.col("queryid").astype(IntegerType())).distinct().where("queryid is not null").orderBy("queryid").toPandas()
        
        self.query_num=len(queryids)
        if self.query_num>0:
            queryidx=queryids.reset_index()
            queryidx['index']=queryidx['index']+1
            #tpcds query
            if self.query_num==103:
                queryidx['index']=queryidx['index'].map(tpcds_query_map)
            qidx=spark.createDataFrame(queryidx)
            qidx=qidx.withColumnRenamed("index","real_queryid")
            self.df=self.df.join(qidx,on="queryid",how="left")
            if self.dfacc is not None:
                self.dfacc=self.dfacc.join(qidx,on="queryid",how='left')

            if self.queryplans:
                self.queryplans=self.queryplans.join(qidx,"queryid",how="right")
        
        self.df=self.df.fillna(0)
        self.df.cache()
        
        
        
        ##############################
        
        dfx=self.df.where("Event='SparkListenerTaskEnd'").select("Stage ID","Launch Time","Finish Time","Task ID")
        dfxpds=dfx.toPandas()
        dfxpds.columns=[l.replace(" ","_") for l in dfxpds.columns]
        dfxpds_ods=sqldf('''select * from dfxpds order by finish_time desc''')
        criticaltasks=[]
        idx=0
        prefinish=0
        launchtime=dfxpds_ods["Launch_Time"][0]
        criticaltasks.append([dfxpds_ods["Task_ID"][0],launchtime,dfxpds_ods["Finish_Time"][0]])
        total_row=len(dfxpds_ods)

        while True:
            while idx<total_row:
                if dfxpds_ods["Finish_Time"][idx]-2<launchtime:
                    break
                idx=idx+1
            else:
                break
            cur_finish=dfxpds_ods["Finish_Time"][idx]
            cur_finish=launchtime-1 if cur_finish>=launchtime else cur_finish
            launchtime=dfxpds_ods["Launch_Time"][idx]
            criticaltasks.append([dfxpds_ods["Task_ID"][idx],launchtime,cur_finish])
        self.criticaltasks=criticaltasks

    def get_basic_state(appals):
        if appals.df is None:
            appals.load_data()
        display(HTML("<a href=http://sr525:18080/history/"+appals.appid+">http://sr525:18080/history/"+appals.appid+"</a>"))
        
        errorcolor="#000000" if appals.executor_instances == appals.realexecutors else "#c0392b"
        
        qtime=appals.get_query_time(plot=False)
        sums=qtime.sum()
        if len(appals.failed_stages)>0:
            failure="<br>".join(["query: " + str(l["real_queryid"])+"|stage: " + str(l["Stage ID"]) for l in appals.df.where("`Stage ID` in ("+",".join(appals.failed_stages)+")").select("real_queryid","Stage ID").distinct().collect()])
        else:
            failure=""
        display(HTML(f'''
        <table border="1" cellpadding="1" cellspacing="1" style="width:500px">
            <tbody>
                <tr>
                    <td style="width:135px">appid</td>
                    <td style="width:351px"><span style="color:#000000"><strong>{appals.appid}</strong></span></td>
                </tr>
                <tr>
                    <td style="width:135px">executor.instances</td>
                    <td style="width:351px"><span style="color:#000000"><strong>{appals.executor_instances}</strong></span></td>
                </tr>
                <tr>
                    <td style="width:135px">executor.cores</td>
                    <td style="width:351px"><span style="color:#000000"><strong>{appals.executor_cores}</strong></span></td>
                </tr>
                <tr>
                    <td style="width:135px">shuffle.partitions</td>
                    <td style="width:351px"><span style="color:#000000"><strong>{appals.parallelism}</strong></span></td>
                </tr>
                <tr>
                    <td style="width:135px">shuffle.partitions</td>
                    <td style="width:351px"><span style="color:#000000"><strong>{appals.batchsize}</strong></span></td>
                </tr>                
                <tr>
                    <td style="width:135px">real executors</td>
                    <td style="width:351px"><span style="color:{errorcolor}"><strong>{appals.realexecutors}</strong></span></td>
                </tr>
                <tr>
                    <td style="width:135px">Failed Tasks</td>
                    <td style="width:351px"><span style="color:{errorcolor}"><strong>{failure}</strong></span></td>
                </tr>
                <tr>
                    <td style="width:135px">runtime</td>
                    <td style="width:351px"><strong>{round(sums['runtime'],2)}</strong></td>
                </tr>
                <tr>
                    <td style="width:135px">disk spilled</td>
                    <td style="width:351px"><strong>{round(sums['disk spilled'],2)}</strong></td>
                </tr>
                <tr>
                    <td style="width:135px">memspilled</td>
                    <td style="width:351px"><strong>{round(sums['memspilled'],2)}</strong></td>
                </tr>
                <tr>
                    <td style="width:135px">local_read</td>
                    <td style="width:351px"><strong>{round(sums['local_read'],2)}</strong></td>
                </tr>
                <tr>
                    <td style="width:135px">remote_read</td>
                    <td style="width:351px"><strong>{round(sums['remote_read'],2)}</strong></td>
                </tr>
                <tr>
                    <td style="width:135px">shuffle_write</td>
                    <td style="width:351px"><strong>{round(sums['shuffle_write'],2)}</strong></td>
                </tr>
                <tr>
                    <td style="width:135px">task run time</td>
                    <td style="width:351px"><strong>{round(sums['run_time'],2)}</strong></td>
                </tr>
                <tr>
                    <td style="width:135px">ser_time</td>
                    <td style="width:351px"><strong>{round(sums['ser_time'],2)}</strong></td>
                </tr>
                <tr>
                    <td style="width:135px">f_wait_time</td>
                    <td style="width:351px"><strong>{round(sums['f_wait_time'],2)}</strong></td>
                </tr>
                <tr>
                    <td style="width:135px">gc_time</td>
                    <td style="width:351px"><strong>{round(sums['gc_time'],2)}</strong></td>
                </tr>
                <tr>
                    <td style="width:135px">input read</td>
                    <td style="width:351px"><strong>{round(sums['input read'],2)}</strong></td>
                </tr>
                <tr>
                    <td style="width:135px">acc_task_time</td>
                    <td style="width:351px"><strong>{round(sums['acc_task_time'],2)}</strong></td>
                </tr>
            </tbody>
        </table>

        '''))        
        
    def generate_trace_view_list_exec(self,id=0,**kwargs):
        Analysis.generate_trace_view_list(self,**kwargs)
        showcpu=kwargs.get('showcpu',False)
        shownodes=kwargs.get("shownodes",None)
        
        showdf=self.df.where(F.col("Host").isin(shownodes)) if shownodes else self.df
        
        events=showdf.toPandas()
        coretrack={}
        trace_events=[]
        starttime=self.starttime
        taskend=[]
        trace={"traceEvents":[]}
        exec_hosts={}
        hostsdf=showdf.select("Host").distinct().orderBy("Host")
        hostid=100000
        ended_event=[]
        
        for i,l in hostsdf.toPandas().iterrows():
            exec_hosts[l['Host']]=hostid
            hostid=hostid+100000

        for idx,l in events.iterrows():
            if l['Event']=='SparkListenerTaskStart':
                hostid=exec_hosts[l['Host']]

                tsk=l['Task ID']
                pid=int(l['Executor ID'])*100+hostid
                self.pids.append(pid)
                stime=l['Launch Time']
                #the task's starttime and finishtime is the same, ignore it.
                if tsk in ended_event:
                    continue
                if not pid in coretrack:
                    tids={}
                    trace_events.append({
                       "name": "process_name",
                       "ph": "M",
                       "pid":pid,
                       "tid":0,
                       "args":{"name":"{:s}.{:s}".format(l['Host'],l['Executor ID'])}
                      })

                else:
                    tids=coretrack[pid]
                for t in tids.keys():
                    if tids[t][0]==-1:
                        tids[t]=[tsk,stime]
                        break
                else:
                    t=len(tids)
                    tids[t]=[tsk,stime]
                #print("task {:d} tid is {:s}.{:d}".format(tsk,pid,t))
                coretrack[pid]=tids

            if l['Event']=='SparkListenerTaskEnd':
                sevt={}
                eevt={}
                hostid=exec_hosts[l['Host']]
                pid=int(l['Executor ID'])*100+hostid
                tsk=l['Task ID']
                fintime=l['Finish Time']

                tids=coretrack[pid]
                for t in tids.keys():
                    if tids[t][0]==tsk:
                        tids[t]=[-1,-1]
                        break
                else:
                    ended_event.append(tsk)
                    continue
                for ps in reversed([key for key in tids.keys()]) :
                    if tids[ps][1]-fintime<0 and tids[ps][1]-fintime>=-2:
                        fintime=tids[ps][1]
                        tids[t]=tids[ps]
                        tids[ps]=[-1,-1]
                        break
                if starttime==0:
                    starttime=l['Launch Time']
                    print(f'applog start time: {starttime}')

                sstime=l['Launch Time']-starttime

                trace_events.append({
                       'tid':pid+int(t),
                       'ts':sstime,
                       'dur':fintime-l['Launch Time'],
                       'pid':pid,
                       "ph":'X',
                       'name':"stg{:d}".format(l['Stage ID']),
                       'args':{"job id": l['job id'],
                               "stage id": l['Stage ID'],
                               "tskid":tsk,
                               "input":builtins.round(l["Bytes Read"]/1024/1024,2),
                               "spill":builtins.round(l["Memory Bytes Spilled"]/1024/1024,2),
                               "Shuffle Read Metrics": "",
                               "|---Local Read": builtins.round(l["Local Bytes Read"]/1024/1024,2),
                               "|---Remote Read":builtins.round(l["Remote Bytes Read"]/1024/1024,2),
                               "Shuffle Write Metrics": "",
                               "|---Write":builtins.round(l['Shuffle Bytes Written']/1024/1024,2)
                               }
                      })

                des_time=l['Executor Deserialize Time']
                read_time=l['Fetch Wait Time']
                exec_time=l['Executor Run Time']
                write_time=math.floor(l['Shuffle Write Time']/1000000)
                ser_time=l['Result Serialization Time']
                getrst_time=l['Getting Result Time']
                durtime=fintime-sstime-starttime;

                times=[0,des_time,read_time,exec_time,write_time,ser_time,getrst_time]
                time_names=['sched delay','deserialize time','read time','executor time','write time','serialize time','result time']
                evttime=reduce((lambda x, y: x + y),times)
                if evttime>durtime:
                    times=[math.floor(l*1.0*durtime/evttime) for l in times]
                else:
                    times[0]=durtime-evttime

                esstime=sstime
                for idx in range(0,len(times)):
                    if times[idx]>0:
                        trace_events.append({
                             'tid':pid+int(t),
                             'ts':esstime,
                             'dur':times[idx],                
                             'pid':pid,
                             'ph':'X',
                             'name':time_names[idx]})
                        if idx==3:
                            trace_events.append({
                                 'tid':pid+int(t),
                                 'ts':esstime,
                                 'dur':l['JVM GC Time'],
                                 'pid':pid,
                                 'ph':'X',
                                 'name':'GC Time'})
                            if showcpu:
                                trace_events.append({
                                     'tid':pid+int(t),
                                     'ts':esstime,
                                     'pid':pid,
                                     'ph':'C',
                                     'name':'cpu% {:d}'.format(pid+int(t)),
                                     'args':{'value':l['Executor CPU Time']/1000000.0/times[idx]}})
                                trace_events.append({
                                     'tid':pid+int(t),
                                     'ts':esstime+times[idx],
                                     'pid':pid,
                                     'ph':'C',
                                     'name':'cpu% {:d}'.format(pid+int(t)),
                                     'args':{'value':0}})
                        esstime=esstime+times[idx]
        self.starttime=starttime
        return [json.dumps(l) for l in trace_events]

    def generate_trace_view_list(self,id=0,**kwargs):
        Analysis.generate_trace_view_list(self,**kwargs)
        showcpu=kwargs.get('showcpu',False)
        shownodes=kwargs.get("shownodes",None)
        
        showdf=self.df.where(F.col("Host").isin(shownodes)) if shownodes else self.df
        
        showdf=showdf.orderBy(["eventtime", "Finish Time"], ascending=[1, 0])
        
        events=showdf.drop("Accumulables").toPandas()
        coretrack={}
        trace_events=[]
        starttime=self.starttime
        taskend=[]
        trace={"traceEvents":[]}
        exec_hosts={}
        hostsdf=showdf.select("Host").distinct().orderBy("Host")
        hostid=100000
        ended_event=[]
        
        for i,l in hostsdf.toPandas().iterrows():
            exec_hosts[l['Host']]=hostid
            hostid=hostid+100000

        tskmap={}
        for idx,l in events.iterrows():
            if l['Event']=='SparkListenerTaskStart':
                hostid=exec_hosts[l['Host']]

                tsk=l['Task ID']
                pid=int(l['Executor ID'])*100+hostid
                self.pids.append(pid)
                stime=l['Launch Time']
                #the task's starttime and finishtime is the same, ignore it.
                if tsk in ended_event:
                    continue
                if not pid in coretrack:
                    tids={}
                    trace_events.append({
                       "name": "process_name",
                       "ph": "M",
                       "pid":pid,
                       "tid":0,
                       "args":{"name":"{:s}.{:s}".format(l['Host'],l['Executor ID'])}
                      })

                else:
                    tids=coretrack[pid]
                for t in tids.keys():
                    if tids[t][0]==-1:
                        tids[t]=[tsk,stime]
                        break
                else:
                    t=len(tids)
                    tids[t]=[tsk,stime]
                #print("task {:d} tid is {:s}.{:d}".format(tsk,pid,t))
                coretrack[pid]=tids

            if l['Event']=='SparkListenerTaskEnd':
                sevt={}
                eevt={}
                hostid=exec_hosts[l['Host']]
                pid=int(l['Executor ID'])*100+hostid
                tsk=l['Task ID']
                fintime=l['Finish Time']
                
                tids=coretrack[pid]
                for t in tids.keys():
                    if tids[t][0]==tsk:
                        tids[t]=[-1,-1]
                        break
                else:
                    ended_event.append(tsk)
                    continue
                for ps in reversed([key for key in tids.keys()]) :
                    if tids[ps][1]-fintime<0 and tids[ps][1]-fintime>=-2:
                        fintime=tids[ps][1]
                        tids[t]=tids[ps]
                        tids[ps]=[-1,-1]
                        break
                if starttime==0:
                    starttime=l['Launch Time']
                    print(f'applog start time: {starttime}')

                sstime=l['Launch Time']-starttime

                trace_events.append({
                       'tid':pid+int(t),
                       'ts':sstime,
                       'dur':fintime-l['Launch Time'],
                       'pid':pid,
                       "ph":'X',
                       'name':"stg{:d}".format(l['Stage ID']),
                       'args':{"job id": l['Job ID'],
                               "stage id": l['Stage ID'],
                               "tskid":tsk,
                               "input":builtins.round(l["Bytes Read"]/1024/1024,2),
                               "spill":builtins.round(l["Memory Bytes Spilled"]/1024/1024,2),
                               "Shuffle Read Metrics": "",
                               "|---Local Read": builtins.round(l["Local Bytes Read"]/1024/1024,2),
                               "|---Remote Read":builtins.round(l["Remote Bytes Read"]/1024/1024,2),
                               "Shuffle Write Metrics": "",
                               "|---Write":builtins.round(l['Shuffle Bytes Written']/1024/1024,2)
                               }
                      })
                tskmap[tsk]={'pid':pid,'tid':pid+int(t)}

        self.starttime=starttime
        self.tskmap=tskmap
        output=[json.dumps(l) for l in trace_events]
        
        df=self.df
        
        if showcpu and len(self.metricscollect)>0:
            metricscollect=self.metricscollect
            metrics_explode=df.where("Event='SparkListenerTaskEnd'").withColumn("metrics",F.explode("Accumulables"))
            m1092=metrics_explode.select(F.col("Executor ID"),F.col("`Stage ID`"),"`Task ID`",F.col("`Finish Time`"),F.col("`Launch Time`"),(F.col("`Finish Time`")-F.col("`Launch Time`")).alias("elapsedtime"),"metrics.*").where(F.col("ID").isin([l[0] for l in metricscollect]))
            metric_name_df = spark.createDataFrame(metricscollect)
            metric_name_df=metric_name_df.withColumnRenamed("_1","ID")
            metric_name_df=metric_name_df.withColumnRenamed("_2","unit")
            metric_name_df=metric_name_df.withColumnRenamed("_3","mname")

            met_df=m1092.join(metric_name_df,on="ID")
            met_df=met_df.withColumn("Update",F.when(F.col("unit")=='nsTiming',F.col("Update")/1000000).otherwise(F.col("Update")+0))
            met_df=met_df.where("Update>1")

            metdfx=met_df.groupBy("Task ID","elapsedtime").agg(F.sum("Update").alias("totalCnt"))
            taskratio=metdfx.withColumn("ratio",F.when(F.col("totalCnt")<F.col("elapsedtime"),1).otherwise(F.col("elapsedtime")/F.col("totalCnt"))).select("Task ID","ratio")
            met_df=met_df.join(taskratio,on="Task ID")
            met_df=met_df.withColumn("Update",F.col("Update")*F.col("ratio"))

            w = (Window.partitionBy('Task ID').orderBy(F.desc("Update")).rangeBetween(Window.unboundedPreceding, 0))
            met_df=met_df.withColumn('cum_sum', F.sum('Update').over(w))

            met_df=met_df.withColumn("starttime",F.col("Launch Time")+F.col("cum_sum")-F.col("Update"))

            tskmapdf = spark.createDataFrame(pandas.DataFrame(self.tskmap).T.reset_index())
            met_df=met_df.join(tskmapdf,on=[met_df["Task ID"]==tskmapdf["index"]])

            rstdf=met_df.select(
                F.col("tid"),
                F.round(F.col("starttime")-self.starttime,0).alias("ts"),
                F.round(F.col("Update"),0).alias("dur"),
                F.col("pid"),
                F.lit("X").alias("ph"),
                F.col("mname").alias("name")
            ).where(F.col("ts").isNotNull()).orderBy('ts')

            output.extend(rstdf.toJSON().collect())

            qtime=df.where("Event='SparkListenerTaskEnd'").groupBy("real_queryid").agg(F.min("Finish Time").alias("time"))
            output.extend(qtime.select(
                F.lit("i").alias("ph"),
                (F.col("time")-starttime).alias('ts'),
                F.lit(0).alias("pid"),
                F.lit(0).alias("tid"),
                F.lit("p").alias("s")
            ).toJSON().collect())
        
        self.starttime=starttime
        
        if kwargs.get("show_criticalshow_time_metric_path",True):
            output.extend(self.generate_critical_patch_traceview(hostid-1))
        
        return output        

    def generate_critical_patch_traceview(self,pid):
        if self.df is None:
            self.load_data()
        traces=[]
        df=self.df.where("Event='SparkListenerTaskEnd' and real_queryid is not null")
        criticaltasks=self.criticaltasks
        cripds=pandas.DataFrame(criticaltasks)
        cripds.columns=['task_id',"launch","finish"]
        cridf=spark.createDataFrame(cripds)
        df_ctsk=df.join(cridf,on=[F.col("task_id")==F.col("Task ID")],how="inner")
        traces.extend(df_ctsk.select(F.lit(38).alias("tid"),
                      (F.col("launch")-F.lit(self.starttime)+1).alias("ts"),
                      (F.col("finish")-F.col("launch")-1).alias("dur"),
                      F.lit(pid).alias("pid"),
                      F.lit("X").alias("ph"),
                      F.concat(F.lit("stg"),F.col("Stage ID")).alias("name"),
                      F.struct(
                          F.col("Task ID").alias('taskid'),
                          F.col("Executor ID").astype(IntegerType()).alias('exec_id'),
                          F.col("Host").alias("host"),
                          ).alias("args")
                        ).toJSON().collect())
        traces.extend(df.groupBy("real_queryid").agg(F.max("Finish Time").alias("finish"),F.min("Launch Time").alias("launch")).select(
                        F.lit(38).alias("tid"),
                      (F.col("launch")-F.lit(self.starttime)).alias("ts"),
                      (F.col("finish")-F.col("launch")).alias("dur"),
                      F.lit(pid).alias("pid"),
                      F.lit("X").alias("ph"),
                      F.concat(F.lit("qry"),F.col("real_queryid")).alias("name")).toJSON().collect())


        metricscollect=self.metricscollect

        metrics_explode=df_ctsk.where("Event='SparkListenerTaskEnd'").withColumn("metrics",F.explode("Accumulables"))
        m1092=metrics_explode.select(F.col("Executor ID"),F.col("`Stage ID`"),"`Task ID`",F.col("`Finish Time`"),F.col("`Launch Time`"),(F.col("`Finish Time`")-F.col("`Launch Time`")).alias("elapsedtime"),"metrics.*").where(F.col("ID").isin([l[0] for l in metricscollect]))
        metric_name_df = spark.createDataFrame(metricscollect)
        metric_name_df=metric_name_df.withColumnRenamed("_1","ID")
        metric_name_df=metric_name_df.withColumnRenamed("_2","unit")
        metric_name_df=metric_name_df.withColumnRenamed("_3","mname")
        metric_name_df=metric_name_df.withColumnRenamed("_4","node")

        metric_name_df=metric_name_df.where("mname <> 'totaltime_collectbatch'")

        met_df=m1092.join(metric_name_df,on="ID")
        met_df=met_df.withColumn("Update",F.when(F.col("unit")=='nsTiming',F.col("Update")/1000000).otherwise(F.col("Update")+0))
        
        #pandas UDF doesn't work. hang
        #tmbk=met_df.groupBy('Task ID').apply(time_breakdown)
        
        w=Window.partitionBy('Task ID')
        met_df1=met_df.withColumn("sum_update",F.sum("Update").over(w))
        met_df2=met_df1.withColumn("ratio",(F.col("Finish Time")-F.col("Launch Time")-2)/F.col("sum_update"))
        met_df3=met_df2.withColumn("ratio",F.when(F.col("ratio")>1,1).otherwise(F.col("ratio")))
        met_df4=met_df3.withColumn("update_ratio",F.floor(F.col("ratio")*F.col("Update")))
        met_df5=met_df4.where(F.col("update_ratio")>2)
        w = (Window.partitionBy('Task ID').orderBy(F.desc("update_ratio")).rowsBetween(Window.unboundedPreceding, Window.currentRow))
        met_df6=met_df5.withColumn('ltime_dur', F.sum('update_ratio').over(w))
        met_df8=met_df6.withColumn("ltime",F.col("ltime_dur")+F.col("Launch Time")-F.col("update_ratio"))

        tmbk=met_df8.withColumn("taskid",F.col("Task ID")).withColumn("start",F.col("ltime")+F.lit(1)).withColumn("dur",F.col("update_ratio")-F.lit(1)).withColumn("name",F.col("mname"))
        
        
        traces.extend(tmbk.select(
                        F.lit(38).alias("tid"),
                      (F.col("start")-F.lit(self.starttime)).alias("ts"),
                      (F.col("dur")).alias("dur"),
                      F.lit(pid).alias("pid"),
                      F.lit("X").alias("ph"),
                      F.col("name").alias("name")).toJSON().collect())
        traces.append(json.dumps({
                       "name": "process_name",
                       "ph": "M",
                       "pid":pid,
                       "tid":0,
                       "args":{"name":"critical path"}
                      }))
        return traces    
    
    def show_Stage_histogram(apps,stageid,bincount):
        if apps.df is None:
            apps.load_data()
        
        inputsize = apps.df.where("`Stage ID`={:d}".format(stageid)).select("Stage ID","Executor ID", "Task ID", F.explode("Accumulables")) \
                      .select("Stage ID","Executor ID", "Task ID","col.*") \
                      .where("Name='input size in bytes' or Name='size of files read'") \
                      .groupBy("Task ID") \
                      .agg((F.sum("Update")).alias("input read"))


        stage37=apps.df.where("`Stage ID`={:d} and event='SparkListenerTaskEnd'".format(stageid) )\
                        .join(inputsize,on=["Task ID"],how="left")\
                        .fillna(0) \
                        .select(F.col('Host'), 
                                F.round((F.col('Finish Time')/1000-F.col('Launch Time')/1000),2).alias('elapsedtime'),
                                F.round((F.col('`input read`')+F.col('`Bytes Read`')+F.col('`Local Bytes Read`')+F.col('`Remote Bytes Read`'))/1024/1024,2).alias('input'))
        stage37=stage37.cache()
        hist_elapsedtime=stage37.select('elapsedtime').rdd.flatMap(lambda x: x).histogram(15)
        hist_input=stage37.select('input').rdd.flatMap(lambda x: x).histogram(15)
        fig, axs = plt.subplots(figsize=(30, 5),nrows=1, ncols=2)
        ax=axs[0]
        binSides, binCounts = hist_elapsedtime
        binSides=[builtins.round(l,2) for l in binSides]

        N = len(binCounts)
        ind = numpy.arange(N)
        width = 0.5

        rects1 = ax.bar(ind+0.5, binCounts, width, color='b')

        ax.set_ylabel('Frequencies')
        ax.set_title('stage{:d} elapsed time breakdown'.format(stageid))
        ax.set_xticks(numpy.arange(N+1))
        ax.set_xticklabels(binSides)

        ax=axs[1]
        binSides, binCounts = hist_input
        binSides=[builtins.round(l,2) for l in binSides]

        N = len(binCounts)
        ind = numpy.arange(N)
        width = 0.5
        rects1 = ax.bar(ind+0.5, binCounts, width, color='b')

        ax.set_ylabel('Frequencies')
        ax.set_title('stage{:d} input data breakdown'.format(stageid))
        ax.set_xticks(numpy.arange(N+1))
        ax.set_xticklabels(binSides)

        out=stage37
        outpds=out.toPandas()

        fig, axs = plt.subplots(nrows=1, ncols=3, sharey=False,figsize=(30,8),gridspec_kw = {'width_ratios':[1, 1, 1]})
        plt.subplots_adjust(wspace=0.01)

        groups= outpds.groupby('Host')
        for name, group in groups:
            axs[0].plot(group.input, group.elapsedtime, marker='o', linestyle='', ms=5, label=name)
        axs[0].set_xlabel('input size (MB)')
        axs[0].set_ylabel('elapsed time (s)')

        axs[0].legend()

        axs[0].get_shared_y_axes().join(axs[0], axs[1])

        sns.violinplot(y='elapsedtime', x='Host', data=outpds,palette=['g'],ax=axs[1])

        sns.violinplot(y='input', x='Host', data=outpds,palette=['g'],ax=axs[2])

        #ax.xaxis.set_major_formatter(mtick.FormatStrFormatter(''))
        #ax.yaxis.set_major_formatter(mtick.FormatStrFormatter(''))

        if False:
            out=stage37
            vecAssembler = VectorAssembler(inputCols=["input",'elapsedtime'], outputCol="features").setHandleInvalid("skip")
            new_df = vecAssembler.transform(out)
            kmeans = KMeans(k=2, seed=1)  # 2 clusters here
            model = kmeans.fit(new_df.select('features'))
            transformed = model.transform(new_df)


            outpds=transformed.select('Host','elapsedtime','input','prediction').toPandas()

            fig, axs = plt.subplots(nrows=1, ncols=2, sharey=False,figsize=(30,8),gridspec_kw = {'width_ratios':[1, 1]})
            plt.subplots_adjust(wspace=0.01)

            groups= outpds.groupby('prediction')
            for name, group in groups:
                axs[0].plot(group.input, group.elapsedtime, marker='o', linestyle='', ms=5, label=name)
            axs[0].legend()

            bars=transformed.where('prediction=1').groupBy("Host").count().toPandas()

            axs[1].bar(bars['Host'], bars['count'], 0.4, color='coral')
            axs[1].set_title('cluster=1')

        plt.show()
        
    def show_Stages_hist(apps,**kwargs):
        if apps.df is None:
            apps.load_data()
        
        bincount=kwargs.get("bincount",15)
        threshold=kwargs.get("threshold",0.9)
        
        query=kwargs.get("queryid",None)
        if query and type(query)==int:
            query = [query,]
        df=apps.df.where(F.col("real_queryid").isin(query)) if query else apps.df
        
        totaltime=df.where("event='SparkListenerTaskEnd'" ).agg(F.sum(F.col('Finish Time')-F.col('Launch Time')).alias('total_time')).collect()[0]['total_time']
        stage_time=df.where("event='SparkListenerTaskEnd'" ).groupBy('`Stage ID`').agg(F.sum(F.col('Finish Time')-F.col('Launch Time')).alias('total_time')).orderBy('total_time', ascending=False).toPandas()
        stage_time['acc_total'] = stage_time['total_time'].cumsum()/totaltime
        stage_time=stage_time.reset_index()
        fig, ax = plt.subplots(figsize=(30, 5))

        rects1 = ax.plot(stage_time['index'],stage_time['acc_total'],'b.-')
        ax.set_xticks(stage_time['index'])
        ax.set_xticklabels(stage_time['Stage ID'])
        ax.set_xlabel('stage')
        ax.grid(which='major', axis='x')
        plt.show()
        shownstage=[]
        for x in stage_time.index:
            if stage_time['acc_total'][x]<=threshold:
                shownstage.append(stage_time['Stage ID'][x])
            else:
                shownstage.append(stage_time['Stage ID'][x])
                break
        for row in shownstage:
            apps.show_Stage_histogram(row,bincount) 
            
    def get_hottest_stages(apps,**kwargs):
        if apps.df is None:
            apps.load_data()
        
        bincount=kwargs.get("bincount",15)
        threshold=kwargs.get("threshold",0.9)
        plot=kwargs.get("plot",True)
        
        query=kwargs.get("queryid",None)
        if query and type(query)==int:
            query = [query,]
        df=apps.df.where(F.col("real_queryid").isin(query)) if query else apps.df.where("queryid is not NULL")

        stage_time=df.where("event='SparkListenerTaskEnd'" ).groupBy('`Stage ID`','Job ID','real_queryid').agg(
            F.sum(F.col('Finish Time')-F.col('Launch Time')).alias('total_time'),
            F.stddev(F.col('Finish Time')/1000-F.col('Launch Time')/1000).alias('stdev_time'),
            F.count("*").alias("cnt"),
            F.first('queryid').astype(IntegerType()).alias('queryid')
            )\
            .select('`Stage ID`','Job ID','real_queryid','queryid',
                    (F.col("total_time")/1000/(F.when(F.col("cnt")>F.lit(apps.executor_instances*apps.executor_cores/apps.taskcpus),F.lit(apps.executor_instances*apps.executor_cores/apps.taskcpus)).otherwise(F.col("cnt")))).alias("total_time"),
                    F.col("stdev_time")
                   ).orderBy('total_time', ascending=False).toPandas()

        totaltime=stage_time['total_time'].sum()
        stage_time['acc_total'] = stage_time['total_time'].cumsum()/totaltime
        stage_time['total'] = stage_time['total_time']/totaltime
        stage_time=stage_time.reset_index()

        shownstage=stage_time.loc[stage_time['acc_total'] <=threshold]
        shownstage['stg']=shownstage['real_queryid'].astype(str)+'_'+shownstage['Job ID'].astype(str)+'_'+shownstage['Stage ID'].astype(str)
        if plot:
            shownstage.plot.bar(x="stg",y="total",figsize=(30,8))



        norm = matplotlib.colors.Normalize(vmin=0, vmax=max(stage_time.queryid))
        cmap = matplotlib.cm.get_cmap('brg')
        def setbkcolor(x):
            rgba=cmap(norm(x['queryid']))
            return ['background-color:rgba({:d},{:d},{:d},1); color:white'.format(int(rgba[0]*255),int(rgba[1]*255),int(rgba[2]*255))]*9

        if plot:
            display(stage_time.style.apply(setbkcolor,axis=1).format({"total_time":lambda x: '{:,.2f}'.format(x),"acc_total":lambda x: '{:,.2%}'.format(x),"total":lambda x: '{:,.2%}'.format(x)}))
        
        return stage_time

    def scatter_elapsetime_input(apps,stageid):
        if apps.df is None:
            apps.load_data()
        stage37=apps.df.where("`Stage ID`={:d} and event='SparkListenerTaskEnd'".format(stageid) ).select(F.round((F.col('Finish Time')/1000-F.col('Launch Time')/1000),2).alias('elapsedtime'),F.round((F.col('`Bytes Read`')+F.col('`Local Bytes Read`')+F.col('`Remote Bytes Read`'))/1024/1024,2).alias('input')).toPandas()
        stage37.plot.scatter('input','elapsedtime',figsize=(30, 5))

    def get_critical_path_stages(self):     
        df=self.df.where("Event='SparkListenerTaskEnd'")
        criticaltasks=self.criticaltasks
        cripds=pandas.DataFrame(criticaltasks)
        cripds.columns=['task_id',"launch","finish"]
        cridf=spark.createDataFrame(cripds)
        df_ctsk=df.join(cridf,on=[F.col("task_id")==F.col("Task ID")],how="inner")
        df_ctsk=df_ctsk.withColumn("elapsed",(F.col("Finish Time")-F.col("Launch Time"))/1000)
        return df_ctsk.where("elapsed>10").orderBy(F.desc("elapsed")).select("real_queryid",F.round("elapsed",2).alias("elapsed"),"Host","executor ID","Stage ID","Task ID",F.round(F.col("Bytes Read")/1000000,0).alias("file read"),F.round((F.col("Local Bytes Read")+F.col("Remote Bytes Read"))/1000000,0).alias("shuffle read")).toPandas()
        
    def show_time_metric(self,**kwargs):
        if self.df is None:
            self.load_data()
        shownodes=kwargs.get("shownodes",None)
        query=kwargs.get("queryid",None)
        plot=kwargs.get("plot",True)
        taskids=kwargs.get("taskids",None)
        
        if query and type(query)==int:
            query = [query,]
        
        showexecutor=kwargs.get("showexecutor",True) if not taskids else False
        queryid = query[0] if query else 0
        
        df=self.df.where(F.col("Host").isin(shownodes)) if shownodes else self.df
        df=df.where(F.col("real_queryid").isin(query)) if query else df.where("queryid is not NULL")

        df=df.where(F.col("Task ID").isin(taskids)) if taskids else df

        exec_cores=1 if taskids else self.executor_cores
        execs=1 if taskids else self.executor_instances

        metricscollect=self.metricscollect

        metrics_explode=df.where("Event='SparkListenerTaskEnd'").withColumn("metrics",F.explode("Accumulables"))
        m1092=metrics_explode.select(F.col("Executor ID"),F.col("`Stage ID`"),"`Task ID`",F.col("`Finish Time`"),F.col("`Launch Time`"),(F.col("`Finish Time`")-F.col("`Launch Time`")).alias("elapsedtime"),"metrics.*").where(F.col("ID").isin([l[0] for l in metricscollect]))
        metric_name_df = spark.createDataFrame(metricscollect)
        metric_name_df=metric_name_df.withColumnRenamed("_1","ID")
        metric_name_df=metric_name_df.withColumnRenamed("_2","unit")
        metric_name_df=metric_name_df.withColumnRenamed("_3","mname")
        metric_name_df=metric_name_df.withColumnRenamed("_4","node")

        runtime=metrics_explode.agg(F.round(F.max("Finish Time")/1000-F.min("Launch Time")/1000,2).alias("runtime")).collect()[0]["runtime"]

        met_df=m1092.join(metric_name_df,on="ID")
        met_df=met_df.withColumn("Update",F.when(F.col("unit")=='nsTiming',F.col("Update")/1000000).otherwise(F.col("Update")+0))
        outpdf=met_df.groupBy("`Executor ID`","mname").sum("Update").orderBy("Executor ID").toPandas()

        met_time_cnt=df.where("Event='SparkListenerTaskEnd'")
        exectime=met_time_cnt.groupBy("Executor ID").agg((F.max("Finish Time")-F.min("Launch Time")).alias("totaltime"),F.sum(F.col("`Finish Time`")-F.col("`Launch Time`")).alias("tasktime"))

        totaltime_query=met_time_cnt.groupBy("real_queryid").agg((F.max("Finish Time")-F.min("Launch Time")).alias("totaltime")).agg(F.sum("totaltime").alias("totaltime")).collect()
        totaltime_query=totaltime_query[0]["totaltime"]
        
        pdf=exectime.toPandas()
        exeids=set(outpdf['Executor ID'])
        outpdfs=[outpdf[outpdf["Executor ID"]==l] for l in exeids]
        tasktime=pdf.set_index("Executor ID").to_dict()['tasktime']

        def comb(l,r):
            execid=list(r['Executor ID'])[0]
            lp=r[['mname','sum(Update)']]
            lp.columns=["mname","val_"+execid]
            idle=totaltime_query*exec_cores-tasktime[execid]
            nocount=tasktime[execid]-sum(lp["val_"+execid])
            if idle<0:
                idle=0
            if nocount<0:
                nocount=0
            lp=lp.append([{"mname":"idle","val_"+execid:idle}])
            lp=lp.append([{"mname":"not_counted","val_"+execid:nocount}])
            if l is not None:
                return pandas.merge(lp, l,on=["mname"],how='outer')
            else:
                return lp

        rstpdf=None
        for l in outpdfs[0:]:
            rstpdf=comb(rstpdf,l)
            
        for l in [l for l in rstpdf.columns if l!="mname"]:
            rstpdf[l]=rstpdf[l]/1000/exec_cores
    
        rstpdf=rstpdf.sort_values(by="val_"+list(exeids)[0],axis=0,ascending=False)
        if showexecutor and plot:
            rstpdf.set_index("mname").T.plot.bar(stacked=True,figsize=(30,8))
        pdf_sum=pandas.DataFrame(rstpdf.set_index("mname").T.sum())
        totaltime=totaltime_query/1000
        pdf_sum[0]=pdf_sum[0]/(execs)
        pdf_sum[0]["idle"]=(totaltime_query-sum(tasktime.values())/execs/exec_cores)/1000
        pdf_sum=pdf_sum.sort_values(by=0,axis=0,ascending=False)
        pdf_sum=pdf_sum.T
        pdf_sum.columns=["{:>2.0f}%_{:s}".format(pdf_sum[l][0]/totaltime*100,l) for l in pdf_sum.columns]
        matplotlib.rcParams['font.sans-serif'] = "monospace"
        matplotlib.rcParams['font.family'] = "monospace"
        import matplotlib.font_manager as font_manager
        if plot:
            ax=pdf_sum.plot.bar(stacked=True,figsize=(30,8))
            font = font_manager.FontProperties(family='monospace',
                                               style='normal', size=14)
            ax.legend(prop=font,loc=4)
            plt.title("{:s} q{:d} executors={:d} cores_per_executor={:d} parallelism={:d} sumtime={:.0f} runtime={:.0f}".format(self.file.split("/")[2],queryid,self.executor_instances,self.executor_cores,self.parallelism,totaltime,runtime),fontdict={'fontsize':24})
        return pdf_sum

    def show_critical_path_time_breakdown(self,**kwargs):
        if self.df is None:
            self.load_data()
        return self.show_time_metric(taskids=[l[0].item() for l in self.criticaltasks])
    
    def get_spark_config(self):
        df=spark.read.json(self.file)
        self.appid=df.where("`App ID` is not null").collect()[0]["App ID"]
        pandas.set_option('display.max_rows', None)
        pandas.set_option('display.max_columns', None)
        pandas.set_option('display.max_colwidth', 100000)
        return df.select("Properties.*").where("`spark.app.id` is not null").limit(1).toPandas().T
    
    def get_app_name(self):
        cfg=self.get_spark_config()
        display(HTML("<font size=5 color=red>" + cfg.loc[cfg.index=='spark.app.name'][0][0]+"</font>"))
        
        
    def get_query_time(self,**kwargs):
        if self.df is None:
            self.load_data()
        queryid=kwargs.get("queryid",None)
        showtable=kwargs.get("showtable",True)
        plot=kwargs.get("plot",True)
        
        if queryid and type(queryid)==int:
            queryid = [queryid,]
           
        df=self.df.where(F.col("real_queryid").isin(queryid)) if queryid else self.df.where("queryid is not NULL")
        
            
        stages=df.select("real_queryid","Stage ID").distinct().orderBy("Stage ID").groupBy("real_queryid").agg(F.collect_list("Stage ID").alias("stages")).orderBy("real_queryid")
        runtimeacc=df.where("Event='SparkListenerTaskEnd'") \
                      .groupBy("real_queryid") \
                      .agg(F.round(F.sum(F.col("Finish Time")-F.col("Launch Time"))/1000/self.executor_instances/self.executor_cores*self.taskcpus,2).alias("acc_task_time"))
        inputsize = df.select("real_queryid","Stage ID","Executor ID", "Task ID", F.explode("Accumulables")) \
                      .select("real_queryid","Stage ID","Executor ID", "Task ID","col.*") \
                      .where("Name='input size in bytes' or Name='size of files read'") \
                      .groupBy("real_queryid") \
                      .agg(F.round(F.sum("Update")/1024/1024/1024,2).alias("input read")).orderBy("real_queryid")
        if self.dfacc is not None:
            inputsizev1 = self.dfacc.where("Name='size of files read'").groupBy("real_queryid").agg(F.round(F.sum("Update")/1024/1024/1024,2).alias("input read v1")).orderBy("real_queryid")
            inputsize=inputsize.join(inputsizev1,on="real_queryid",how="outer")
            inputsize=inputsize.withColumn("input read",F.coalesce(F.col("input read"),F.col("input read v1"))).drop("input read v1")
        
        outputrows = df.select("real_queryid","Stage ID","Stage ID",F.explode("Accumulables"))\
                        .select("real_queryid","Stage ID","Stage ID","col.*")\
                        .where("Name='number of output rows'")\
                        .groupBy("real_queryid")\
                        .agg(F.round(F.sum("Update")/1000000000,2).alias("output rows"))
        
        stages=runtimeacc.join(stages,on="real_queryid",how="left")
        stages=inputsize.join(stages,on="real_queryid",how="left")
        stages=stages.join(outputrows,on='real_queryid',how="left")
        
        out=df.groupBy("real_queryid").agg(
            F.round(F.max("job_stop_time")/1000-F.min("job_start_time")/1000,2).alias("runtime"),
            F.round(F.sum("Disk Bytes Spilled")/1024/1024/1024,2).alias("disk spilled"),
            F.round(F.sum("Memory Bytes Spilled")/1024/1024/1024,2).alias("memspilled"),
            F.round(F.sum("Local Bytes Read")/1024/1024/1024,2).alias("local_read"),
            F.round(F.sum("Remote Bytes Read")/1024/1024/1024,2).alias("remote_read"),
            F.round(F.sum("Shuffle Bytes Written")/1024/1024/1024,2).alias("shuffle_write"),
            F.round(F.sum("Executor Deserialize Time")/1000/self.parallelism,2).alias("deser_time"),
            F.round(F.sum("Executor Run Time")/1000/self.parallelism,2).alias("run_time"),
            F.round(F.sum("Result Serialization Time")/1000/self.parallelism,2).alias("ser_time"),
            F.round(F.sum("Fetch Wait Time")/1000/self.parallelism,2).alias("f_wait_time"),
            F.round(F.sum("JVM GC Time")/1000/self.parallelism,2).alias("gc_time"),
            F.max("queryid").alias("queryid")
            ).join(stages,"real_queryid",how="left").orderBy("real_queryid").toPandas().set_index("real_queryid")
        out["executors"]=self.executor_instances
        out["core/exec"]=self.executor_cores
        out["task.cpus"]=self.taskcpus
        out['parallelism']=self.parallelism
        
        if not showtable:
            return out

        def highlight_greater(x):
            m1 = x['acc_task_time'] / x['runtime'] * 100
            m2 = x['run_time'] / x['runtime'] * 100
            m3 = x['f_wait_time'] / x['runtime'] * 100
            

            df1 = pandas.DataFrame('', index=x.index, columns=x.columns)

            df1['acc_task_time'] = m1.apply(lambda x: 'background-image: linear-gradient(to right,#5fba7d {:f}%,white {:f}%)'.format(x,x))
            df1['run_time'] = m2.apply(lambda x: 'background-image: linear-gradient(to right,#5fba7d {:f}%,white {:f}%)'.format(x,x))
            df1['f_wait_time'] = m3.apply(lambda x: 'background-image: linear-gradient(to right,#d65f5f {:f}%,white {:f}%)'.format(x,x))
            return df1


        cm = sns.light_palette("green", as_cmap=True)
        if plot:
            display(out.style.apply(highlight_greater, axis=None).background_gradient(cmap=cm,subset=['input read', 'shuffle_write']))
        
        return out
    
    def get_query_time_metric(self):
        if self.df is None:
            self.load_data()
        querids=self.df.select("queryid").distinct().collect()
        for idx,q in enumerate([l["queryid"] for l in querids]):
            self.show_time_metric(query=[q,],showexecutor=False)
            
    def getOperatorCount(self):
        if self.df is None:
            self.load_data()
        df=spark.read.json(self.file)
        queryids=self.df.select(F.col("queryid").astype(LongType()),F.col("real_queryid")).distinct()
        queryplans=self.queryplans.collect()
        list_queryid=[l.real_queryid for l in queryids.collect()]

        def get_child(execid,node):
            if node["nodeName"] not in qps:
                qps[node["nodeName"]]={l:0 for l in list_queryid}
            qps[node["nodeName"]][execid]=qps[node["nodeName"]][execid]+1
            if node["children"] is not None:
                for c in node["children"]:
                    get_child(execid,c)

        qps={}
        for c in queryplans:
            get_child(c['real_queryid'],c)

        return pandas.DataFrame(qps).T.sort_index(axis=0)        
    
    def get_query_plan(self,**kwargs):
        if self.df is None:
            self.load_data()

        queryid=kwargs.get("queryid",None)
        stageid=kwargs.get("stageid",None)
        
        show_plan_only=kwargs.get("show_plan_only",False)
        show_simple_string=kwargs.get("show_simple_string",False)

        plot=kwargs.get("plot",True)
        
        colors=["#{:02x}{:02x}{:02x}".format(int(l[0]*255),int(l[1]*255),int(l[2]*255)) for l in matplotlib.cm.get_cmap('tab20').colors]

        if queryid is not None and (type(queryid)==int or type(queryid)==str):
            queryid = [queryid,]
            shown_stageid = [l["Stage ID"] for l in self.df.where(F.col("real_queryid").isin(queryid)).select("Stage ID").distinct().collect()]
        if stageid is not None and type(stageid)==int:
            shown_stageid = [stageid,]
            queryid = [l["real_queryid"] for l in self.df.where(F.col("`Stage ID`").isin(shown_stageid)).select("real_queryid").limit(1).collect()]


        queryplans=[]
        queryplans = self.queryplans.where(F.col("real_queryid").isin(queryid)).orderBy("real_queryid").collect() if queryid else self.queryplans.orderBy("real_queryid").collect()
        dfmetric=self.df.where("Event='SparkListenerTaskEnd'").select("queryid","real_queryid","Stage ID","Job ID",F.explode("Accumulables").alias("metric")).select("*","metric.*").select("Stage ID","ID","Update").groupBy("ID","Stage ID").agg(F.round(F.sum("Update"),1).alias("value"),F.round(F.stddev("Update"),1).alias("stdev")).collect()
        accid2stageid={l.ID:(l["Stage ID"],l["value"],l["stdev"]) for l in dfmetric}

        stagetime=self.df.where((F.col("real_queryid").isin(queryid))).where(F.col("Event")=='SparkListenerTaskEnd').groupBy("Stage ID").agg(
            F.round(F.sum(F.col("Finish Time")-F.col("Launch Time"))/1000/self.executor_instances/self.executor_cores*self.taskcpus,1).alias("elapsed time"),
            F.round(F.stddev(F.col("Finish Time")-F.col("Launch Time"))/1000,1).alias("time stdev"),
            F.count(F.col("Task ID")).alias("partitions")
            ).orderBy(F.desc("elapsed time")).collect()

        apptotaltime=reduce(lambda x,y: x+y['elapsed time'], stagetime,0)
        if apptotaltime==0:
            display(HTML("<font size=4 color=red>Error, totaltime is 0 </font>"))
            apptotaltime=1
            return ""

        stagemap={l["Stage ID"]:l["elapsed time"] for l in stagetime}
        stage_time_stdev_map={l["Stage ID"]:l["time stdev"] for l in stagetime}
        stagepartmap={l["Stage ID"]:l["partitions"] for l in stagetime}

        keystage=[]
        keystagetime=[]
        subtotal=0
        for s in stagetime:
            subtotal=subtotal+s['elapsed time']
            keystage.append(s['Stage ID'])
            keystagetime.append(s['elapsed time'])
            if subtotal/apptotaltime>0.9:
                break
        keystagetime=["{:02x}{:02x}".format(int(255*l/keystagetime[0]),255-int(255*l/keystagetime[0])) for l in keystagetime if keystagetime[0]>0]
        keystagemap=dict(zip(keystage,keystagetime))
        outstr=[]
        def print_plan(level,node,parent_stageid):
            stageid = accid2stageid[int(node["metrics"][0]["accumulatorId"])][0]  if len(node["metrics"])>0 and node["metrics"][0]["accumulatorId"] in accid2stageid else parent_stageid

            if stageid in shown_stageid:
                fontcolor=f"color:#{keystagemap[stageid]}00;font-weight:bold" if stageid in keystage else "color:#000000"
                stagetime=0 if stageid not in stagemap else stagemap[stageid]
                stageParts=0 if stageid not in stagepartmap else stagepartmap[stageid]

                input_rowcntstr=""
                output_rowcntstr=""
                timestr=""
                timename=""
                input_columnarbatch=""
                output_columnarbatch=""
                output_row_batch=""
                other_metric_name_str=""
                other_metric_str=""

                outputrows=0
                outputbatches=0
                for m in node["metrics"]:

                    if m["accumulatorId"] not in accid2stageid:
                        continue

                    value=accid2stageid[m["accumulatorId"]][1]
                    stdev_value=accid2stageid[m["accumulatorId"]][2]
                    if m["metricType"] in ['nsTiming','timing']:
                        totaltime=value/1000 if  m["metricType"] == 'timing' else value/1000000000
                        stdev_value=stdev_value/1000 if  m["metricType"] == 'timing' else stdev_value/1000000000
                        timename=timename+m["name"]+"<br>"
                        timeratio= 0  if stagetime==0 else totaltime/self.executor_instances/self.executor_cores*self.taskcpus/stagetime*100
                        timestr=timestr+"{:.2f}s ({:.1f}%, {:.1f}%, {:.2f})<br>".format(totaltime,timeratio, totaltime/self.executor_instances/self.executor_cores*self.taskcpus/apptotaltime*100,stdev_value)
                    elif m["name"]=="number of output rows":
                        output_rowcntstr="{:,.1f}".format(value/1000/1000)+" M"
                        outputrows=value
                    elif m["name"] in ["number of output columnar batches","number of output batches","output_batches"]:
                        output_columnarbatch="{:,d}".format(int(value))
                        outputbatches=value
                    elif m["name"]=="number of input rows":
                        input_rowcntstr="{:,.1f}".format(value/1000/1000)+" M"
                    elif m["name"] in ["number of input batches","number of Input batches","input_batches"]:
                        input_columnarbatch="{:,d}".format(int(value))
                    else:
                        other_metric_name_str=other_metric_name_str+m["name"]+"<br>"
                        if value>1000000000:
                            other_metric_str=other_metric_str+"{:,.1f} G (stdev: {:,.1f})<br>".format(value/1000000000,stdev_value/1000000000)
                        elif value>1000000:
                            other_metric_str=other_metric_str+"{:,.1f} M (stdev: {:,.1f})<br>".format(value/1000000,stdev_value/1000000)
                        elif value>1000:
                            other_metric_str=other_metric_str+"{:,.1f} K (stdev: {:,.1f})<br>".format(value/1000,stdev_value/1000)
                        else:
                            other_metric_str=other_metric_str+"{:,d} (stdev: {:,.1f})<br>".format(int(value),stdev_value)


                if outputrows>0 and outputbatches>0:
                    output_row_batch="{:,d}".format(int(outputrows/outputbatches))


                fontcolor=f"color:#{keystagemap[stageid]}00;font-weight:bold" if stageid in keystage else "color:#000000"
                stagetime=0 if stageid not in stagemap else stagemap[stageid]
                stage_time_stdev=0 if stageid not in stage_time_stdev_map else stage_time_stdev_map[stageid]
                
                if not show_plan_only:
                    nodestr= " ".join(["|_" for l in range(0,level)]) + " " + node["nodeName"]
                    if show_simple_string :
                        nodestr = nodestr +  "<br>" + node['simpleString']
                    outstr.append(f"<tr><td style='{fontcolor}'>{stageid}</td>"+
                                  f"<td style='{fontcolor}'> {stagetime}({stage_time_stdev}) </td>"+
                                  f"<td style='{fontcolor}'> {stageParts} </td>"+
                                  f"<td style='text-align:left; background-color:{colors[stageid % 20]}'>" + nodestr + f"</td>"+
                                  f"<td style='{fontcolor}'> {input_rowcntstr} </td>"+
                                  f"<td style='{fontcolor}'> {input_columnarbatch} </td>"+
                                  f"<td style='{fontcolor}'> {output_rowcntstr} </td>"+
                                  f"<td style='{fontcolor}'> {output_columnarbatch} </td>"+
                                  f"<td style='{fontcolor}'> {output_row_batch} </td>"+
                                  f"<td style='{fontcolor}'> {timename} </td>"+
                                  f"<td style='{fontcolor}'>{timestr}</td>"+
                                  f"<td style='{fontcolor}'> {other_metric_name_str} </td>"+
                                  f"<td style='{fontcolor}'>{other_metric_str}</td></tr>")
                else:
                    outstr.append(f"<tr><td style='{fontcolor}'>{stageid}</td>"+
                                  f"<td style='{fontcolor}'> {stagetime} </td>"+
                                  f"<td style='{fontcolor}'> {stageParts} </td>"+
                                  f"<td style='text-align:left; background-color:{colors[stageid % 20]}'>" + " ".join(["|_" for l in range(0,level)]) + " " + node["nodeName"] + f"</td>"+
                                  f"<td style='{fontcolor}'> {output_rowcntstr} </td></tr>")
                    
            if node["children"] is not None:
                for c in node["children"]:
                    print_plan(level+1,c,stageid)

        for c in queryplans:
            outstr.append("<font color=red size=4>"+str(c['real_queryid'])+"</font><table>")
            if not show_plan_only:
                outstr.append('''<tr>
                                    <td>stage id</td>
                                    <td>stage time</td>
                                    <td>partions</td>
                                    <td>operator</td>
                                    <td>input rows</td>
                                    <td>input batches</td>
                                    <td>output rows</td>
                                    <td>output batches</td>
                                    <td>output rows/batch</td>
                                    <td>time metric name</td>
                                    <td>time(%stage,%total,stdev)</td>
                                    <td>other metric name</td>
                                    <td>value</td>
                                </tr>''')
            else:
                outstr.append('''<tr>
                                    <td>stage id</td>
                                    <td>stage time</td>
                                    <td>partions</td>
                                    <td>operator</td>
                                    <td>output rows</td>
                                </tr>''')

            print_plan(0,c,0)
            outstr.append("</table>")
        if plot:
            display(HTML(" ".join(outstr)))
        return " ".join(outstr)

    def get_metric_output_rowcnt(self, **kwargs):
        return self.get_metric_rowcnt("number of output rows",**kwargs)
        
    def get_metric_input_rowcnt(self, **kwargs):
        return self.get_metric_rowcnt("number of input rows",**kwargs)
        
    def get_metric_rowcnt(self,rowname, **kwargs):
        if self.df is None:
            self.load_data()

        queryid=kwargs.get("queryid",None)

        if queryid and type(queryid)==int:
            queryid = [queryid,]
        queryplans=[]
        queryplans = self.queryplans.where(F.col("real_queryid").isin(queryid)).orderBy("real_queryid").collect() if queryid else self.queryplans.orderBy("real_queryid").collect()


        qps=[]

        rownames=rowname if type(rowname)==list else [rowname,]
        def get_child(execid,node):
            if node['metrics'] is not None:
                outputrows=[x for x in node["metrics"] if "name" in x and x["name"] in rownames]
                if len(outputrows)>0:
                    qps.append([node["nodeName"],execid,outputrows[0]['accumulatorId']])
            if node["children"] is not None:
                for c in node["children"]:
                    get_child(execid,c)
        for c in queryplans:
            get_child(c['real_queryid'],c)

        if len(qps)==0:
            print("Metric ",rowname," is not found. ")
            return None
        stagetime=self.df.where("Event='SparkListenerTaskEnd'").groupBy("Stage ID").agg(F.round(F.sum(F.col("Finish Time")-F.col("Launch Time"))/1000/self.executor_instances/self.executor_cores*self.taskcpus,2).alias("stage time"))
        dfmetric=self.df.where("Event='SparkListenerTaskEnd'").select("queryid","real_queryid","Stage ID","Job ID",F.explode("Accumulables").alias("metric")).select("*","metric.*").drop("metric")
        numrowmetric=spark.createDataFrame(qps)
        numrowmetric=numrowmetric.withColumnRenamed("_1","metric").withColumnRenamed("_2","real_queryid").withColumnRenamed("_3","metricid")
        dfmetric_rowcnt=dfmetric.join(numrowmetric.drop("real_queryid"),on=[F.col("metricid")==F.col("ID")],how="right")
        stagemetric=dfmetric_rowcnt.groupBy("queryid","real_queryid","Job ID","Stage ID","metricid").agg(F.round(F.sum("Update")/1000000,2).alias("total_row"),F.max("metric").alias("nodename")).join(stagetime,"Stage ID")

        if queryid:
            return stagemetric.where(F.col("real_queryid").isin(queryid)).orderBy("Stage ID").toPandas()
        else:
            noderow=stagemetric.groupBy("real_queryid","nodename").agg(F.round(F.sum("total_row"),2).alias("total_row")).orderBy("nodename").collect()
            out={}
            qids=set([r.real_queryid for r in noderow])
            for r in noderow:
                if r.nodename not in out:
                    out[r.nodename]={c:0 for c in qids}
                out[r.nodename][r.real_queryid]=r.total_row
            return pandas.DataFrame(out).T
    
    def get_query_info(self,queryid):
        display(HTML("<font color=red size=7 face='Courier New'><b> time stat info </b></font>",))
        tmp=self.get_query_time(queryid=queryid)
        display(HTML("<font color=red size=7 face='Courier New'><b> stage stat info </b></font>",))
        display(self.get_stage_stat(queryid=queryid))
        display(HTML("<font color=red size=7 face='Courier New'><b> query plan </b></font>",))
        self.get_query_plan(queryid=queryid)
        display(HTML("<font color=red size=7 face='Courier New'><b> stage hist info </b></font>",))
        self.show_Stages_hist(queryid=queryid)
        display(HTML("<font color=red size=7 face='Courier New'><b> time info </b></font>",))
        display(self.show_time_metric(queryid=queryid))
        display(HTML("<font color=red size=7 face='Courier New'><b> operator and rowcount </b></font>",))
        display(self.get_metric_input_rowcnt(queryid=queryid))
        display(self.get_metric_output_rowcnt(queryid=queryid))
        
    def get_app_info(self,**kwargs):
        if self.df is None:
            self.load_data()

        display(HTML(f"<font color=red size=7 face='Courier New'><b> {self.appid} </b></font>",))
        print(f"http://sr525:18080/history/{self.appid}")
        display(HTML("<font color=red size=7 face='Courier New'><b> query time </b></font>",))
        tmp=self.get_query_time(**kwargs)
        display(HTML("<font color=red size=7 face='Courier New'><b> operator count </b></font>",))
        pdf=self.getOperatorCount()
        display(pdf.style.apply(background_gradient,
               cmap='OrRd',
               m=pdf.min().min(),
               M=pdf.max().max(),
               low=0,
               high=1))
        
        display(HTML("<font color=red size=7 face='Courier New'><b> operator input row count </b></font>",))
        pdf=self.get_metric_input_rowcnt(**kwargs)
        if pdf is not None:
            display(pdf.style.apply(background_gradient,
                   cmap='OrRd',
                   m=pdf.min().min(),
                   M=pdf.max().max(),
                   low=0,
                   high=1))
        display(HTML("<font color=red size=7 face='Courier New'><b> operator output row count </b></font>",))
        pdf=self.get_metric_output_rowcnt(**kwargs)
        if pdf is not None:
            display(pdf.style.apply(background_gradient,
                   cmap='OrRd',
                   m=pdf.min().min(),
                   M=pdf.max().max(),
                   low=0,
                   high=1))
        self.show_time_metric(**kwargs)
        
    def get_stage_stat(self,**kwargs):
        if self.df is None:
            self.load_data()

        queryid=kwargs.get("queryid",None)

        if queryid and type(queryid)==int:
            queryid = [queryid,]
            
        df=self.df.where(F.col("real_queryid").isin(queryid)).where(F.col("Event")=='SparkListenerTaskEnd')
        
        inputsize = df.select("real_queryid","Stage ID","Executor ID", "Task ID", F.explode("Accumulables")) \
                      .select("real_queryid","Stage ID","Executor ID", "Task ID","col.*") \
                      .where("Name='input size in bytes' or Name='size of files read'") \
                      .groupBy("Stage ID") \
                      .agg(F.round(F.sum("Update")/1024/1024/1024,2).alias("input read"))
        
        return df.groupBy("Job ID","Stage ID").agg(
            F.round(F.sum(F.col("Finish Time")-F.col("Launch Time"))/1000/self.executor_instances/self.executor_cores*self.taskcpus,1).alias("elapsed time"),
            F.round(F.sum(F.col("Disk Bytes Spilled"))/1024/1024/1024,1).alias("disk spilled"),
            F.round(F.sum(F.col("Memory Bytes Spilled"))/1024/1024/1024,1).alias("mem spilled"),
            F.round(F.sum(F.col("Local Bytes Read"))/1024/1024/1024,1).alias("local read"),
            F.round(F.sum(F.col("Remote Bytes Read"))/1024/1024/1024,1).alias("remote read"),
            F.round(F.sum(F.col("Shuffle Bytes Written"))/1024/1024/1024,1).alias("shuffle write"),
            F.round(F.sum(F.col("Executor Deserialize Time"))/1000,1).alias("deseri time"),
            F.round(F.sum(F.col("Fetch Wait Time"))/1000,1).alias("fetch wait time"),
            F.round(F.sum(F.col("Shuffle Write Time"))/1000000000,1).alias("shuffle write time"),
            F.round(F.sum(F.col("Result Serialization Time"))/1000,1).alias("seri time"),
            F.round(F.sum(F.col("Getting Result Time"))/1000,1).alias("get result time"),
            F.round(F.sum(F.col("JVM GC Time"))/1000,1).alias("gc time"),
            F.round(F.sum(F.col("Executor CPU Time"))/1000000000,1).alias("exe cpu time")    
            ).join(inputsize,on=["Stage ID"],how="left").orderBy("Stage ID").toPandas()
    
    def get_metrics_by_node(self,node_name):
        if self.df is None:
            self.load_data()
            
        metrics=self.queryplans.collect()
        coalesce=[]
        metricsid=[0]
        def get_metric(root):
            if root['nodeName']==node_name:
                metricsid[0]=metricsid[0]+1
                for l in root["metrics"]:
                    coalesce.append([l['accumulatorId'],l["metricType"],l['name'],root["nodeName"],metricsid[0]])
            for c in root["children"]:
                get_metric(c)
        for c in metrics:
            get_metric(c)

        df=self.df.select("queryid","real_queryid",'Stage ID','Task ID','Job ID',F.explode("Accumulables"))
        df=df.select("*","col.*")
        metricdf=spark.createDataFrame(coalesce)
        metricdf=metricdf.withColumnRenamed("_1","ID").withColumnRenamed("_2","Unit").withColumnRenamed("_3","metricName").withColumnRenamed("_4","nodeName").withColumnRenamed("_5","nodeID")
        df=df.join(metricdf,on=["ID"],how="right")
        shufflemetric=set(l[2] for l in coalesce)
        metricdfs=[df.where(F.col("Name")==l).groupBy("real_queryid","nodeID").agg(F.min("Stage ID").alias("Stage ID"),F.mean("Update").alias(l) if l.startswith("avg") else F.sum("Update").alias(l)) for l in shufflemetric]
        return reduce(lambda x,y: x.join(y, on=['nodeID',"Stage ID","real_queryid"],how="full"),metricdfs)
    
    
    def get_coalesce_batch_row_cnt(self,**kwargs):
        stagesum=self.get_metrics_by_node("CoalesceBatches")
        
        pandas.options.display.float_format = '{:,}'.format
        
        stagesum=stagesum.withColumnRenamed("number of output rows","rows")
        
        coalescedf = stagesum.orderBy("real_queryid",'Stage ID').where("rows>4000").toPandas()
        
        coalescedf["row/input_batch"] = coalescedf["rows"]/coalescedf["input_batches"]
        coalescedf["row/out_batch"] = coalescedf["rows"]/coalescedf["output_batches"]
        coalescedf['stage']=coalescedf["real_queryid"].astype(str)+"_"+coalescedf['Stage ID'].astype(str)
        
        ax=coalescedf.plot(y=["row/input_batch","row/out_batch"],figsize=(30,8),style="-*")
        coalescedf.plot(ax=ax,y=['rows'],secondary_y=['rows'],style="k_")
        self.print_real_queryid(ax,coalescedf)
        
        return coalescedf
    
    def print_real_queryid(self,ax,dataset):
        ax.axes.get_xaxis().set_ticks([])

        ymin, ymax = ax.get_ybound()

        real_queryid=list(dataset['real_queryid'])
        s=real_queryid[0]
        lastx=0
        for idx,v in enumerate(real_queryid):
            if v!=s:
                xmin = xmax = idx-1+0.5
                l = mlines.Line2D([xmin,xmax], [ymin,ymax],color="green")
                ax.add_line(l)
                ax.text(lastx+(xmin-lastx)/2-0.25,ymin-(ymax-ymin)/20,f"{s}",size=20)
                s=v
                lastx=xmin

    def get_shuffle_stat(self,**kwargs):
        exchangedf=self.get_metrics_by_node("ColumnarExchange")
        exchangedf.cache()
        exchangedf.count()
        mapdf=exchangedf.where("totaltime_split is not null").select("nodeID",F.col("Stage ID").alias("map_stageid"),"real_queryid","totaltime_split","shuffle write time",'shuffle records written','data size','shuffle bytes written','shuffle bytes spilled','number of input rows')
        reducerdf=exchangedf.where("totaltime_split is null").select("nodeID",F.col("Stage ID").alias("reducer_stageid"),"real_queryid",'local blocks read','local bytes read','avg read batch num rows','remote bytes read','records read','remote blocks read')
        shuffledf=mapdf.join(reducerdf,on=["nodeID","real_queryid"],how="full")
        shuffle_pdf=shuffledf.where("`shuffle bytes written`>1000000000").orderBy("real_queryid","map_stageid").toPandas()
        shuffle_pdf["shuffle bytes written"]=shuffle_pdf["shuffle bytes written"]/1000000000
        shuffle_pdf["data size"]=shuffle_pdf["data size"]/1000000000

        ax=shuffle_pdf.plot(y="avg read batch num rows",figsize=(30,8),style="b-*",title="average batch size after split")
        self.print_real_queryid(ax,shuffle_pdf)
        shuffle_pdf["split_ratio"]=shuffle_pdf["records read"]/shuffle_pdf['shuffle records written']
        ax=shuffle_pdf.plot(y=["split_ratio","records read"],secondary_y=["records read"],figsize=(30,8),style="-*",title="Split Ratio")
        self.print_real_queryid(ax,shuffle_pdf)
        shuffle_pdf["compress_ratio"]=shuffle_pdf["data size"]/shuffle_pdf['shuffle bytes written']
        ax=shuffle_pdf.plot(y=["shuffle bytes written","compress_ratio"],secondary_y=["compress_ratio"],figsize=(30,8),style="-*",title="compress ratio")
        self.print_real_queryid(ax,shuffle_pdf)
        shufflewritepdf=shuffle_pdf
        ax=shufflewritepdf.plot.bar(y=["shuffle write time","totaltime_split"],stacked=True,figsize=(30,8),title="split time + shuffle write time vs. shuffle bytes written")
        ax=shufflewritepdf.plot(ax=ax,y=["shuffle bytes written"],secondary_y=["shuffle bytes written"],style="-*")
        self.print_real_queryid(ax,shufflewritepdf)
        shufflewritepdf.plot(x="shuffle bytes written",y=["shuffle write time","totaltime_split"],figsize=(30,8),style="*")
        shufflewritepdf["avg batch size after split"]=shufflewritepdf["shuffle bytes written"]*1000/shufflewritepdf['records read']
        ax=shufflewritepdf.plot(y=["shuffle bytes written","avg batch size after split"],secondary_y=["shuffle bytes written"],figsize=(30,8),style="-*",title="avg batch MB after split")
        self.print_real_queryid(ax,shufflewritepdf)
        shufflewritepdf["avg batch# per splitted partition"]=shufflewritepdf['records read']/(shufflewritepdf['local blocks read']+shufflewritepdf['remote blocks read'])
        ax=shufflewritepdf.plot(y=["avg batch# per splitted partition",'records read'],secondary_y=['records read'],figsize=(30,8),style="-*",title="avg batch# per splitted partition")
        self.print_real_queryid(ax,shufflewritepdf)
        return shufflewritepdf
    
    def get_stages_w_odd_partitions(appals,**kwargs):
        if appals.df is None:
            appals.load_data()
        return appals.df.where("Event='SparkListenerTaskEnd'")\
                    .groupBy("Stage ID","real_queryid")\
                    .agg((F.sum(F.col('Finish Time')-F.col('Launch Time'))/1000).alias("elapsed time"),
                         F.count('*').alias('partitions'))\
                    .where(F.col("partitions")%(appals.executor_cores*appals.executor_instances/appals.taskcpus)!=0)\
                    .orderBy(F.desc("elapsed time")).toPandas()
   
    def get_scaned_column_v1(appals):
        def get_scans(node):
            if node['nodeName'].startswith("Scan arrow"):
                scans.append(node)
            for c in node['children']:
                get_scans(c)

        alltable=[]
        for qid in range(1,23):
            scans=[]
            plans=appals.queryplans.where("real_queryid="+str(qid)).collect()
            get_scans(plans[0])
            for s in scans:
                alltable.append([qid,",".join([l.split(":")[0] for l in re.split(r'[<>]',s['metadata']['ReadSchema'])[1].split(",")])])
        return alltable
    
    def get_scaned_column_v2(appals):
        def get_scans(node):
            if node['nodeName'].startswith("ColumnarBatchScan"):
                scans.append(node)
            for c in node['children']:
                get_scans(c)

        alltable=[]
        for qid in range(1,23):
            scans=[]
            plans=appals.queryplans.where("real_queryid="+str(qid)).collect()
            get_scans(plans[0])
            for s in scans:
                alltable.append([qid,",".join([l.split("#")[0] for l in re.split(r"[\[\]]",s['simpleString'])[1].split(",")])])
        return alltable

In [ ]:
notlist=['resource.executor.cores',
 'spark.app.id',
 'spark.app.initial.file.urls',
 'spark.app.name',
 'spark.app.startTime',
 'spark.driver.port',
 'spark.job.description',
 'spark.jobGroup.id',
 'spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_HOSTS',
 'spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES',
 'spark.rdd.scope',
 'spark.sql.execution.id',
 '__fetch_continuous_blocks_in_batch_enabled',
 'spark.driver.appUIAddress'
 'spark.driver.appUIAddress',
 'spark.driver.host',
 'spark.driver.appUIAddress',
 'spark.driver.extraClassPath',
 'spark.eventLog.dir',
 'spark.executorEnv.CC',
 'spark.executorEnv.LD_LIBRARY_PATH',
 'spark.executorEnv.LD_PRELOAD',
 'spark.executorEnv.LIBARROW_DIR',
 'spark.files',
 'spark.history.fs.logDirectory',
 'spark.sql.warehouse.dir',
 'spark.yarn.appMasterEnv.LD_PRELOAD',
 'spark.yarn.dist.files'
]
def comp_spark_conf(app0,app1):   
    pdf_sparkconf_0=app0.get_spark_config()
    pdf_sparkconf_1=app1.get_spark_config()
    pdfc=pdf_sparkconf_0.join(pdf_sparkconf_1,lsuffix=app0.appid[-8:],rsuffix=app1.appid[-8:])
    pdfc["0"+app0.appid[-8:]]=pdfc["0"+app0.appid[-8:]].str.lower()
    pdfc["0"+app1.appid[-8:]]=pdfc["0"+app1.appid[-8:]].str.lower()
    
    pdfc['comp']=(pdfc["0"+app0.appid[-8:]]==pdfc["0"+app1.appid[-8:]])
    return pdfc.loc[(pdfc['comp']==False) & (~pdfc.index.isin(notlist))]

## Node log analysis

In [ ]:
@pandas_udf("host string, id string,taskid int, time double", PandasUDFType.GROUPED_MAP)
def collect_udf_time(pdf):
    proxy_handler = request.ProxyHandler({})
    opener = request.build_opener(proxy_handler)

    rst=[]
    for idx,l in pdf.iterrows():
        ip="10.1.2.19"+l['Host'][-1:]
        execid="{:06d}".format(int(l['Executor ID'])+1)
        appid=l['appid']
        url = f'http://{ip}:8042/node/containerlogs/container_{appid}_01_{execid}/yuzhou/stderr/?start=0'
        # open the website with the opener
        req = opener.open(url)
        data = req.read().decode('utf8')
        cnt=data.split("\n")
        cnt_udf=[l.split(" ") for l in cnt if l.startswith('start UDF') or l.startswith('stop UDF')]
        unf_pdf=pandas.DataFrame(cnt_udf)
        srst=unf_pdf.loc[:,[0,4,6]]
        srst.columns=['id','taskid','time']
        srst['host']=l['Host']
        srst['taskid']=srst['taskid'].astype(int)
        srst['time']=srst['time'].apply(lambda f: float(re.search('\d+\.\d+',f).group(0)))
        rst.append(srst)
    return pandas.concat(rst)


class App_Log_Analysis_Node_log(App_Log_Analysis):
    def __init__(self, appid,jobids):
        App_Log_Analysis.__init__(self, appid,jobids)
    
    def generate_trace_view_list(self,id=0, **kwargs):
        if self.df is None:
            self.load_data()

        showcpu=kwargs['showcpu'] if 'showcpu' in kwargs else False
        
        appid=self.appid
        events=self.df.toPandas()
        coretrack={}
        trace_events=[]
        starttime=0
        taskend=[]
        trace={"traceEvents":[]}
        exec_hosts={}
        hostsdf=self.df.select("Host").distinct().orderBy("Host")
        hostid=100000
        ended_event=[]

        for i,l in hostsdf.toPandas().iterrows():
            exec_hosts[l['Host']]=hostid
            hostid=hostid+100000

        tskmap={}
        for idx,l in events.iterrows():
            if l['Event']=='SparkListenerTaskStart':
                hostid=exec_hosts[l['Host']]

                tsk=l['Task ID']
                pid=int(l['Executor ID'])*100+hostid
                stime=l['Launch Time']
                #the task's starttime and finishtime is the same, ignore it.
                if tsk in ended_event:
                    continue
                if not pid in coretrack:
                    tids={}
                    trace_events.append({
                       "name": "process_name",
                       "ph": "M",
                       "pid":pid,
                       "tid":0,
                       "args":{"name":"{:s}.{:s}".format(l['Host'],l['Executor ID'])}
                      })

                else:
                    tids=coretrack[pid]
                for t in tids.keys():
                    if tids[t][0]==-1:
                        tids[t]=[tsk,stime]
                        break
                else:
                    t=len(tids)
                    tids[t]=[tsk,stime]
                #print("task {:d} tid is {:s}.{:d}".format(tsk,pid,t))
                coretrack[pid]=tids

            if l['Event']=='SparkListenerTaskEnd':
                sevt={}
                eevt={}
                hostid=exec_hosts[l['Host']]
                pid=int(l['Executor ID'])*100+hostid
                tsk=l['Task ID']
                fintime=l['Finish Time']

                tids=coretrack[pid]
                for t in tids.keys():
                    if tids[t][0]==tsk:
                        tids[t]=[-1,-1]
                        break
                else:
                    ended_event.append(tsk)
                    continue
                for ps in reversed([key for key in tids.keys()]) :
                    if tids[ps][1]-fintime<0 and tids[ps][1]-fintime>=-2:
                        fintime=tids[ps][1]
                        tids[t]=tids[ps]
                        tids[ps]=[-1,-1]
                        break
                if starttime==0:
                    starttime=l['Launch Time']

                sstime=l['Launch Time']-starttime

                trace_events.append({
                       'tid':pid+int(t),
                       'ts':sstime,
                       'dur':fintime-l['Launch Time'],
                       'pid':pid,
                       "ph":'X',
                       'name':"stg{:d}".format(l['Stage ID']),
                       'args':{"job id": l['job id'],
                               "stage id": l['Stage ID'],
                               "tskid":tsk,
                               "input":builtins.round(l["Bytes Read"]/1024/1024,2),
                               "spill":builtins.round(l["Memory Bytes Spilled"]/1024/1024,2),
                               "Shuffle Read Metrics": "",
                               "|---Local Read": builtins.round(l["Local Bytes Read"]/1024/1024,2),
                               "|---Remote Read":builtins.round(l["Remote Bytes Read"]/1024/1024,2),
                               "Shuffle Write Metrics": "",
                               "|---Write":builtins.round(l['Shuffle Bytes Written']/1024/1024,2)
                               }
                      })
                tskmap[tsk]={'pid':pid,'tid':pid+int(t)}

        self.starttime=starttime
        self.tskmap=tskmap

        hostdf=self.df.select('Host','Executor ID',F.lit(appid[len('application_'):]).alias('appid')).distinct().orderBy('Host')
        rst=hostdf.groupBy('Host').apply(collect_udf_time)
        rst.cache()
        start_df=rst.where("id='start'").select(F.col('taskid').alias('start_taskid'),F.col('time').alias("starttime"))
        stop_df=rst.where("id='stop'").select('taskid',F.col('time').alias("stop_time"))
        df=start_df.join(stop_df, on=[start_df.start_taskid==stop_df.taskid,stop_df['stop_time']>=start_df['starttime']],how='left').groupBy('taskid','starttime').agg(F.min('stop_time').alias('stop_time'))
        pdf=df.toPandas() 
        for idx,l in pdf.iterrows():
                trace_events.append({
                     'tid':self.tskmap[l['taskid']]['tid'],
                     'ts':l['starttime']*1000-self.starttime,
                     'dur':(l['stop_time']-l['starttime'])*1000,                
                     'pid':self.tskmap[l['taskid']]['pid'],
                     'ph':'X',
                     'name':'udf'})
        
        return [json.dumps(l) for l in trace_events]


In [ ]:
class App_Log_Analysis_Node_log(App_Log_Analysis):
    def __init__(self, appid,jobids):
        App_Log_Analysis.__init__(self, appid,jobids)
    
    def generate_trace_view_list(self,id=0, **kwargs):
        if self.df is None:
            self.load_data()

        showcpu=kwargs['showcpu'] if 'showcpu' in kwargs else False
        
        appid=self.appid
        events=self.df.toPandas()
        coretrack={}
        trace_events=[]
        starttime=0
        taskend=[]
        trace={"traceEvents":[]}
        exec_hosts={}
        hostsdf=self.df.select("Host").distinct().orderBy("Host")
        hostid=100000
        ended_event=[]

        for i,l in hostsdf.toPandas().iterrows():
            exec_hosts[l['Host']]=hostid
            hostid=hostid+100000

        tskmap={}
        for idx,l in events.iterrows():
            if l['Event']=='SparkListenerTaskStart':
                hostid=exec_hosts[l['Host']]

                tsk=l['Task ID']
                pid=int(l['Executor ID'])*100+hostid
                stime=l['Launch Time']
                #the task's starttime and finishtime is the same, ignore it.
                if tsk in ended_event:
                    continue
                if not pid in coretrack:
                    tids={}
                    trace_events.append({
                       "name": "process_name",
                       "ph": "M",
                       "pid":pid,
                       "tid":0,
                       "args":{"name":"{:s}.{:s}".format(l['Host'],l['Executor ID'])}
                      })

                else:
                    tids=coretrack[pid]
                for t in tids.keys():
                    if tids[t][0]==-1:
                        tids[t]=[tsk,stime]
                        break
                else:
                    t=len(tids)
                    tids[t]=[tsk,stime]
                #print("task {:d} tid is {:s}.{:d}".format(tsk,pid,t))
                coretrack[pid]=tids

            if l['Event']=='SparkListenerTaskEnd':
                sevt={}
                eevt={}
                hostid=exec_hosts[l['Host']]
                pid=int(l['Executor ID'])*100+hostid
                tsk=l['Task ID']
                fintime=l['Finish Time']

                tids=coretrack[pid]
                for t in tids.keys():
                    if tids[t][0]==tsk:
                        tids[t]=[-1,-1]
                        break
                else:
                    ended_event.append(tsk)
                    continue
                for ps in reversed([key for key in tids.keys()]) :
                    if tids[ps][1]-fintime<0 and tids[ps][1]-fintime>=-2:
                        fintime=tids[ps][1]
                        tids[t]=tids[ps]
                        tids[ps]=[-1,-1]
                        break
                if starttime==0:
                    starttime=l['Launch Time']

                sstime=l['Launch Time']-starttime

                trace_events.append({
                       'tid':pid+int(t),
                       'ts':sstime,
                       'dur':fintime-l['Launch Time'],
                       'pid':pid,
                       "ph":'X',
                       'name':"stg{:d}".format(l['Stage ID']),
                       'args':{"job id": l['job id'],
                               "stage id": l['Stage ID'],
                               "tskid":tsk,
                               "input":builtins.round(l["Bytes Read"]/1024/1024,2),
                               "spill":builtins.round(l["Memory Bytes Spilled"]/1024/1024,2),
                               "Shuffle Read Metrics": "",
                               "|---Local Read": builtins.round(l["Local Bytes Read"]/1024/1024,2),
                               "|---Remote Read":builtins.round(l["Remote Bytes Read"]/1024/1024,2),
                               "Shuffle Write Metrics": "",
                               "|---Write":builtins.round(l['Shuffle Bytes Written']/1024/1024,2)
                               }
                      })
                tskmap[tsk]={'pid':pid,'tid':pid+int(t)}

        self.starttime=starttime
        self.tskmap=tskmap

        hostdf=self.df.select('Host','Executor ID',F.lit(appid[len('application_'):]).alias('appid')).distinct().orderBy('Host')
        rst=hostdf.groupBy('Host').apply(collect_udf_time)
        rst.cache()
        start_df=rst.where("id='start'").select(F.col('taskid').alias('start_taskid'),F.col('time').alias("starttime"))
        stop_df=rst.where("id='stop'").select('taskid',F.col('time').alias("stop_time"))
        df=start_df.join(stop_df, on=[start_df.start_taskid==stop_df.taskid,stop_df['stop_time']>=start_df['starttime']],how='left').groupBy('taskid','starttime').agg(F.min('stop_time').alias('stop_time'))
        pdf=df.toPandas() 
        for idx,l in pdf.iterrows():
                trace_events.append({
                     'tid':self.tskmap[l['taskid']]['tid'],
                     'ts':l['starttime']*1000-self.starttime,
                     'dur':(l['stop_time']-l['starttime'])*1000,                
                     'pid':self.tskmap[l['taskid']]['pid'],
                     'ph':'X',
                     'name':'udf'})
        
        return [json.dumps(l) for l in trace_events]

In [ ]:
class App_Log_Analysis_Node_Log_Uni(App_Log_Analysis):
    def __init__(self, file,jobids):
        App_Log_Analysis.__init__(self, file,jobids)
    
    def generate_trace_view_list(self,id=0, **kwargs):
        if self.df is None:
            self.load_data()

        showcpu=False
        
        shownodes=kwargs.get("shownodes",None)

        showdf=self.df #self.df.where(F.col("Host").isin(shownodes)) if shownodes else self.df

        events=showdf.drop("Accumulables","Stage IDs").orderBy("Launch Time","Finish Time").toPandas()
        coretrack={}
        trace_events=[]
        starttime=0
        taskend=[]
        trace={"traceEvents":[]}
        exec_hosts={}
        hostsdf=showdf.select("Host").distinct().orderBy("Host")
        hostid=100000
        ended_event=[]

        applog=os.path.splitext(self.file)[0]+".stdout"
        logdfs=[]
        if fs.exists(applog):
            logdata=sc.textFile(os.path.splitext(self.file)[0]+".stdout",84)
            logdf=logdata.mapPartitions(splits).toDF()
            logdfs.append(logdf)

        p=os.path.split(self.file)
        for c in shownodes:
            f=p[0]+"/"+c+"/xgbtck.txt"
            if fs.exists(f):
                logdata=sc.textFile(f,84)
                logdf=logdata.mapPartitions(splits).toDF()
                logdfs.append(logdf)
        logdf=reduce(lambda l,r: l.concat(r),logdfs)
        logdf=logdf.cache()
        logdf.count()

        firstrow=logdf.limit(1).collect()

        for c in logdf.columns:
            if firstrow[0][c]!="xgbtck":
                logdf=logdf.drop(c)
            else:
                break

        usefulc=["xgbtck","event","ts","elapsed","threadid","taskid"]
        for i in range(0,len(usefulc)):
            logdf=logdf.withColumnRenamed(logdf.columns[i],usefulc[i])

        logdf=logdf.where(F.col("event").isin(['load_library','data_load','data_convert']))
        
        task_thread=logdf.where("event='data_convert'").select(F.col("taskid").astype(IntegerType()),F.col("threadid").astype(IntegerType())).distinct().toPandas().set_index('taskid').to_dict('index')
        #task_thread={}

        for i,l in hostsdf.toPandas().iterrows():
            exec_hosts[l['Host']]=hostid
            hostid=hostid+100000

        tskmap={}
        for idx,l in events.iterrows():
            if l['Event']=='SparkListenerTaskStart':
                hostid=exec_hosts[l['Host']]

                tsk=l['Task ID']
                pid=int(l['Executor ID'])*100+hostid
                stime=l['Launch Time']
                #the task's starttime and finishtime is the same, ignore it.
                if tsk in ended_event:
                    continue
                if not pid in coretrack:
                    tids={}
                    trace_events.append({
                       "name": "process_name",
                       "ph": "M",
                       "pid":pid,
                       "tid":0,
                       "args":{"name":"{:s}.{:s}".format(l['Host'],l['Executor ID'])}
                      })

                else:
                    tids=coretrack[pid]

                tidarr=[tsk,stime]

                for t in tids.keys():
                    if tids[t][0]==-1:
                        tids[t]=tidarr
                        break
                else:
                    t=len(tids)
                    tids[t]=tidarr
                #print("task {:d} tid is {:s}.{:d}".format(tsk,pid,t))
                coretrack[pid]=tids

            if l['Event']=='SparkListenerTaskEnd':
                sevt={}
                eevt={}
                hostid=exec_hosts[l['Host']]
                pid=int(l['Executor ID'])*100+hostid
                tsk=l['Task ID']
                fintime=l['Finish Time']

                tids=coretrack[pid]
                for t in tids.keys():
                    if tids[t][0]==tsk:
                        tids[t]=[-1,-1]
                        break
                else:
                    ended_event.append(tsk)
                    continue
                for ps in reversed([key for key in tids.keys()]):
                    if (tids[ps][1]-fintime<0 and tids[ps][1]-fintime>=-2) or \
                        (tsk in task_thread and tids[ps][0] in task_thread and task_thread[tsk]["threadid"]==task_thread[tids[ps][0]]["threadid"]):
                        fintime=tids[ps][1]
                        tids[t]=tids[ps]
                        tids[ps]=[-1,-1]
                        break
                if starttime==0:
                    starttime=l['Launch Time']

                sstime=l['Launch Time']-starttime

                trace_events.append({
                       'tid':pid+int(t),
                       'ts':sstime,
                       'dur':fintime-l['Launch Time'],
                       'pid':pid,
                       "ph":'X',
                       'name':"stg{:d}".format(l['Stage ID']),
                       'args':{"job id": l['Job ID'],
                               "stage id": l['Stage ID'],
                               "tskid":tsk,
                               "input":builtins.round(l["Bytes Read"]/1024/1024,2),
                               "spill":builtins.round(l["Memory Bytes Spilled"]/1024/1024,2),
                               "Shuffle Read Metrics": "",
                               "|---Local Read": builtins.round(l["Local Bytes Read"]/1024/1024,2),
                               "|---Remote Read":builtins.round(l["Remote Bytes Read"]/1024/1024,2),
                               "Shuffle Write Metrics": "",
                               "|---Write":builtins.round(l['Shuffle Bytes Written']/1024/1024,2)
                               }
                      })
                tskmap[tsk]={'pid':pid,'tid':pid+int(t)}

        self.starttime=starttime
        self.tskmap=tskmap

        tskmapdf = spark.createDataFrame(pandas.DataFrame(self.tskmap).T.reset_index())
        logdf=logdf.withColumn("ts",F.col("ts").astype(LongType()))
        logdf=logdf.withColumn("taskid",F.col("taskid").astype(LongType()))
        logdf=logdf.withColumnRenamed("event",'type')
        mgd=logdf.join(tskmapdf,on=(F.col('taskid')==F.col("index")),how="right")
        rstdf=mgd.select(F.col('tid').alias("tid"),
          (F.round(F.col('ts')-F.lit(self.starttime),3)).alias("ts"),
          F.round(F.col("elapsed"),3).alias("dur"),
           F.lit(F.col('pid')).alias("pid"),
           F.lit("X").alias("ph"),
           F.col("type").alias("name")
           ).where(F.col("ts").isNotNull()).orderBy('ts')

        #        logdf=logdf.withColumn("type",F.substring_index("event","_",1))
        #        window= Window.partitionBy(logdf['taskid']).orderBy("type","ts")
        #        logdfx=logdf.select("taskid","event","type","ts",F.lag('ts',1).over(window).alias("last"),F.lag('rownum',1).over(window).alias("rownum")).orderBy("taskid","ts").where("event like '%end'")


        output=[json.dumps(l) for l in trace_events]
        output.extend(rstdf.toJSON().collect())

        return output

# perf trace analysis

In [3]:
def split_trace(x):
    fi=[]
    for l in x:
        rst1=re.search(r"^(\d+\.\d+).*sched:(sched_switch):.+:(\d+) \[\d+\] (\S+) ==> .+:(\d+) """,l)
        rst2=re.search(r"(\d+\.\d+) \( +(\d+\.\d+) +ms\):[^/]+/(\d+) (recvfrom|sendto)\(fd: \d+<\S+:\[\d+\]>, \S+: 0x[a-f0-9]+, \S+: (\d+)",l)
        rst3=re.search(r"(\d+\.\d+) \( +\): [^/]+/(\d+) (recvfrom|sendto)\(fd: \d+<\S+:\[\d+\]>, \S+: 0x[a-f0-9]+, \S+: (\d+)",l)
        rst4=re.search(r"(\d+\.\d+) \( *(\d+\.\d+) ms\): [^/]+/(\d+)  ... \[continued\]: (sendto|recvfrom|poll)",l)
        rst5=re.search(r"(\d+\.\d+) \( +(\d+\.\d+) +ms\): [^/]+/(\d+) (poll)",l)
        rst6=re.search(r"(\d+\.\d+) \( +\): [^/]+/(\d+) (poll)",l)

        rstx=re.search(r"(\d+\.\d+)*sched:(sched_switch):.*prev_pid=(\d+).*prev_state=(\S+) ==> .*next_pid=(\d+)""",l)
        if not rst1:
            rst1=rstx
        
        if rst1:
            fi.append((rst1.group(1),rst1.group(2),rst1.group(3),rst1.group(4),rst1.group(5))) #time, switch, src, status, dst
        elif rst2:
            fi.append((rst2.group(1),rst2.group(4),rst2.group(3),rst2.group(2),rst2.group(5))) #time, sed/rcv, pid, ms, size 
        elif rst3:
            fi.append((rst3.group(1),rst3.group(3),rst3.group(2),0, rst3.group(4)))             #time, sed/rcv, pid, 0, size
        elif rst4:
            fi.append((rst4.group(1),rst4.group(4),rst4.group(3),rst4.group(2), 0))              #time, sed/rcv, pid, ms, 0
        elif rst5:
            fi.append((rst5.group(1),rst5.group(4),rst5.group(3),rst5.group(2), 0))              #time, sed/rcv, pid, ms, 0
        elif rst6:
            fi.append((rst6.group(1),rst6.group(3),rst6.group(2),0, 0))              #time, sed/rcv, pid, ms0, 0
        elif not re.match(r"^ +?",l):
            fi.append((0,l,'','',''))
    return iter(fi)
                  


class Perf_trace_analysis(Analysis):
    def __init__(self,sar_file):
        Analysis.__init__(self,sar_file)
        self.starttime=None
        
    def load_data(self):
        sardata=sc.textFile(self.file)
        sardf=sardata.mapPartitions(split_trace).toDF()
        display(sardf.where("_1=0").limit(5).collect())
        sardf=sardf.withColumn("_1",F.col("_1").astype(DoubleType()))
        sardf=sardf.where("_1>0")
        starttime=sardf.agg(F.min("_1")).collect()[0][0]
        if self.starttime is None:
            self.starttime=(float(starttime))
        else:
            paths=os.path.split(self.file)
            if fs.exists(paths[0]+"/uptime.txt"):
                with fs.open(paths[0]+"/uptime.txt") as f:
                    strf=f.read().decode('ascii')
                    print("input starttime:",self.starttime,"uptime:",float(strf)*1000,"record starttime:",starttime)
                    self.starttime=self.starttime-float(strf)*1000
            else:
                print("uptime.txt isn't found, wrong")
                return
            
        self.df=sardf
        return sardf

    def generate_sched_view_list(self,id=0,**kwargs):
        sardf=self.df
        starttime=self.starttime
        starttime=starttime+kwargs.get("sched_time_offset",0)
        print("offset time",starttime)
        
        swdf=sardf.where("_2='sched_switch'")
        
        cputhreshold=kwargs.get("cpu_threshold",0.1)
        sched_cnt = kwargs.get("sched_cnt",10)
        
        pidstat_tids=kwargs.get("pidstat_tids",None)
        pidstat_tids_txt=kwargs.get("pidstat_tids_txt","sched_threads.txt")
        
        if pidstat_tids:
            if type(pidstat_tids) is list:
                tids=pidstat_tids
            else:
                tids=[re.split(r'\s+',t) for t in pidstat_tids.split("\n")]
                tids=[t[3] for t in tids if len(t)>4]
        else:
            paths=os.path.split(self.file)
            if fs.exists(paths[0]+"/"+pidstat_tids_txt):
                with fs.open(paths[0]+"/"+pidstat_tids_txt) as f:
                    tids=[l.strip() for l in f.read().decode('ascii').split("\n") if len(l)>0] 
            else:
                print("Wrong, no pidstat_tids args and no sched_threads.txt file")
                return []
        tidcnt=swdf.where(F.col("_5").isin(tids)).groupBy("_5").count()
        tidm10=tidcnt.where("count>{:d}".format(sched_cnt)).select("_5").collect()
        rtids=[t[0] for t in tidm10]
        rtiddf=swdf.where(F.col("_5").isin(rtids) | F.col("_3").isin(rtids))
        rtiddf=rtiddf.withColumn("_1",F.col("_1").astype(DoubleType())-starttime)
        rtiddf=rtiddf.withColumn("_3",F.col("_3").astype(IntegerType()))
        rtiddf=rtiddf.withColumn("_5",F.col("_5").astype(IntegerType()))
        rtiddf=rtiddf.withColumn("_1",F.round(F.col("_1"),3))
        rtidcol=rtiddf.collect()
        tidmap={}
        tidtotal={}
        for t in rtids:
            tidmap[int(t)]=0
            tidtotal[int(t)]=0
        trace_events=[]
        mintime=rtidcol[0]["_1"]
        maxtime=0
        for r in rtidcol:
            if r["_3"] in tidtotal:
                tidtotal[r["_3"]]=tidtotal[r["_3"]]+r["_1"]-tidmap[r["_3"]]
                tidmap[r["_3"]]=r["_1"]
                maxtime=r["_1"]
            if r["_5"] in tidmap:
                tidmap[r["_5"]]=r["_1"]
        for r in rtidcol:
            if r["_3"] in tidmap and tidtotal[r["_3"]]/(maxtime-mintime)>cputhreshold:
                trace_events.append({
                    'tid':r["_3"],
                     'ts':tidmap[r["_3"]],
                     'pid':id,
                     'ph':'X',
                     'dur':round(r["_1"]-tidmap[r["_3"]],3),
                     'name':r["_4"]
                })

                tidmap[r["_3"]]=r["_1"]
            if r["_5"] in tidmap:
                tidmap[r["_5"]]=r["_1"]
        return [json.dumps(l) for l in trace_events]

    def generate_nic_view_list(self,id=0,**kwargs):
        sardf=self.df
        starttime=self.starttime
        starttime=starttime+kwargs.get("sched_time_offset",0)
        print("offset time",starttime)
        
        nicdf=sardf.where("_2<>'sched_switch'")
        cntdf=nicdf.where("_2='continued'")
        cntdf=cntdf.select("_1","_3","_4").withColumnRenamed("_4","cnt_4")
        nicdf=nicdf.join(cntdf,on=["_1","_3"],how="leftouter")
        nicdf=nicdf.where("_2<>'continued'")
        nicdf=nicdf.select(F.col("_1"),F.col("_2"),F.col("_3"),F.when(F.col("cnt_4").isNull(), F.col("_4")).otherwise(F.col("cnt_4")).alias("_4"),F.col("_5"))
        nicdf=nicdf.withColumn("_1",F.col("_1").astype(DoubleType())-starttime)
        nicdf=nicdf.withColumn("_3",F.col("_3").astype(IntegerType()))
        nicdf=nicdf.withColumn("_5",F.col("_5").astype(IntegerType()))
        nicdf=nicdf.withColumn("_1",F.col("_1").astype(IntegerType()))
        nicdf=nicdf.withColumn("_4",F.col("_4").astype(DoubleType()))
        nicdf=nicdf.withColumn("_4",F.col("_4").astype(LongType()))
        return nicdf.select(
                F.col("_3").alias('tid'),
                (F.col("_1")).alias('ts'),
                F.lit(0).alias('pid'),
                F.lit('X').alias('ph'),
                F.col("_4").alias('dur'),
                F.col("_2").alias('name'),
                F.struct(
                    F.col("_5").alias("size")
                ).alias('args')
            ).toJSON().collect()
    
    def generate_trace_view_list(self,id=0,**kwargs):
        trace_events=Analysis.generate_trace_view_list(self,id,**kwargs)
        sardf=self.df
        starttime=self.starttime
        
        events=self.generate_sched_view_list(id,**kwargs)
        events.extend(self.generate_nic_view_list(id,**kwargs))
        events.extend(trace_events)
        
#        events.extend(nicdf.where("_5>1000 and _2='sendto'").select(
#                 F.lit(0).alias('tid'),
#                F.col("_1").alias('ts'),
#                F.lit(0).alias('pid'),
#                F.lit('i').alias('ph'),
#                F.col("_2").alias('name'),
#                F.lit("g").alias("s")
#            ).toJSON().collect())


        return events
                      

NameError: name 'Analysis' is not defined

# Sar analysis

In [ ]:
def splits(x):
    fi=[]
    for l in x:
        li=re.split(r'\s+',l)
        for j in range(len(li),118):
            li.append('')
        fi.append(li)
    return iter(fi)

class Sar_analysis(Analysis):
    def __init__(self,sar_file):
        Analysis.__init__(self,sar_file)
    
    def load_data(self):
        sardata=sc.textFile(self.file)
        sardf=sardata.mapPartitions(splits).toDF()
        sardf=sardf.where("_1<>'Average:'")
        
        colstart=1;
        ampm=sardf.where("_2='AM' or _2='PM'").count()
        if ampm==0:
            for i in range(len(sardf.columns),1,-1):
                sardf=sardf.withColumnRenamed(f'_{i}',f'_{i+1}')
            self.timeformat='yyyy-MM-dd HH:mm:ss '
            sardf=sardf.withColumn('_2',F.lit(''))
            #print('no PM/AM')
            colstart=1
        else:
            self.timeformat='yyyy-MM-dd hh:mm:ss a'
            colstart=2
            #print('with PM/AM')
        
        f=fs.open(self.file)
        t=f.readline()
        t=f.readline()
        while len(t)==1:
            t=f.readline()
        cols=t.decode('ascii')
        li=re.split(r'\s+',cols)
        ci=3;
        for c in li[colstart:]:
            sardf=sardf.withColumnRenamed(f"_{ci}",c)
            ci=ci+1
            
        sardf=sardf.where(F.col(li[-2])!=li[-2]).where(F.col("_1")!=F.lit("Linux"))        
        
        sardf.cache()
        self.df=sardf
        
        self.sarversion=""
        paths=os.path.split(self.file)
        if fs.exists(paths[0]+"/sarv.txt"):
            with fs.open(paths[0]+"/sarv.txt") as f:
                allcnt = f.read().decode('ascii')
                #print(allcnt)
                self.sarversion=allcnt.split("\n")[0].split(" ")[2]
        
        return sardf

    def col_df(self,cond,colname,args,slaver_id=0, thread_id=0):
        sardf=self.df
        starttime=self.starttime
        cpudf=sardf.where(cond)
        #cpudf.select(F.date_format(F.from_unixtime(F.lit(starttime/1000)), 'yyyy-MM-dd HH:mm:ss').alias('starttime'),'_1').show(1)

        cpudf=cpudf.withColumn('time',F.unix_timestamp(F.concat_ws(' ',F.date_format(F.from_unixtime(F.lit(starttime/1000)), 'yyyy-MM-dd'),F.col('_1'),F.col('_2')),self.timeformat))

        cols=cpudf.columns
                
        cpudf=cpudf.groupBy('time').agg(
            F.sum(F.when(F.col(cols[1]).rlike('^\d+(\.\d+)*$'),F.col(cols[1]).astype(FloatType())).otherwise(0)).alias(cols[1]),
            F.sum(F.when(F.col(cols[2]).rlike('^\d+(\.\d+)*$'),F.col(cols[2]).astype(FloatType())).otherwise(0)).alias(cols[2]),
            *[F.sum(F.col(c)).alias(c) for c in cols[3:] if not c.startswith("_") and c!="" and c!="time"]
        )
        
        traces=cpudf.orderBy(F.col("time")).select(
                F.lit(thread_id).alias('tid'),
                (F.expr("time*1000")-F.lit(self.starttime)).astype(IntegerType()).alias('ts'),
                F.lit(slaver_id).alias('pid'),
                F.lit('C').alias('ph'),
                F.lit(colname).alias('name'),
                args(cpudf).alias('args')
            ).toJSON().collect()
        return traces

    def generate_trace_view_list(self,id,**kwargs):
        trace_events=Analysis.generate_trace_view_list(self,id, **kwargs)
        return trace_events

    def get_stat(self,**kwargs):
        if self.df is None:
            self.load_data()
            
class Sar_cpu_analysis(Sar_analysis):
    def __init__(self,sar_file):
        Sar_analysis.__init__(self,sar_file)
    
    def generate_trace_view_list(self,id,**kwargs):
        trace_events=Sar_analysis.generate_trace_view_list(self,id, **kwargs)
        
        self.df=self.df.withColumn("%iowait",F.when(F.col("%iowait")>100,F.lit(100)).otherwise(F.col("%iowait")))
        
        trace_events.extend(self.col_df("CPU='all'",             "all cpu%",    lambda l: F.struct(
                                                                                                    F.floor(F.col('%user').astype(FloatType())).alias('user'),
                                                                                                    F.floor(F.col('%system').astype(FloatType())).alias('system'),
                                                                                                    F.floor(F.col('%iowait').astype(FloatType())).alias('iowait')
                                                                                                    ),                            id, 0))
        trace_events.append(json.dumps({"name": "thread_sort_index","ph": "M","pid":id,"tid":0,"args":{"sort_index ":0}}))
        
        return trace_events    
    def get_stat(sar_cpu,**kwargs):
        Sar_analysis.get_stat(sar_cpu)
        
        cpuutil=sar_cpu.df.where("CPU='all'").groupBy("_1").agg(*[F.mean(F.col(l).astype(FloatType())).alias(l) for l in ["%user","%system","%iowait"]]).orderBy("_1")
        cnt=cpuutil.count()
        user_morethan_90=cpuutil.where("`%user`>0.9").count()
        kernel_morethan_10=cpuutil.where("`%system`>0.1").count()
        iowait_morethan_10=cpuutil.where("`%iowait`>0.1").count()
        out=[['%user>90%',user_morethan_90/cnt],['%kernel>10%',kernel_morethan_10/cnt],["%iowait>10%",iowait_morethan_10/cnt]]
        avgutil=cpuutil.agg(*[F.mean(l).alias(l) for l in ["%user","%system","%iowait"]]).collect()
        out.extend([["avg " + l,avgutil[0][l]] for l in ["%user","%system","%iowait"]])
        pdout=pandas.DataFrame(out).set_index(0)
        pdout.columns=[sar_cpu.file.split("/")[-2]]
        return pdout
    
class Sar_mem_analysis(Sar_analysis):
    def __init__(self,sar_file):
        Sar_analysis.__init__(self,sar_file)
    
    def load_data(self):
        Sar_analysis.load_data(self)
        if self.sarversion in ("12.2.0","12.5.4"):
            self.df=self.df.withColumn("kbrealused",F.col("kbmemused"))
        else:
            # sar 10.1.5, sar 11.6.1
            self.df=self.df.withColumn("kbrealused",F.col("kbmemused")-F.col("kbcached")-F.col("kbbuffers"))
    
    def generate_trace_view_list(self,id,**kwargs):
        trace_events=Sar_analysis.generate_trace_view_list(self,id, **kwargs)
        
        
        trace_events.extend(self.col_df(F.col('kbmemfree').rlike('^\d+$'),"mem % ",      lambda l: F.struct(F.floor(l['kbcached']*l['%memused']/l['kbmemused']).alias('cached'),  # kbcached / (kbmemfree+kbmemused)
                                                                                                       F.floor(l['kbbuffers']*l['%memused']/l['kbmemused']).alias('buffered'),# kbbuffers / (kbmemfree+kbmemused)
                                                                                                       F.floor(l['kbrealused']*l['%memused']/l['kbmemused']).alias('used')), # (%memused- kbcached-kbbuffers )/  (kbmemfree+kbmemused)
                                          id,1))
        #trace_events.extend(self.col_df(self.df._3.rlike('^\d+$'),"mem cmt % ",  lambda l: F.struct(F.floor(l._8*F.lit(100)/(l._3+l._4)).alias('commit/phy'),
        #                                                                                                   F.floor(l._10-l._8*F.lit(100)/(l._3+l._4)).alias('commit/all')),                                                             id))
        trace_events.append(json.dumps({"name": "thread_sort_index","ph": "M","pid":id,"tid":1,"args":{"sort_index ":1}}))
        return trace_events    
    def get_stat(sar_mem,**kwargs):
        Sar_analysis.get_stat(sar_mem)
        
        memutil=sar_mem.df.where(F.col('kbmemfree').rlike('^\d+$')).select(F.floor(F.col('kbcached').astype(FloatType())*F.lit(100)*F.col('%memused')/F.col('kbmemused')).alias('cached'),  
                                                                                   F.floor(F.col('kbbuffers').astype(FloatType())*F.lit(100)*F.col('%memused')/F.col('kbmemused')).alias('buffered'),
                                                                                   F.floor(F.col('kbrealused').astype(FloatType())*F.lit(100)*F.col('%memused')/F.col('kbmemused')).alias('used'))
        memsum=memutil.summary().toPandas()
        memsum=memsum.set_index("summary")
        out=[
            [[l + ' mean',float(memsum[l]["mean"])],
            [l + ' 75%',float(memsum[l]["75%"])],
            [l + ' max',float(memsum[l]["max"])]] for l in ["cached","used"]]
        out=[*out[0],*out[1]]
        pdout=pandas.DataFrame(out).set_index(0)
        pdout.columns=[sar_mem.file.split("/")[-2]]
        return pdout
    

class Sar_disk_analysis(Sar_analysis):
    def __init__(self,sar_file):
        Sar_analysis.__init__(self,sar_file)
        
    def load_data(self):
        Sar_analysis.load_data(self)
        
        self.df=self.df.withColumn("%util",F.col("%util").astype(IntegerType()))
        used_disk=self.df.groupBy("DEV").agg(F.max(F.col("%util")).alias("max_util"),F.mean("%util").alias("avg_util")).where(F.col("max_util")>10).collect()
        self.df=self.df.where(F.col("DEV").isin([l['DEV'] for l in used_disk]))
        #print("used disks with its max util% and avg util% are: ")
        #display([(l['DEV'],l["max_util"],l["avg_util"]) for l in used_disk])
        
        if "rd_sec/s" in self.df.columns:
            self.df=self.df.withColumn("rkB/s",F.expr('cast(`rd_sec/s` as float)*512/1024'))
        if "wr_sec/s" in self.df.columns:
            self.df=self.df.withColumn("wkB/s",F.expr('cast(`wr_sec/s` as float)*512/1024'))
        
        if "areq-sz" in self.df.columns:
            self.df=self.df.withColumnRenamed("areq-sz","avgrq-sz")
        if "aqu-sz" in self.df.columns:
            self.df=self.df.withColumnRenamed("aqu-sz","avgqu-sz")
            
        if "rkB/s" in self.df.columns:
            self.df=self.df.withColumn("rkB/s",F.expr('cast(`rkB/s` as float)/1024'))
        if "wkB/s" in self.df.columns:
            self.df=self.df.withColumn("wkB/s",F.expr('cast(`wkB/s` as float)/1024'))

    def generate_trace_view_list(self,id,**kwargs):
        trace_events=Sar_analysis.generate_trace_view_list(self,id, **kwargs)

        disk_prefix=kwargs.get('disk_prefix',"")
        
        
        devcnt=self.df.where("DEV like '"+disk_prefix+"%'").select("DEV").distinct().count()
        
        trace_events.extend(self.col_df("DEV like '"+disk_prefix+"%'",      "disk b/w",       lambda l: F.struct(
                                                                                                            F.floor(F.col("rKB/s")).alias('read'),
                                                                                                            F.floor(F.col("wKB/s")).alias('write')),id, 2))
        trace_events.extend(self.col_df("DEV like '"+disk_prefix+"%'",      "disk%",       lambda l: F.struct(
                                                                                                            (F.col("%util")/F.lit(devcnt)).alias('%util')),id, 3))
        trace_events.extend(self.col_df("DEV like '"+disk_prefix+"%'",      "req size",       lambda l: F.struct(
                                                                                                            (F.col("avgrq-sz")/F.lit(devcnt)).alias('avgrq-sz')),id, 4))
        trace_events.extend(self.col_df("DEV like '"+disk_prefix+"%'",      "queue size",       lambda l: F.struct(
                                                                                                            (F.col("avgqu-sz")/F.lit(512*devcnt/1024)).alias('avgqu-sz')),id, 5))
        trace_events.extend(self.col_df("DEV like '"+disk_prefix+"%'",      "await",       lambda l: F.struct(
                                                                                                            (F.col("await")/F.lit(devcnt)).alias('await')),id, 6))
        
        trace_events.append(json.dumps({"name": "thread_sort_index","ph": "M","pid":id,"tid":2,"args":{"sort_index ":2}}))
        trace_events.append(json.dumps({"name": "thread_sort_index","ph": "M","pid":id,"tid":3,"args":{"sort_index ":3}}))
        trace_events.append(json.dumps({"name": "thread_sort_index","ph": "M","pid":id,"tid":4,"args":{"sort_index ":4}}))
        trace_events.append(json.dumps({"name": "thread_sort_index","ph": "M","pid":id,"tid":5,"args":{"sort_index ":5}}))
        trace_events.append(json.dumps({"name": "thread_sort_index","ph": "M","pid":id,"tid":6,"args":{"sort_index ":6}}))
        return trace_events    

    def get_stat(sar_disk,**kwargs):
        Sar_analysis.get_stat(sar_disk)
        disk_prefix=kwargs.get('disk_prefix',"")

        diskutil=sar_disk.df.where(f"DEV like '{disk_prefix}%'").groupBy("_1").agg(F.mean(F.col("%util").astype(FloatType())).alias("%util")).orderBy("_1")
        totalcnt=diskutil.count()
        time_morethan_90=diskutil.where(F.col("%util")>90).count()/totalcnt
        avgutil=diskutil.agg(F.mean("%util")).collect()
        out=[["avg disk util",avgutil[0]["avg(%util)"]],
            ["time more than 90%", time_morethan_90]]
        diskbw=sar_disk.df.where(f"DEV like '{disk_prefix}%'").groupBy("_1").agg(F.sum(F.col("rKB/s")).alias("rd_bw"),F.sum(F.col("wKB/s")).alias("wr_bw"))
        bw=diskbw.agg(F.sum("rd_bw").alias("total read"),F.sum("wr_bw").alias("total write"),F.mean("rd_bw").alias("read bw"),F.mean("wr_bw").alias("write bw"),F.max("rd_bw").alias("max read"),F.max("wr_bw").alias("max write")).collect()
        maxread=bw[0]["max read"]
        maxwrite=bw[0]["max write"]
        rdstat, wrstat = diskbw.stat.approxQuantile(['rd_bw','wr_bw'],[0.75,0.95,0.99],0.0)
        time_rd_morethan_95 = diskbw.where(F.col("rd_bw")>rdstat[1]).count()/totalcnt
        time_wr_morethan_95 = diskbw.where(F.col("wr_bw")>rdstat[1]).count()/totalcnt
        out.append(['total read (G)' , bw[0]["total read"]/1024])
        out.append(['total write (G)', bw[0]["total write"]/1024])
        out.append(['avg read bw (MB/s)', bw[0]["read bw"]])
        out.append(['avg write bw (MB/s)', bw[0]["write bw"]])
        out.append(['read bw %75', rdstat[0]])
        out.append(['read bw %95', rdstat[1]])
        out.append(['read bw max', rdstat[2]])
        out.append(['time_rd_morethan_95', time_rd_morethan_95])
        out.append(['write bw %75', wrstat[0]])
        out.append(['write bw %95', wrstat[1]])
        out.append(['write bw max', wrstat[2]])
        out.append(['time_wr_morethan_95', time_wr_morethan_95])
        pdout=pandas.DataFrame(out).set_index(0)
        pdout.columns=[sar_disk.file.split("/")[-2]]
        return pdout
    
class Sar_nic_analysis(Sar_analysis):
    def __init__(self,sar_file):
        Sar_analysis.__init__(self,sar_file)
    
    def generate_trace_view_list(self,id,**kwargs):
        trace_events=Sar_analysis.generate_trace_view_list(self,id, **kwargs)
        
        nicfilter=""
        if 'nic_prefix' in kwargs.keys():
            nicfilter= "IFACE in (" + ",".join(kwargs.get('nic_prefix',["'eth3'","'enp24s0f1'"])) + ")"
        else:
            nicfilter= "IFACE != 'lo'"
        
        trace_events.extend(self.col_df(nicfilter,       "eth ",        lambda l: F.struct(F.floor(F.expr('cast(`rxkB/s` as float)/1024')).alias('rxmb/s'),F.floor(F.expr('cast(`txkB/s` as float)/1024')).alias('txmb/s')),                id, 7))
        trace_events.extend(self.col_df("_3 like 'ib%'",        "ib ",        lambda l: F.struct(F.floor(F.expr('cast(`rxkB/s` as float)/1024')).alias('rxmb/s'),F.floor(F.expr('cast(`txkB/s` as float)/1024')).alias('txmb/s')),                id, 8))
        trace_events.extend(self.col_df("_3 = 'lo'",            "lo ",         lambda l: F.struct(F.floor(F.expr('cast (`rxkB/s` as float)/1024')).alias('rxmb/s'),F.floor(F.expr('cast (`txkB/s` as float)/1024')).alias('txmb/s')),              id, 9))
        trace_events.append(json.dumps({"name": "thread_sort_index","ph": "M","pid":id,"tid":7,"args":{"sort_index ":7}}))
        trace_events.append(json.dumps({"name": "thread_sort_index","ph": "M","pid":id,"tid":8,"args":{"sort_index ":8}}))
        trace_events.append(json.dumps({"name": "thread_sort_index","ph": "M","pid":id,"tid":9,"args":{"sort_index ":9}}))
        return trace_events  
    
    def get_stat(sar_nic,**kwargs):
        Sar_analysis.get_stat(sar_nic)
        nicfilter=""
        
        if 'nic_prefix' in kwargs.keys():
            nicfilter= "IFACE in (" + ",".join(kwargs.get('nic_prefix',["'eth3'","'enp24s0f1'"])) + ")"
        else:
            nicfilter= "IFACE != 'lo'"
            
        nicbw=sar_nic.df.where(nicfilter).groupBy("_1").agg(F.sum(F.col("rxkB/s").astype(FloatType())/1024).alias("rx MB/s")).orderBy("_1")
        out=nicbw.stat.approxQuantile(['rx MB/s'],[0.75,0.95,0.99],0.0)[0]
        out=[["rx MB/s 75%",out[0]],["rx MB/s 95%",out[1]],["rx MB/s 99%",out[2]]]
        pdout=pandas.DataFrame(out).set_index(0)
        pdout.columns=[sar_nic.file.split("/")[-2]]
        return pdout

# PID State analysis

In [ ]:
class Pidstat_analysis(Analysis):
    def __init__(self,sar_file):
        Analysis.__init__(self,sar_file)
    
    def load_data(self):
        sardata=sc.textFile(self.file)
        sardf=sardata.mapPartitions(splits).toDF()
        sardf=sardf.where("_1<>'Average:'")
        
        headers=sardf.where("_4='TID' or _5='TID'").limit(1).collect()
        r=headers[0].asDict()
        findtime=False
        for i,v in r.items():
            if(v=="Time"):
                findtime=True
        if not findtime:
            r["_1"]="Time"
        for i,v in r.items():
            if(v!=""):
                sardf=sardf.withColumnRenamed(i,v)
        sardf=sardf.where("TGID='0' or TGID='-'") 

        self.df=sardf
        return sardf


    def generate_trace_view_list(self,id,**kwargs):
        trace_list=Analysis.generate_trace_view_list(self,id,**kwargs)
        sardf=self.df
        starttime=self.starttime
        
        sardf=sardf.withColumn("%CPU",F.col("%CPU").astype(FloatType()))
        sardf=sardf.withColumn("Time",F.col("Time").astype(LongType()))
        sardf=sardf.withColumn("TID",F.col("TID").astype(LongType()))
        hotthreads=sardf.where("`%CPU`>30").groupBy("TID").count().collect()
        hts=[(r[0],r[1]) for r in hotthreads]
        htc=[r[1] for r in hotthreads]
        if len(htc)==0:
            return trace_list
        maxcnt=max(htc)
        hts=[r[0] for r in hts if r[1]>maxcnt/2]
        tdfs=list(map(lambda x: sardf.withColumnRenamed("TID","TID_"+str(x)).withColumnRenamed("%CPU","CPU_"+str(x)).where(F.col("TID")==x).select("Time","TID_"+str(x),"CPU_"+str(x)),hts))
        finaldf=reduce(lambda x,y: x.join(y,on=["Time"]),tdfs)
        othersdf=sardf.where("TID not in ("+",".join(map(lambda x: str(x),hts))+")").groupBy("Time").agg(F.sum("%CPU").alias("CPU_Other"))
        finaldf=finaldf.join(othersdf,on=["Time"])
        finaldf=finaldf.orderBy("Time")
        hts.append("Other")
        stt=[F.col("CPU_"+str(x)).alias(str(x)) for x in hts]
        args=F.struct(*stt)
        
        trace_list.extend(finaldf.select(
                F.lit(6).alias('tid'),
                (F.expr("Time*1000")-F.lit(starttime)).astype(IntegerType()).alias('ts'),
                F.lit(id).alias('pid'),
                F.lit('C').alias('ph'),
                F.lit("pidstat").alias('name'),
                args.alias('args')
            ).toJSON().collect())
        return trace_list
        

# Perf Trace Analysis

In [1]:
class Perfstat_analysis(Analysis):
    def __init__(self,sar_file):
        Analysis.__init__(self,sar_file)
    
    def load_data(self):
        sardata=sc.textFile(self.file)
        sardf=sardata.mapPartitions(splits).toDF()
        
        paths=os.path.split(self.file)
        if fs.exists(paths[0]+"/perfstarttime"):
            with fs.open(paths[0]+"/perfstarttime") as f:
                strf=f.read().decode('ascii')
        else:
            print("error, perfstarttime not found")
            return
        
        strf=strf[len("# started on "):].strip()
        starttime=datetime.strptime(strf, "%a %b %d %H:%M:%S %Y").timestamp()*1000
        sardf=sardf.where("_1<>'#'")
        sardf=sardf.withColumn("ts",F.col("_2").astype(DoubleType())*1000+F.lit(starttime)).where("ts is not null").select("ts","_3","_4")
        sardf=sardf.withColumn('_3', F.regexp_replace('_3', ',', '').astype(LongType()))
        sardf=sardf.cache()
        self.df=sardf
        return sardf


    def generate_trace_view_list(self,id,**kwargs):
        trace_list=Analysis.generate_trace_view_list(self,id,**kwargs)
        sardf=self.df
        starttime=self.starttime
        
        stringIndexer = StringIndexer(inputCol="_4", outputCol="syscall_idx")
        model = stringIndexer.fit(sardf)
        sardf=model.transform(sardf)
        
#        cnts=sardf.select("_4").distinct().collect()
#        cnts=[l['_4'] for l in cnts]
#        cntmap={ cnts[i]:i  for i in range(0, len(cnts) ) }
#        mapexpr=F.create_map([F.lit(x) for x in chain(*cntmap.items())])
#        sardf.select(mapexpr.getItem(F.col("_4")))
        
        sardf=sardf.withColumn("syscall_idx",F.col("syscall_idx").astype(IntegerType()))
        
        trace_list.extend(sardf.select(
            (F.lit(100)+F.col("syscall_idx")).alias('tid'),
            (F.col("ts")-F.lit(starttime)).astype(LongType()).alias('ts'),
            F.lit(id).alias('pid'),
            F.lit('C').alias('ph'),
            F.col("_4").alias('name'),
            F.struct(F.col("_3").alias("cnt")).alias('args')
        ).toJSON().collect())
        return trace_list
        


NameError: name 'Analysis' is not defined

# GPU analysis

In [ ]:
class gpu_analysis(Analysis):
    def __init__(self,gpu_file):
        Analysis.__init__(self,gpu_file)
        
    def load_data(self):
        df_pf=spark.read.format("com.databricks.spark.csv").option("header","true").option("mode", "DROPMALFORMED").option("delimiter", ",").load(self.file)
        df_pf2=df_pf.withColumn('timestamp',F.unix_timestamp(F.col('timestamp'),'yyyy/MM/dd HH:mm:ss')*1000+(F.split(F.col('timestamp'),'\.')[1]).astype(IntegerType()))
        df_pf2=df_pf2.withColumnRenamed(' utilization.gpu [%]','gpu_util')
        df_pf2=df_pf2.withColumnRenamed(' utilization.memory [%]','mem_util')
        df_pf2=df_pf2.withColumnRenamed(' memory.used [MiB]','mem_used')
        df_pf2=df_pf2.withColumnRenamed(' index','index')
        df_pf2=df_pf2.withColumn('gpu_util', (F.split('gpu_util',' ')[1]).astype(IntegerType()))
        df_pf2=df_pf2.withColumn('mem_util', (F.split('mem_util',' ')[1]).astype(IntegerType()))
        df_pf2=df_pf2.withColumn('mem_used', (F.split('mem_used',' ')[1]).astype(IntegerType()))
        df_pf.cache()
        self.df=df_pf2
        return df_pf2

    def generate_trace_view_list(self,id,**kwargs):
        Analysis.generate_trace_view_list(self,id)
            
        df_pf2=self.df
        starttime=self.starttime
        trace_events=[]
        
        trace_events.extend(df_pf2.orderBy(df_pf2['timestamp']).select(
            F.col('index').alias('tid'),
            (F.expr("timestamp")-F.lit(starttime)).astype(IntegerType()).alias('ts'),
            F.lit(id).alias('pid'),
            F.lit('C').alias('ph'),
            F.concat(F.lit('gpu_util_'),F.col('index')).alias('name'),
            F.struct(F.col('gpu_util').alias('gpu')).alias('args')
        ).toJSON().collect())

        trace_events.extend(df_pf2.orderBy(df_pf2['timestamp']).select(
            F.col('index').alias('tid'),
            (F.expr("timestamp")-F.lit(starttime)).astype(IntegerType()).alias('ts'),
            F.lit(int(id)+1).alias('pid'),
            F.lit('C').alias('ph'),
            F.concat(F.lit('mem_util_'),F.col('index')).alias('name'),
            F.struct((F.col('mem_used')/F.lit(32768)).alias('mem')).alias('args')
        ).toJSON().collect())

        return trace_events

In [ ]:
def splits_dmon(x):
    fi=[]
    for l in x:
        l=l.strip()
        if l.startswith('20'):
            li=re.split(r'\s+',l)
            if len(li)==11:
                fi.append(li)
    return iter(fi)

class gpu_dmon_analysis(Analysis):
    def __init__(self,gpu_file):
        Analysis.__init__(self,gpu_file)
        
    def load_data(self):
        df_pf=sc.textFile(self.file)
        df_pf=df_pf.mapPartitions(splits_dmon).toDF()
        
        df_pf2=df_pf.withColumn('_1',F.unix_timestamp(F.concat_ws(' ',F.col('_1'),F.col('_2')),'yyyyMMdd HH:mm:ss')*1000)
        for c in range(3,12):
            df_pf2=df_pf2.withColumn(f'_{c}',F.col(f'_{c}').astype(IntegerType()))

        df_pf.cache()
        self.df=df_pf2
        return df_pf2

    def generate_trace_view_list(self,id,**kwargs):
        Analysis.generate_trace_view_list(self,id)

        df_pf2=self.df
        starttime=self.starttime
        trace_events=[]
        
        trace_events.extend(df_pf2.orderBy(df_pf2['_1']).select(
            F.col('_3').alias('tid'),
            (F.expr("_1")-F.lit(starttime)).astype(IntegerType()).alias('ts'),
            F.lit(id).alias('pid'),
            F.lit('C').alias('ph'),
            F.concat(F.lit('gpu_util_'),F.col('_3')).alias('name'),
            F.struct(F.col('_4').alias('gpu')).alias('args')
        ).toJSON().collect())

        trace_events.extend(df_pf2.orderBy(df_pf2['_1']).select(
            F.col('_3').alias('tid'),
            (F.expr("_1")-F.lit(starttime)).astype(IntegerType()).alias('ts'),
            F.lit(id+1).alias('pid'),
            F.lit('C').alias('ph'),
            F.concat(F.lit('mem_util_'),F.col('_3')).alias('name'),
            F.struct(F.col('_5').alias('mem')).alias('args')
        ).toJSON().collect())

        trace_events.extend(df_pf2.orderBy(df_pf2['_1']).select(
            F.col('_3').alias('tid'),
            (F.expr("_1")-F.lit(starttime)).astype(IntegerType()).alias('ts'),
            F.lit(id+2).alias('pid'),
            F.lit('C').alias('ph'),
            F.concat(F.lit('gpu_freq_'),F.col('_3')).alias('name'),
            F.struct(F.col('_9').alias('gpu_freq')).alias('args')
        ).toJSON().collect())

        trace_events.extend(df_pf2.orderBy(df_pf2['_1']).select(
            F.col('_3').alias('tid'),
            (F.expr("_1")-F.lit(starttime)).astype(IntegerType()).alias('ts'),
            F.lit(id+3).alias('pid'),
            F.lit('C').alias('ph'),
            F.concat(F.lit('pcie_'),F.col('_3')).alias('name'),
            F.struct(F.col('_10').alias('tx'),F.col('_11').alias('rx')).alias('args')
        ).toJSON().collect())

        return trace_events


# DASK analysis

In [26]:
def split_dask(x):
    fi=[]
    for l in x:
        print(l)
        li=[]
        if l.startswith('('):
            lx=re.split(r'[()]',l)
            lv=lx[1]
            p=re.search(r"'(.*)-([0-9a-f]+)', *(\d+)",lv)
            if not p:
                print("dask log first field doesn't match (.*)-[0-9a-f]+', *(\d+)")
                return
            li.append(p.group(1))
            li.extend(lx[2].split(",")[1:])
            li.append(p.group(3))
        else:
            li=l.split(',')
            p=re.search(r"(.*)-([0-9a-f]+-[0-9a-f]+-[0-9a-f]+-[0-9a-f]+-[0-9a-f]+)$",li[0])
            if not p:
                p=re.search(r"(.*)-([0-9a-f]+)$",li[0])
            
            li[0]=p.group(1)
            li.append(p.group(2))
        fi.append(li)
    return iter(fi)

class dask_analysis(Analysis):
    def __init__(self,dask_file):
        Analysis.__init__(self,dask_file)

    def load_data(self):
        rdds=sc.textFile(self.file)
        df_pf=rdds.mapPartitions(split_dask).toDF()
        df_pf=df_pf.withColumnRenamed('_1','_c0')
        df_pf=df_pf.withColumnRenamed('_2','_c1')
        df_pf=df_pf.withColumnRenamed('_3','_c2')
        df_pf=df_pf.withColumnRenamed('_4','_c3')
        df_pf=df_pf.withColumnRenamed('_5','_id')
        
        df_pf=df_pf.withColumn('_c1',F.split(F.col('_c1'),":")[2])
        df_pf=df_pf.withColumn('_c3',df_pf._c3.astype(DoubleType())*1000) 
        df_pf=df_pf.withColumn('_c2',df_pf._c2.astype(DoubleType())*1000)
        
        df_pf.cache()
        self.df=df_pf
        self.starttime=df_pf.agg(F.min("_c2")).collect()[0]['min(_c2)']
        return df_pf

    def generate_trace_view_list(self,id,**kwargs):
        Analysis.generate_trace_view_list(self,id)
        
        df_pf=self.df

        window = Window.partitionBy("_c1").orderBy("_c3")
        df_pf=df_pf.withColumn("last_tsk_done", F.lag('_c3', 1, None).over(window))
        df_pf=df_pf.withColumn('last_tsk_done',F.coalesce('last_tsk_done','_c2'))
        df_pf=df_pf.withColumn('last_tsk_done',F.when(F.col('_c2')>F.col('last_tsk_done'),F.col('_c2')).otherwise(F.col('last_tsk_done')) )
        
        trace_events=[]
        
        trace_events.extend(df_pf.select(
            F.col('_c1').alias('tid'),
            (F.col('last_tsk_done')-F.lit(self.starttime)).astype(IntegerType()).alias('ts'),
            F.expr('_c3 - last_tsk_done  ').alias('dur'),
            F.lit(id).alias('pid'),
            F.lit('X').alias('ph'),
            F.col('_c0').alias('name'),
            F.struct(F.col('_id').alias('uuid')).alias('args')
        ).toJSON().collect())

        return trace_events

In [ ]:
class dask_analysis_log(dask_analysis):
    def __init__(self,dask_file,logs):
        Analysis.__init__(self,dask_file)

    def load_data(self):
        rdds=sc.textFile(self.file)
        df_pf=rdds.mapPartitions(split_dask).toDF()
        df_pf=df_pf.withColumnRenamed('_1','_c0')
        df_pf=df_pf.withColumnRenamed('_2','_c1')
        df_pf=df_pf.withColumnRenamed('_3','_c2')
        df_pf=df_pf.withColumnRenamed('_4','_c3')
        df_pf=df_pf.withColumnRenamed('_5','_id')
        
        df_pf=df_pf.withColumn('_c1',F.split(F.col('_c1'),":")[2])
        df_pf=df_pf.withColumn('_c3',df_pf._c3.astype(DoubleType())*1000) 
        df_pf=df_pf.withColumn('_c2',df_pf._c2.astype(DoubleType())*1000)
        
        df_pf.cache()
        self.df=df_pf
        self.starttime=df_pf.agg(F.min("_c2")).collect()[0]['min(_c2)']
        return df_pf

    def generate_trace_view_list(self,id,**kwargs):
        Analysis.generate_trace_view_list(self,id)
        
        df_pf=self.df

        window = Window.partitionBy("_c1").orderBy("_c3")
        df_pf=df_pf.withColumn("last_tsk_done", F.lag('_c3', 1, None).over(window))
        df_pf=df_pf.withColumn('last_tsk_done',F.coalesce('last_tsk_done','_c2'))
        df_pf=df_pf.withColumn('last_tsk_done',F.when(F.col('_c2')>F.col('last_tsk_done'),F.col('_c2')).otherwise(F.col('last_tsk_done')) )
        
        trace_events=[]
        
        trace_events.extend(df_pf.select(
            F.col('_c1').alias('tid'),
            (F.col('last_tsk_done')-F.lit(self.starttime)).astype(IntegerType()).alias('ts'),
            F.expr('_c3 - last_tsk_done  ').alias('dur'),
            F.lit(id).alias('pid'),
            F.lit('X').alias('ph'),
            F.col('_c0').alias('name'),
            F.struct(F.col('_id').alias('uuid')).alias('args')
        ).toJSON().collect())

        return trace_events

# instantevent analysis

In [ ]:
## format: _2 = Name; _3 = time

class InstantEvent_analysis(Analysis):
    def __init__(self,sar_file):
        Analysis.__init__(self,sar_file)
    
    def load_data(self):
        sardata=sc.textFile(self.file)
        sardf=sardata.mapPartitions(splits).toDF()
        self.df=sardf
        return sardf


    def generate_trace_view_list(self,id=0,**kwargs):
        Analysis.generate_trace_view_list(self,id)
        sardf=self.df
        starttime=self.starttime
        return sardf.select(F.lit(0).alias('tid'),
                (F.col("_3").astype(DoubleType())*1000-F.lit(starttime)).astype(IntegerType()).alias('ts'),
                F.lit(0).alias('pid'),
                F.lit('i').alias('ph'),
                F.col("_2").alias('name'),
                F.lit("g").alias("s")
            ).toJSON().collect()

# Run base

In [ ]:
class Run:
    def __init__(self,samples):
        self.samples=samples
    
    def generate_trace_view(self,appid,**kwargs):
        traces=[]
        
        for idx, s in enumerate(self.samples):
            traces.extend(s.generate_trace_view_list(idx,**kwargs))        
        output='''
        {
            "traceEvents": [
        
        ''' + \
        ",\n".join(traces)\
       + '''
            ]
        }'''

        with open('/home/yuzhou/trace_result/'+appid+'.json', 'w') as outfile:  
            outfile.write(output)

        print("http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/"+appid+".json")

# Dask Application Run

In [ ]:
class Dask_Application_Run:
    def __init__(self, appid):
        self.appid=appid
        self.filedir="/tmp/dgx-2Log/"+self.appid+"/"
        
        self.analysis={
            'dask':{'als':dask_analysis(self.filedir+"cluster.log"),'pid':8000},
            'sar_cpu':{'als':Sar_cpu_analysis(self.filedir + "/"+"sar_cpu.sar"),'pid':10*0+0},
            'sar_disk':{'als':Sar_disk_analysis(self.filedir + "/"+"sar_disk.sar"),'pid':10*0+1},
            'sar_mem':{'als':Sar_mem_analysis(self.filedir + "/"+"sar_mem.sar"),'pid':10*0+2},
            'sar_nic':{'als':Sar_nic_analysis(self.filedir  + "/"+"sar_nic.sar"),'pid':10*0+3},
            'emon':{'als':Emon_Analysis(self.filedir +  "/"+"emon.rst"),'pid':10*0+4},
            'gpu':{'als':gpu_analysis(self.filedir + "/gpu.txt"),'pid':10*0+5},
        }
        
    
    def generate_trace_view(self,showsar=True,showemon=False,showgpu=True,**kwargs):
        traces=[]
        daskals=self.analysis['dask']['als']
        traces.extend(daskals.generate_trace_view_list(self.analysis['dask']['pid'],**kwargs))
        if showsar:
            sarals=self.analysis['sar_cpu']['als']
            sarals.starttime=daskals.starttime
            traces.extend(sarals.generate_trace_view_list(self.analysis['sar_cpu']['pid'],**kwargs))
            sarals=self.analysis['sar_disk']['als']
            sarals.starttime=daskals.starttime
            traces.extend(sarals.generate_trace_view_list(self.analysis['sar_disk']['pid'],**kwargs))
            sarals=self.analysis['sar_mem']['als']
            sarals.starttime=daskals.starttime
            traces.extend(sarals.generate_trace_view_list(self.analysis['sar_mem']['pid'],**kwargs))
            sarals=self.analysis['sar_nic']['als']
            sarals.starttime=daskals.starttime
            traces.extend(sarals.generate_trace_view_list(self.analysis['sar_nic']['pid'],**kwargs))
        if showemon:
            emonals=self.analysis['emon']['als']
            emonals.starttime=daskals.starttime
            traces.extend(emonals.generate_trace_view_list(self.analysis['emon']['pid'],**kwargs))
        if showgpu:
            gpuals=self.analysis['gpu']['als']
            gpuals.starttime=daskals.starttime
            traces.extend(gpuals.generate_trace_view_list(self.analysis['gpu']['pid'],**kwargs))
        
        output='''
        {
            "traceEvents": [
        
        ''' + \
        ",\n".join(traces)\
       + '''
            ]
        }'''

        with open('/home/yuzhou/trace_result/'+self.appid+'.json', 'w') as outfile:  
            outfile.write(output)

        print("http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/"+self.appid+".json")

In [ ]:
from datetime import datetime
datetime.fromtimestamp(1546439400)

In [ ]:
class Dask_Application_Run2:
    def __init__(self, appid):
        self.appid=appid
        
        self.filedir="/tmp/dgx-2Log/"+self.appid+"/"
        self.dask=self.load_dask()
        self.sar=self.load_sar()
        self.gpu=self.load_gpu()
        
    
    def load_dask(self):
        return dask_analysis(self.filedir+"cluster.log")
    
    def load_sar(self):
        return Sar_analysis(self.filedir+"sar_data.sar")
        
    def load_emon(self):
        return Emon_Analysis(self.filedir+"emon.rst")
    
    def load_gpu(self):
        return gpu_dmon_analysis(self.filedir+"gpu_dmon.txt")
    
    def generate_dask_trace_view(self):
        return self.dask.generate_dask_trace_view(8000)
    
    def generate_sar_trace_view(self):
        return self.sar.generate_sar_trace_view(0)
    
    def generate_gpu_trace_view(self):
        return self.gpu.generate_gpu_trace_view(1)

    def generate_emon_trace_view(self,collected_cores):
        return self.emon.generate_emon_trace_view(5,collected_cores)
    
    def generate_trace_view(self,showsar=True,showemon=False,showgpu=True):
        traces=[]
        traces.extend(self.generate_dask_trace_view())
        if showsar:
            self.sar.starttime=self.dask.starttime
            traces.extend(self.generate_sar_trace_view())
        if showemon:
            traces.extend(self.generate_emon_trace_view(collected_cores))
        if showgpu:
            self.gpu.starttime=self.dask.starttime
            traces.extend(self.generate_gpu_trace_view())
        
        output='''
        {
            "traceEvents": [
        
        ''' + \
        ",\n".join(traces)\
       + '''
            ]
        }'''

        with open('/home/yuzhou/trace_result/'+self.appid+'.json', 'w') as outfile:  
            outfile.write(output)

        print("http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/"+self.appid+".json")

# Application RUN STD

In [ ]:
class Application_Run_STD:
    def __init__(self, appid):
        self.appid=appid
        self.filedir="/tmp/dgx-2Log/"+self.appid+"/"
        
        self.analysis={
            'sar':{'als':Sar_analysis(self.filedir+"sar_data.sar"),'pid':0},
            'emon':{'als':Emon_Analysis(self.filedir+"emon.rst"),'pid':1},
            'gpu':{'als':gpu_analysis(self.filedir+"gpu.txt"),'pid':100},
        }
        
    
    def generate_trace_view(self,showsar=True,showemon=False,showgpu=True,**kwargs):
        traces=[]
        starttime=time.time()*1000
        if showsar:
            sarals=self.analysis['sar']['als']
            sarals.starttime=starttime
            traces.extend(sarals.generate_trace_view_list(self.analysis['sar']['pid'],**kwargs))
        if showemon:
            emonals=self.analysis['emon']['als']
            emonals.starttime=starttime
            traces.extend(emonals.generate_trace_view_list(self.analysis['emon']['pid'],**kwargs))
        if showgpu:
            gpuals=self.analysis['gpu']['als']
            gpuals.starttime=starttime
            traces.extend(gpuals.generate_trace_view_list(self.analysis['gpu']['pid'],**kwargs))
        
        output='''
        {
            "traceEvents": [
        
        ''' + \
        ",\n".join(traces)\
       + '''
            ]
        }'''

        with open('/home/yuzhou/trace_result/'+self.appid+'.json', 'w') as outfile:  
            outfile.write(output)

        print("http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/"+self.appid+".json")

# application Run

In [1]:




class Application_Run:
    def __init__(self, appid,**kwargs):
        self.appid=appid
        
        basedir=kwargs.get("basedir","skylake")
        self.filedir="/"+basedir+"/"+self.appid+"/"
        self.basedir=basedir
        
        slaves=fs.list_status("/"+basedir+"/"+appid)
        slaves=[f['pathSuffix'] for f in slaves if f['type']=='DIRECTORY']
        
        jobids=kwargs.get("jobids",None)
        
        self.clients=slaves
        
        sarclnt={}
        for idx,l in enumerate(self.clients):
            sarclnt[l]={'sar_cpu':{'als':Sar_cpu_analysis(self.filedir + l + "/"+"sar_cpu.sar"),'pid':idx},
                'sar_disk':{'als':Sar_disk_analysis(self.filedir + l + "/"+"sar_disk.sar"),'pid':idx},
                'sar_mem':{'als':Sar_mem_analysis(self.filedir + l + "/"+"sar_mem.sar"),'pid':idx},
                'sar_nic':{'als':Sar_nic_analysis(self.filedir + l + "/"+"sar_nic.sar"),'pid':idx}
                
            }
            if fs.exists(self.filedir + l + "/pidstat.out"):
                sarclnt[l]['sar_pid']={'als':Pidstat_analysis(self.filedir + l + "/pidstat.out"),'pid':idx}
            if fs.exists(self.filedir + l + "/sched.txt"):
                sarclnt[l]['sar_perf']={'als':Perf_trace_analysis(self.filedir + l + "/sched.txt"),'pid':100+idx}
            if fs.exists(self.filedir + l + "/emon.rst"):
                sarclnt[l]['emon']={'als':Emon_Analysis(self.filedir + l + "/emon.rst"),'pid':200+idx}
            if fs.exists(self.filedir + l + "/perfstat.txt"):
                sarclnt[l]['perfstat']={'als':Perfstat_analysis(self.filedir + l + "/perfstat.txt"),'pid':300+idx}
            if fs.exists(self.filedir + l + "/gpu.txt"):
                sarclnt[l]['gpu']={'als':gpu_analysis(self.filedir + l + "/gpu.txt"),'pid':400+idx}
            
                
        self.analysis={
            "sar": sarclnt
        }
        
        if fs.exists(self.filedir+"app.log"):
            self.analysis['app']={'als':App_Log_Analysis(self.filedir+"app.log",jobids)}
        
        if fs.exists(self.filedir+"instevent.out"):
            self.analysis['instant']={'als':InstantEvent_analysis(self.filedir+"instevent.out")}
        
        self.starttime=0
        if fs.exists(self.filedir+"starttime"):
            with fs.open(self.filedir+"starttime") as f:
                st = f.read().decode('ascii')
                self.starttime=int(st)
            
        
    
    def generate_trace_view(self,showsar=True,showemon=False,showgpu=True,**kwargs):
        traces=[]
        shownodes=kwargs.get("shownodes",self.clients)
        for l in shownodes:
            if l not in self.clients:
                print(l,"is not in clients",self.clients)
                return
        self.clients=shownodes
        
        xgbtcks=kwargs.get('xgbtcks',("calltrain",'enter','begin','end'))
        
        if "app" in self.analysis:
            appals=self.analysis['app']['als']
            appals.starttime=self.starttime
            traces.extend(appals.generate_trace_view_list(self.analysis['app'],**kwargs))
            self.starttime=appals.starttime
        
        if 'instant' in self.analysis:
            als=self.analysis['instant']['als']
            als.starttime=self.starttime
            traces.extend(als.generate_trace_view_list(**kwargs))
        
        counttime=kwargs.get("counttime",False)
        
        pidmap={}
        if showsar:
            for l in self.clients:
                for alskey, sarals in self.analysis["sar"][l].items():
                    t1 = time.time()
                    if alskey!="emon":
                        sarals['als'].starttime=self.starttime
                        traces.extend(sarals['als'].generate_trace_view_list(sarals['pid'],node=l, **kwargs))
                    elif showemon:
                        sarals['als'].load_data()
                        pidmap[l]=sarals['pid']
                    if counttime:
                        print(l,alskey," spend time: ", time.time()-t1)
        if showemon:
            t1 = time.time()
            emondfs=get_emon_parquets([self.appid,],self.basedir)
            emons=Emon_Analysis_All(emondfs)
            emons.starttime=self.starttime
            traces.extend(emons.generate_trace_view_list(0,pidmap=pidmap,**kwargs))
            if counttime:
                print("emon process spend time: ", time.time()-t1)
            self.emons=emons
        
        for idx,l in enumerate(self.clients):
            traces.append(json.dumps({"name": "process_sort_index","ph": "M","pid":idx,"tid":0,"args":{"sort_index ":idx}}))
            traces.append(json.dumps({"name": "process_sort_index","ph": "M","pid":idx+100,"tid":0,"args":{"sort_index ":idx+100}}))
            traces.append(json.dumps({"name": "process_sort_index","ph": "M","pid":idx+200,"tid":0,"args":{"sort_index ":idx+200}}))
        
        if "app" in self.analysis:
            for pid in self.analysis['app']['als'].pids:
                traces.append(json.dumps({"name": "process_sort_index","ph": "M","pid":pid+200,"tid":0,"args":{"sort_index ":pid+200}}))

        allcnt=""
        for c in self.clients:
            paths=self.filedir+c
            if fs.exists(paths+"/xgbtck.txt"):
                with fs.open(paths+"/xgbtck.txt") as f:
                    tmp = f.read().decode('ascii')
                    allcnt=allcnt+tmp
        allcnt=allcnt.strip().split("\n")
        if len(allcnt) > 1:
            allcnt=[l.split(" ") for l in allcnt]
            cnts=pandas.DataFrame([[l[0],l[1],l[2],l[3]] for l in allcnt if len(l)>1 and l[1] in xgbtcks])
            if len(cnts) > 0:
                cnts.columns=['xgbtck','name','rank','time']
                cntgs=cnts.groupby("name").agg({"time":"min"})
                cntgs=cntgs.reset_index()
                cntgs.columns=['name','ts']
                cntgs['ph']="i"
                cntgs['ts']=pandas.to_numeric(cntgs['ts'])-self.starttime
                cntgs['pid']=0
                cntgs['tid']=0
                cntgs['s']='g'
                traces.extend([json.dumps(l) for l in cntgs.to_dict(orient='records')])
        
        output='''
        {
            "traceEvents": [
        
        ''' + \
        ",\n".join(traces)\
       + '''
            ],
            "displayTimeUnit": "ns"
        }'''

        with open('/home/yuzhou/trace_result/'+self.appid+'.json', 'w') as outfile:  
            outfile.write(output)

        print("http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/"+self.appid+".json")
        
    def getemonmetric(app,emonmetric):
        emondfs=get_emon_parquets([app.appid],app.basedir)
        emons=Emon_Analysis_All(emondfs)
        metric_msg_map={
            'emon_cpuutil':F.avg,
            'emon_cpufreq':F.avg,
            'emon_instr_retired':F.sum,
            'emon_ipc':F.avg,
            'emon_L3 stall':F.avg,
            'emon_mem stall':F.avg,
            'emon_total stall':F.avg,
            'emon_dtlb walk':F.avg,
            'emon_uop issue stall':F.avg,
            'emon_mem_bw':F.avg,
            'emon_remote ratio':F.avg,
            'emon_local lat':F.avg,
            'emon_remote lat':F.avg,
            'emon_l3 miss':F.avg,
            'emon_l3 read miss bw':F.avg,
            'emon_pcie_bw':F.avg
        }

        outdf=None
        for k in emonmetric:
            m=emons.emon_metrics[k]
            for fk,fm in m['formula'].items():
                df=emons.gen_reduce_metric(k,list(range(0,emons.totalcores)),fk,metric_msg_map[k])
                tmpdf=df.groupBy("appid",'client').agg(*[l("`{:s}`".format(fk)).alias(get_alias_name(fk,l)) for l in [metric_msg_map[k]]]).toPandas()
                tmpdf=tmpdf.set_index("client").drop(columns=['appid']).T
                if outdf is None:
                    outdf=tmpdf
                else:
                    outdf=outdf.append(tmpdf)
        pandas.options.display.float_format = '{:,.2f}'.format
        return outdf
    
    def get_sar_stat(app_ww44,**kwargs):
        disk_prefix=kwargs.get("disk_prefix","dev259")
        nic_prefix = kwargs.get("nic_prefix",["'eth3'","'enp24s0f1'"])
        cpustat=[app_ww44.analysis["sar"][l]['sar_cpu']['als'].get_stat() for l in app_ww44.clients]
        cpustat=reduce(lambda l,r:l.join(r),cpustat)
        diskstat=[app_ww44.analysis["sar"][l]['sar_disk']['als'].get_stat(disk_prefix=disk_prefix) for l in app_ww44.clients]
        diskstat=reduce(lambda l,r:l.join(r),diskstat)
        memstat=[app_ww44.analysis["sar"][l]['sar_mem']['als'].get_stat() for l in app_ww44.clients]
        memstat=reduce(lambda l,r:l.join(r),memstat)
        nicstat=[app_ww44.analysis["sar"][l]['sar_nic']['als'].get_stat(nic_prefix=nic_prefix) for l in app_ww44.clients]
        nicstat=reduce(lambda l,r:l.join(r),nicstat)
        pandas.options.display.float_format = '{:,.2f}'.format
        return pandas.concat([cpustat,diskstat,memstat,nicstat])
    
    def compare_app(app2,**kwargs):
        output=[]
        
        lbasedir=kwargs.get("basedir",app2.basedir)
        r_appid=kwargs.get("r_appid",app2.appid)
        
        app=kwargs.get("rapp",Application_Run(r_appid,basedir=lbasedir))

        show_queryplan_diff=kwargs.get("show_queryplan_diff",True)
        
        appals=app.analysis["app"]["als"]
        appals2=app2.analysis["app"]["als"]

        out=appals.get_query_time(plot=False)
        out2=appals2.get_query_time(plot=False)

        lrun=app.appid
        rrun=app2.appid
        cmpcolumns=['runtime','shuffle_write','f_wait_time','input read','acc_task_time','output rows']
        outcut=out[cmpcolumns]
        out2cut=out2[cmpcolumns]
        cmp=outcut.join(out2cut,lsuffix='_'+lrun,rsuffix='_'+rrun)

        pdsout=pandas.DataFrame(outcut.sum(),columns=[lrun])
        pdsout2=pandas.DataFrame(out2cut.sum(),columns=[rrun])
        pdstime=pdsout.join(pdsout2)

        print("emon metric")
        
        emondf=app.getemonmetric(emonmetric=emonmetric)
        emondf2=app2.getemonmetric(emonmetric=emonmetric)
        #in case we comare with two clsuter
        emondf.columns=emondf2.columns
        def get_agg(emondf):
            aggs=[]
            for x in emondf.index:
                if x.endswith("avg"):
                    aggs.append(emondf.loc[x].mean())
                else:
                    aggs.append(emondf.loc[x].sum())

            emondf['agg']=aggs
            return emondf
        emondf=get_agg(emondf)
        emondf2=get_agg(emondf2)

        emoncolumns=emondf.columns
        emoncmp=emondf.join(emondf2,lsuffix='_'+lrun,rsuffix='_'+rrun)
        emonsum=emoncmp[["agg_"+lrun,"agg_"+rrun]]

        emonsum.columns=[lrun,rrun]

        print("sar metric")
        sardf=app.get_sar_stat(**kwargs)
        sardf2=app2.get_sar_stat(**kwargs)
        
        def get_sar_agg(sardf):
            aggs=[]
            for x in sardf.index:
                if "total" in x:
                    aggs.append(sardf.loc[x].sum())
                elif "max" in x:
                    aggs.append(sardf.loc[x].max())
                else:
                    aggs.append(sardf.loc[x].mean())

            sardf['agg']=aggs
            return sardf
        sardf=get_sar_agg(sardf)
        sardf2=get_sar_agg(sardf2)
        #in case we compare two clusters
        sardf2.columns=sardf.columns

        sarcolumns=sardf.columns
        sarcmp=sardf.join(sardf2,lsuffix='_'+lrun,rsuffix='_'+rrun)
        sarsum=sarcmp[["agg_"+lrun,"agg_"+rrun]]

        sarsum.columns=[lrun,rrun]
        
        
        summary=pandas.concat([pdstime,sarsum,emonsum])
        summary["diff"]=summary[lrun]/summary[rrun]-1
        def highlight_diff(x):
            styles=[]
            mx=x.max()
            mn=x.min()
            mx=max(mx,-mn,0.2)
            for j in x.index:
                m1=(x[j])/mx*100 if x[j]!=None else 0
                if m1>0:
                    styles.append(f'width: 400px ; background-image: linear-gradient(to right, transparent 50%, #5fba7d 50%, #5fba7d {50+m1/2}%, transparent {50+m1/2}%)')
                else:
                    styles.append(f'width: 400px ;background-image: linear-gradient(to left, transparent 50%, #f1a863 50%, #f1a863 {50-m1/2}%, transparent {50-m1/2}%)')
            return styles

        output.append(summary.style.apply(highlight_diff,subset=['diff']).format({lrun:"{:,.2f}",rrun:"{:,.2f}",'diff':"{:,.2%}"}).render())

        cmp_plot=cmp
        cmp_plot['diff']=cmp_plot['runtime_'+lrun]-cmp_plot['runtime_'+rrun]

        pltx=cmp_plot.sort_values(by='diff',axis=0).plot.bar(y=['runtime_'+lrun,'runtime_'+rrun],figsize=(30,8))
        better_num=sqldf('''select count(*) from cmp_plot where diff>0''')['count(*)'][0]
        pltx.text(0.1, 0.8,'{:d} queries are better'.format(better_num), ha='center', va='center', transform=pltx.transAxes)

        df1 = pandas.DataFrame('', index=cmp.index, columns=cmpcolumns)
        for l in cmpcolumns:
            for j in cmp.index:
                df1[l][j]=[cmp[l+"_"+lrun][j],cmp[l+"_"+rrun][j],cmp[l+"_"+lrun][j]/cmp[l+"_"+rrun][j]-1]

        def highlight_greater(x,columns):
            df1 = pandas.DataFrame('', index=x.index, columns=x.columns)
            for l in columns:
                m={}
                for j in x.index:
                    m[j] = (x[l][j][1] / x[l][j][0])*100 if x[l][j][0]!=0 else 100
                mx=max(m.values())-100
                mn=100-min(m.values())
                mx=max(mx,mn)
                for j in x.index:
                    m1=-(100-m[j])/mx*100 if x[l][j][0]!=0 else 0
                    if m1>0:
                        df1[l][j] = f'background-image: linear-gradient(to right, transparent 50%, #5fba7d 50%, #5fba7d {50+m1/2}%, transparent {50+m1/2}%)'
                    else:
                        df1[l][j] = f'background-image: linear-gradient(to left, transparent 50%, #f1a863 50%, #f1a863 {50-m1/2}%, transparent {50-m1/2}%)'

            return df1

        def display_compare(df,columns):
            output.append(df.style.set_properties(**{'width': '300px','border-style':'solid','border-width':'1px'}).apply(lambda x: highlight_greater(x,columns), axis=None).format(lambda x: '''
                                                                          <div style='max-width: 30%; min-width:30%;display:inline-block;'>{:,.2f}</div>
                                                                          <div style='max-width: 30%; min-width:30%; display:inline-block;'>{:,.2f}</div>
                                                                          <div style='max-width: 30%; min-width:30%; display:inline-block;color:blue'>{:,.2f}%</div>
                                                                       '''.format(x[0],x[1],x[2]*100)).render())
        display_compare(df1,cmpcolumns)

        df3 = pandas.DataFrame('', index=sarcmp.index, columns=sarcolumns)
        for l in sarcolumns:
            for j in df3.index:
                df3[l][j]=[sarcmp[l+"_"+lrun][j],sarcmp[l+"_"+rrun][j],sarcmp[l+"_"+lrun][j]/sarcmp[l+"_"+rrun][j]-1]
        display_compare(df3,sarcolumns)

        
        df2 = pandas.DataFrame('', index=emoncmp.index, columns=emoncolumns)
        for l in emoncolumns:
            for j in df2.index:
                df2[l][j]=[emoncmp[l+"_"+lrun][j],emoncmp[l+"_"+rrun][j],emoncmp[l+"_"+lrun][j]/emoncmp[l+"_"+rrun][j]-1]
        display_compare(df2,emoncolumns)

        print("time breakdown")
        ################################ time breakdown ##################################################################################################
        timel=appals.show_time_metric(plot=False)
        timer=appals2.show_time_metric(plot=False)
        timer.columns=[l.replace("scan time","totaltime_batchscan") for l in timer.columns]
        timel.columns=[l.replace("scan time","totaltime_batchscan") for l in timel.columns]
        rcols=timer.columns
        lcols=[]
        for c in [l.split("%")[1][1:] for l in rcols]:
            for t in timel.columns:
                if t.endswith(c):
                    lcols.append(t)
        for t in timel.columns:
            if t not in lcols:
                lcols.append(t)
        timel_adj=timel[lcols]

        fig, axs = plt.subplots(nrows=1, ncols=2, sharey=True,figsize=(30,8),gridspec_kw = {'width_ratios':[1, 1]})
        plt.subplots_adjust(wspace=0.01)
        ax=timel_adj.plot.bar(ax=axs[0],stacked=True)
        list_values=timel_adj.loc[0].values
        for rect, value in zip(ax.patches, list_values):
            h = rect.get_height() /2.
            w = rect.get_width() /2.
            x, y = rect.get_xy()
            ax.text(x+w, y+h,"{:,.2f}".format(value),horizontalalignment='center',verticalalignment='center',color="white")
        ax=timer.plot.bar(ax=axs[1],stacked=True)
        list_values=timer.loc[0].values
        for rect, value in zip(ax.patches, list_values):
            h = rect.get_height() /2.
            w = rect.get_width() /2.
            x, y = rect.get_xy()
            ax.text(x+w, y+h,"{:,.2f}".format(value),horizontalalignment='center',verticalalignment='center',color="white")

################################ critical time breakdown ##################################################################################################
        timel=appals.show_time_metric(plot=False,taskids=[l[0].item() for l in appals.criticaltasks])
        timer=appals2.show_time_metric(plot=False,taskids=[l[0].item() for l in appals2.criticaltasks])
        timer.columns=[l.replace("scan time","totaltime_batchscan") for l in timer.columns]
        timel.columns=[l.replace("scan time","totaltime_batchscan") for l in timel.columns]
        rcols=timer.columns
        lcols=[]
        for c in [l.split("%")[1][1:] for l in rcols]:
            for t in timel.columns:
                if t.endswith(c):
                    lcols.append(t)
        for t in timel.columns:
            if t not in lcols:
                lcols.append(t)
        timel_adj=timel[lcols]

        fig, axs = plt.subplots(nrows=1, ncols=2, sharey=True,figsize=(30,8),gridspec_kw = {'width_ratios':[1, 1]})
        plt.subplots_adjust(wspace=0.01)
        ax=timel_adj.plot.bar(ax=axs[0],stacked=True)
        list_values=timel_adj.loc[0].values
        for rect, value in zip(ax.patches, list_values):
            h = rect.get_height() /2.
            w = rect.get_width() /2.
            x, y = rect.get_xy()
            ax.text(x+w, y+h,"{:,.2f}".format(value),horizontalalignment='center',verticalalignment='center',color="white")
        ax=timer.plot.bar(ax=axs[1],stacked=True)
        list_values=timer.loc[0].values
        for rect, value in zip(ax.patches, list_values):
            h = rect.get_height() /2.
            w = rect.get_width() /2.
            x, y = rect.get_xy()
            ax.text(x+w, y+h,"{:,.2f}".format(value),horizontalalignment='center',verticalalignment='center',color="white")


        ################################ hot stage ##########################################################################################################

        hotstagel=appals.get_hottest_stages(plot=False)
        hotstager=appals2.get_hottest_stages(plot=False)
        hotstagel.style.format(lambda x: '''{:,.2f}'''.format(x))

        norm = matplotlib.colors.Normalize(vmin=0, vmax=max(hotstager.queryid))
        cmap = matplotlib.cm.get_cmap('brg')
        def setbkcolor(x):
            rgba=cmap(norm(x['queryid']))
            return ['background-color:rgba({:d},{:d},{:d},1); color:white'.format(int(rgba[0]*255),int(rgba[1]*255),int(rgba[2]*255))]*9

        output.append("<table><tr><td>" + hotstagel.style.apply(setbkcolor,axis=1).format({"total_time":lambda x: '{:,.2f}'.format(x),"stdev_time":lambda x: '{:,.2f}'.format(x),"acc_total":lambda x: '{:,.2%}'.format(x),"total":lambda x: '{:,.2%}'.format(x)}).render()+
             "</td><td>" +  hotstager.style.apply(setbkcolor,axis=1).format({"total_time":lambda x: '{:,.2f}'.format(x),"stdev_time":lambda x: '{:,.2f}'.format(x),"acc_total":lambda x: '{:,.2%}'.format(x),"total":lambda x: '{:,.2%}'.format(x)}).render()+             "</td></tr></table>")

        if not show_queryplan_diff:
            return "\n".join(output)
        
        print("hot stage")

        loperators=appals.getOperatorCount()
        roperators=appals2.getOperatorCount()
        loperators_rowcnt=appals.get_metric_output_rowcnt()
        roperators_rowcnt=appals2.get_metric_output_rowcnt()
        
        def show_query_diff(queryid):
            lops=pandas.DataFrame(loperators[queryid])
            lops.columns=['calls_l']
            lops=lops.loc[lops['calls_l'] >0]

            rops=pandas.DataFrame(roperators[queryid])
            rops.columns=["calls_r"]
            rops=rops.loc[rops['calls_r'] >0]
            lops_row=pandas.DataFrame(loperators_rowcnt[queryid])
            lops_row.columns=["rows_l"]
            lops_row=lops_row.loc[lops_row['rows_l'] >0]

            rops_row=pandas.DataFrame(roperators_rowcnt[queryid])
            rops_row.columns=["rows_r"]
            rops_row=rops_row.loc[rops_row['rows_r'] >0]

            opscmp=pandas.merge(pandas.merge(pandas.merge(lops,rops,how="outer",left_index=True,right_index=True),lops_row,how="outer",left_index=True,right_index=True),rops_row,how="outer",left_index=True,right_index=True)
            opscmp=opscmp.fillna("")

            def set_bk_color_opscmp(x):
                calls_l= 0 if x['calls_l']=="" else x['calls_l']
                calls_r= 0 if x['calls_r']=="" else x['calls_r']
                rows_l= 0 if x['rows_l']=="" else x['rows_l']
                rows_r= 0 if x['rows_r']=="" else x['rows_r']

                if calls_l > calls_r or rows_l > rows_r:
                    return ['background-color:#eb6b34']*4
                if calls_l < calls_r or rows_l < rows_r:
                    return ['background-color:#8ad158']*4
                return ['color:#dbd4d0']*4

            output.append(opscmp.style.apply(set_bk_color_opscmp,axis=1).render())

            planl=appals.get_query_plan(queryid=queryid,show_plan_only=True,plot=False)
            planr=appals2.get_query_plan(queryid=queryid,show_plan_only=True,plot=False)
            output.append("<table><tr><td>"+planl+"</td><td>"+planr+"</td></tr></table>")

        outputx=df1['output rows']
        runtimex = df1['runtime']
        for x in outputx.index:
            if outputx[x][0]!=outputx[x][1] or runtimex[x][0]/runtimex[x][1]<0.95 or runtimex[x][0]/runtimex[x][1]>1.05:
                print("query ",x," queryplan diff ")
                output.append(f"<p><font size=4 color=red>query{x} is different,{lrun} time: {df1['runtime'][x][0]}, {rrun} time: {df1['runtime'][x][1]}</font></p>")
                show_query_diff(x)
        return "\n".join(output)
                              
    def show_queryplan_diff(app2, queryid,**kwargs):
        lbasedir=kwargs.get("basedir",app2.basedir)
        r_appid=kwargs.get("r_appid",app2.appid)
        
        app=kwargs.get("rapp",Application_Run(r_appid,basedir=lbasedir))

        appals=app.analysis["app"]["als"]
        appals2=app2.analysis["app"]["als"]

        hotstagel=appals.get_hottest_stages(plot=False)
        hotstager=appals2.get_hottest_stages(plot=False)
        hotstagel.style.format(lambda x: '''{:,.2f}'''.format(x))

        loperators=appals.getOperatorCount()
        roperators=appals2.getOperatorCount()
        loperators_rowcnt=appals.get_metric_output_rowcnt()
        roperators_rowcnt=appals2.get_metric_output_rowcnt()

        lrun=app.appid
        rrun=app2.appid

        output=[]

        def show_query_diff(queryid):
            lops=pandas.DataFrame(loperators[queryid])
            lops.columns=['calls_l']
            lops=lops.loc[lops['calls_l'] >0]

            rops=pandas.DataFrame(roperators[queryid])
            rops.columns=["calls_r"]
            rops=rops.loc[rops['calls_r'] >0]
            lops_row=pandas.DataFrame(loperators_rowcnt[queryid])
            lops_row.columns=["rows_l"]
            lops_row=lops_row.loc[lops_row['rows_l'] >0]

            rops_row=pandas.DataFrame(roperators_rowcnt[queryid])
            rops_row.columns=["rows_r"]
            rops_row=rops_row.loc[rops_row['rows_r'] >0]

            opscmp=pandas.merge(pandas.merge(pandas.merge(lops,rops,how="outer",left_index=True,right_index=True),lops_row,how="outer",left_index=True,right_index=True),rops_row,how="outer",left_index=True,right_index=True)
            opscmp=opscmp.fillna("")

            def set_bk_color_opscmp(x):
                calls_l= 0 if x['calls_l']=="" else x['calls_l']
                calls_r= 0 if x['calls_r']=="" else x['calls_r']
                rows_l= 0 if x['rows_l']=="" else x['rows_l']
                rows_r= 0 if x['rows_r']=="" else x['rows_r']

                if calls_l > calls_r or rows_l > rows_r:
                    return ['background-color:#eb6b34']*4
                if calls_l < calls_r or rows_l < rows_r:
                    return ['background-color:#8ad158']*4
                return ['color:#dbd4d0']*4

            output.append(opscmp.style.apply(set_bk_color_opscmp,axis=1).render())

            planl=appals.get_query_plan(queryid=queryid,show_plan_only=True,plot=False)
            planr=appals2.get_query_plan(queryid=queryid,show_plan_only=True,plot=False)
            output.append("<table><tr><td>"+planl+"</td><td>"+planr+"</td></tr></table>")

        x=queryid
        print("query ",x," queryplan diff ")
        #output.append(f"<p><font size=4 color=red>query{x} is different,{lrun} time: {df1['runtime'][x][0]}, {rrun} time: {df1['runtime'][x][1]}</font></p>")
        show_query_diff(x)
        display(HTML("\n".join(output)))
        return

# MISC

In [17]:
def show_rst(pdrst):
    html='''<style type="text/css">
    .tg  {border-collapse:collapse;border-spacing:0;border-color:#aabcfe;}
    .tg td{font-family:Courier New;font-size:18px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:#aabcfe;color:#669;background-color:#e8edff;}
    .tg th{font-family:Courier New;font-size:18px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:#aabcfe;color:#039;background-color:#b9c9fe;}
    .tg .tg-phtq{background-color:#D2E4FC;border-color:inherit;text-align:left;vertical-align:top}
    .tg .tg-c3ow{border-color:inherit;text-align:center;vertical-align:top}
    .tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
    .tg .tg-phtq_v{background-color:#D2E4FC;border-color:inherit;text-align:left;vertical-align:top; color:#FF0000}
    .tg .tg-0pky_v{border-color:inherit;text-align:left;vertical-align:top; color:#FF0000}
    </style>
    <table class="tg">
      <tr>
    '''
    cols=pdrst.columns  
    html=html+''.join(['<th class="tg-c3ow">{:s}</th>'.format(l) for l in cols if l!='app_id'])
    html=html+'<th class="tg-c3ow">spark log</th><th class="tg-c3ow">trace_view</th></tr>'
    for idx, r in pdrst.iterrows():
        html=html+"<tr>"
        html=html+"".join(['<td class="{:s}">{:s}</td>'.format('tg-phtq' if l!='elapsed time' else 'tg-phtq_v', str(r[l])) for l in cols if l!='app_id'])
        html=html+'''<td class="tg-phtq"><a href="http://10.1.2.107:18080/history/{:s}/jobs"> {:s}</a></td><td class="tg-phtq"><a href=http://sr525:1088/tracing_examples/trace_viewer.html#/tracing/test_data/{:s}.json> {:s}</a></td>'''.format(r['app_id'].appid,r['app_id'].appid,r['app_id'].appid,r['app_id'].appid)
        html=html+"</tr>"
    html=html+"</table>"
    display(HTML(html))

In [ ]:
def reduce_metric(pdrst,slave_id,metric,core,agg_func):
    pdrst['rst']=pdrst.apply(lambda x:x['app_id'].get_reduce_metric(slave_id,metric,core,agg_func), axis=1)
    for l in agg_func:
        pdrst[get_alias_name(metric,l)]=pdrst.apply(lambda x:x['rst'].iloc[0][get_alias_name(metric,l)],axis=1)
    return pdrst.drop(columns=['rst'])

In [ ]:
def clean_data(rsta):
    for i in range(0,r):
        m=rsta.loc[i,'emon_mem_bw']
        if m>40000:
            for j in range(i,i+5):
                if j>=r:
                    break
                if rsta.loc[j,'emon_mem_bw']<40000:
                    m=0
                    break
            rsta.loc[i,'emon_mem_bw']=m
        else:
            rsta.loc[i,'emon_mem_bw']=False

# TPCDS query map

In [15]:
m='''1	q01
    2	q02
    3	q03
    4	q04
    5	q05
    6	q06
    7	q07
    8	q08
    9	q09
    10	q10
    11	q11
    12	q12
    13	q13
    14	q14a
    15	q14b
    16	q15
    17	q16
    18	q17
    19	q18
    20	q19
    21	q20
    22	q21
    23	q22
    24	q23a
    25	q23b
    26	q24a
    27	q24b
    28	q25
    29	q26
    30	q27
    31	q28
    32	q29
    33	q30
    34	q31
    35	q32
    36	q33
    37	q34
    38	q35
    39	q36
    40	q37
    41	q38
    42	q39a
    43	q39b
    44	q40
    45	q41
    46	q42
    47	q43
    48	q44
    49	q45
    50	q46
    51	q47
    52	q48
    53	q49
    54	q50
    55	q51
    56	q52
    57	q53
    58	q54
    59	q55
    60	q56
    61	q57
    62	q58
    63	q59
    64	q60
    65	q61
    66	q62
    67	q63
    68	q64
    69	q65
    70	q66
    71	q67
    72	q68
    73	q69
    74	q70
    75	q71
    76	q72
    77	q73
    78	q74
    79	q75
    80	q76
    81	q77
    82	q78
    83	q79
    84	q80
    85	q81
    86	q82
    87	q83
    88	q84
    89	q85
    90	q86
    91	q87
    92	q88
    93	q89
    94	q90
    95	q91
    96	q92
    97	q93
    98	q94
    99	q95
    100	q96
    101	q97
    102	q98
    103	q99'''.split("\n")
tpcds_query_map=[l.strip().split("\t") for l in m]
tpcds_query_map={int(l[0]):l[1] for l in tpcds_query_map}